In [ ]:
## This file implements neural networks before and after lasso selection for p0006presabs_qual with four replicates.
## We compute the mean and standarad deviation of training and test accuracies.
## We also compute the mean and standard deviation of AUC ROC values for each model.

In [1]:
from numpy.random import seed
import numpy as np
seed(100)
import tensorflow
tensorflow.random.set_seed(123)

In [2]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/p0006presabs_qual.csv')
df.shape

(253, 96)

In [3]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [4]:
df['pheno']

0      1
1      0
2      1
3      1
4      1
      ..
248    1
249    0
250    0
251    0
252    0
Name: pheno, Length: 253, dtype: int64

In [5]:
df.head()

,id,TTGTTATAGTC,TTAATTTAATAGA,TTAACATAATAAT,TGCAATCTCTTTAT,TATTATGTTAATG,TACATACCGAT,GTGTATCATAAT,GCTGTTGAAATGGC,GCAAACATGCG,...,group_7824,group_7845,group_10140,group_10143,group_3985,group_4427,group_7172,group_8210,group_9586,pheno
0,107,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1
1,109,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,115,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1
3,120335,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1
4,120337,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1


In [6]:
df['pheno'].value_counts()

0    129
1     85
2     39
Name: pheno, dtype: int64

In [7]:
df_clean = df.drop(columns=['id'])

In [8]:
df_clean.shape

(253, 95)

In [9]:
df_clean.head()

,TTGTTATAGTC,TTAATTTAATAGA,TTAACATAATAAT,TGCAATCTCTTTAT,TATTATGTTAATG,TACATACCGAT,GTGTATCATAAT,GCTGTTGAAATGGC,GCAAACATGCG,GAGTCCTGTT,...,group_7824,group_7845,group_10140,group_10143,group_3985,group_4427,group_7172,group_8210,group_9586,pheno
0,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1
1,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1
3,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1
4,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1


In [10]:
X = df.loc[:, df.columns != 'pheno']
y = df['pheno']
print(X.shape, y.shape)

(253, 95) (253,)


In [11]:
# over-sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_over, y_over = overS.fit_resample(X, y)
print(sorted(Counter(y_over).items()))

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.bagging module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.base module is  deprecated in versi

[(0, 129), (1, 129), (2, 129)]


/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is depreca

In [12]:
############# Fully-Connected Neural Network ################

In [13]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1

In [14]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_over)

In [15]:
dat = pd.DataFrame(X_test_over[:,0])
dat['test'] = y_test_over

In [16]:
dat

,0,test
0,NRS383,1
1,NRS254,1
2,NRS218,1
3,NRS215,0
4,BCH-SA-14,2
...,...,...
112,NRS227,1
113,NRS272,1
114,CFBRSa24,0
115,CFBRSa74,0


In [17]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [18]:
#### neural network on over-sampling data
model1_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [19]:
model1_over.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [20]:
model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 270 samples, validate on 117 samples
Epoch 1/1000
270/270 [==============================] - 0s 451us/step - loss: 1.0906 - accuracy: 0.3259 - val_loss: 1.0419 - val_accuracy: 0.4188
Epoch 2/1000
270/270 [==============================] - 0s 93us/step - loss: 0.9993 - accuracy: 0.5593 - val_loss: 1.0124 - val_accuracy: 0.5385
Epoch 3/1000
270/270 [==============================] - 0s 112us/step - loss: 0.9526 - accuracy: 0.6000 - val_loss: 0.9882 - val_accuracy: 0.5299
Epoch 4/1000
270/270 [==============================] - 0s 103us/step - loss: 0.9198 - accuracy: 0.5889 - val_loss: 0.9709 - val_accuracy: 0.5385
Epoch 5/1000
270/270 [==============================] - 0s 109us/step - loss: 0.8919 - accuracy: 0.5926 - val_loss: 0.9572 - val_accuracy: 0.5470
Epoch 6/1000
270/270 [==============================] - 0s 109us/step - loss: 0.8736 - accuracy: 0.5963 - val_loss: 0.9511 - val_accuracy: 0.5556
Epoch 7/1000
270/270 [==============================] - 0s 100us/step - loss: 0

Epoch 112/1000
270/270 [==============================] - 0s 181us/step - loss: 0.3641 - accuracy: 0.8519 - val_loss: 0.8336 - val_accuracy: 0.6496
Epoch 113/1000
270/270 [==============================] - 0s 253us/step - loss: 0.3584 - accuracy: 0.8667 - val_loss: 0.8406 - val_accuracy: 0.6410
Epoch 114/1000
270/270 [==============================] - 0s 155us/step - loss: 0.3546 - accuracy: 0.8667 - val_loss: 0.8367 - val_accuracy: 0.6410
Epoch 115/1000
270/270 [==============================] - 0s 112us/step - loss: 0.3589 - accuracy: 0.8556 - val_loss: 0.8401 - val_accuracy: 0.6325
Epoch 116/1000
270/270 [==============================] - 0s 112us/step - loss: 0.3510 - accuracy: 0.8593 - val_loss: 0.8596 - val_accuracy: 0.6239
Epoch 117/1000
270/270 [==============================] - 0s 100us/step - loss: 0.3535 - accuracy: 0.8593 - val_loss: 0.8473 - val_accuracy: 0.6239
Epoch 118/1000
270/270 [==============================] - 0s 115us/step - loss: 0.3543 - accuracy: 0.8519 - val_

270/270 [==============================] - 0s 103us/step - loss: 0.2679 - accuracy: 0.8741 - val_loss: 0.9257 - val_accuracy: 0.6581
Epoch 223/1000
270/270 [==============================] - 0s 95us/step - loss: 0.2690 - accuracy: 0.8815 - val_loss: 0.9518 - val_accuracy: 0.6581
Epoch 224/1000
270/270 [==============================] - ETA: 0s - loss: 0.4043 - accuracy: 0.78 - 0s 98us/step - loss: 0.2646 - accuracy: 0.8926 - val_loss: 0.9242 - val_accuracy: 0.6923
Epoch 225/1000
270/270 [==============================] - 0s 111us/step - loss: 0.2750 - accuracy: 0.8852 - val_loss: 0.9252 - val_accuracy: 0.6923
Epoch 226/1000
270/270 [==============================] - 0s 127us/step - loss: 0.2656 - accuracy: 0.8852 - val_loss: 0.9460 - val_accuracy: 0.6667
Epoch 227/1000
270/270 [==============================] - 0s 96us/step - loss: 0.2695 - accuracy: 0.8815 - val_loss: 0.9334 - val_accuracy: 0.6667
Epoch 228/1000
270/270 [==============================] - 0s 101us/step - loss: 0.2663 -

270/270 [==============================] - 0s 129us/step - loss: 0.2337 - accuracy: 0.8852 - val_loss: 1.0029 - val_accuracy: 0.7009
Epoch 333/1000
270/270 [==============================] - 0s 119us/step - loss: 0.2315 - accuracy: 0.9000 - val_loss: 0.9962 - val_accuracy: 0.7094
Epoch 334/1000
270/270 [==============================] - 0s 122us/step - loss: 0.2335 - accuracy: 0.8963 - val_loss: 1.0035 - val_accuracy: 0.7094
Epoch 335/1000
270/270 [==============================] - 0s 129us/step - loss: 0.2376 - accuracy: 0.9000 - val_loss: 1.0154 - val_accuracy: 0.7179
Epoch 336/1000
270/270 [==============================] - 0s 111us/step - loss: 0.2359 - accuracy: 0.8815 - val_loss: 0.9969 - val_accuracy: 0.7094
Epoch 337/1000
270/270 [==============================] - 0s 118us/step - loss: 0.2371 - accuracy: 0.8889 - val_loss: 0.9996 - val_accuracy: 0.7094
Epoch 338/1000
270/270 [==============================] - 0s 121us/step - loss: 0.2351 - accuracy: 0.8963 - val_loss: 0.9938 - 

270/270 [==============================] - 0s 108us/step - loss: 0.2207 - accuracy: 0.8926 - val_loss: 1.0631 - val_accuracy: 0.7094
Epoch 443/1000
270/270 [==============================] - 0s 108us/step - loss: 0.2153 - accuracy: 0.9000 - val_loss: 1.0747 - val_accuracy: 0.7009
Epoch 444/1000
270/270 [==============================] - 0s 122us/step - loss: 0.2210 - accuracy: 0.8963 - val_loss: 1.0804 - val_accuracy: 0.7094
Epoch 445/1000
270/270 [==============================] - 0s 175us/step - loss: 0.2218 - accuracy: 0.8852 - val_loss: 1.0714 - val_accuracy: 0.7265
Epoch 446/1000
270/270 [==============================] - 0s 122us/step - loss: 0.2193 - accuracy: 0.8778 - val_loss: 1.0904 - val_accuracy: 0.7009
Epoch 447/1000
270/270 [==============================] - 0s 121us/step - loss: 0.2184 - accuracy: 0.8889 - val_loss: 1.0789 - val_accuracy: 0.7179
Epoch 448/1000
270/270 [==============================] - 0s 126us/step - loss: 0.2207 - accuracy: 0.8889 - val_loss: 1.0666 - 

270/270 [==============================] - 0s 118us/step - loss: 0.2213 - accuracy: 0.8852 - val_loss: 1.1286 - val_accuracy: 0.7094
Epoch 553/1000
270/270 [==============================] - 0s 212us/step - loss: 0.2084 - accuracy: 0.8963 - val_loss: 1.1363 - val_accuracy: 0.7179
Epoch 554/1000
270/270 [==============================] - 0s 278us/step - loss: 0.2094 - accuracy: 0.8926 - val_loss: 1.1386 - val_accuracy: 0.7179
Epoch 555/1000
270/270 [==============================] - 0s 370us/step - loss: 0.2097 - accuracy: 0.9000 - val_loss: 1.1353 - val_accuracy: 0.7179
Epoch 556/1000
270/270 [==============================] - 0s 167us/step - loss: 0.2085 - accuracy: 0.8963 - val_loss: 1.1299 - val_accuracy: 0.7009
Epoch 557/1000
270/270 [==============================] - 0s 91us/step - loss: 0.2114 - accuracy: 0.8926 - val_loss: 1.1353 - val_accuracy: 0.7094
Epoch 558/1000
270/270 [==============================] - 0s 95us/step - loss: 0.2109 - accuracy: 0.8778 - val_loss: 1.1248 - va

Epoch 663/1000
270/270 [==============================] - 0s 99us/step - loss: 0.2062 - accuracy: 0.9000 - val_loss: 1.1903 - val_accuracy: 0.7179
Epoch 664/1000
270/270 [==============================] - 0s 92us/step - loss: 0.2044 - accuracy: 0.9000 - val_loss: 1.1845 - val_accuracy: 0.7179
Epoch 665/1000
270/270 [==============================] - 0s 104us/step - loss: 0.2065 - accuracy: 0.9000 - val_loss: 1.1917 - val_accuracy: 0.7179
Epoch 666/1000
270/270 [==============================] - 0s 103us/step - loss: 0.2030 - accuracy: 0.9000 - val_loss: 1.1816 - val_accuracy: 0.7179
Epoch 667/1000
270/270 [==============================] - 0s 102us/step - loss: 0.2035 - accuracy: 0.9000 - val_loss: 1.1842 - val_accuracy: 0.7179
Epoch 668/1000
270/270 [==============================] - 0s 99us/step - loss: 0.2060 - accuracy: 0.8963 - val_loss: 1.1892 - val_accuracy: 0.7179
Epoch 669/1000
270/270 [==============================] - 0s 94us/step - loss: 0.2078 - accuracy: 0.9000 - val_loss

270/270 [==============================] - 0s 104us/step - loss: 0.2024 - accuracy: 0.9000 - val_loss: 1.2295 - val_accuracy: 0.7094
Epoch 774/1000
270/270 [==============================] - 0s 127us/step - loss: 0.2035 - accuracy: 0.8889 - val_loss: 1.2154 - val_accuracy: 0.7350
Epoch 775/1000
270/270 [==============================] - 0s 99us/step - loss: 0.2059 - accuracy: 0.8963 - val_loss: 1.2211 - val_accuracy: 0.7094
Epoch 776/1000
270/270 [==============================] - 0s 140us/step - loss: 0.2013 - accuracy: 0.8963 - val_loss: 1.2317 - val_accuracy: 0.7179
Epoch 777/1000
270/270 [==============================] - 0s 170us/step - loss: 0.2009 - accuracy: 0.9000 - val_loss: 1.2244 - val_accuracy: 0.7094
Epoch 778/1000
270/270 [==============================] - 0s 161us/step - loss: 0.2032 - accuracy: 0.9000 - val_loss: 1.2161 - val_accuracy: 0.7179
Epoch 779/1000
270/270 [==============================] - 0s 161us/step - loss: 0.2053 - accuracy: 0.8815 - val_loss: 1.2119 - v

270/270 [==============================] - 0s 116us/step - loss: 0.2016 - accuracy: 0.8963 - val_loss: 1.2733 - val_accuracy: 0.7179
Epoch 884/1000
270/270 [==============================] - 0s 90us/step - loss: 0.1983 - accuracy: 0.9000 - val_loss: 1.2639 - val_accuracy: 0.7094
Epoch 885/1000
270/270 [==============================] - 0s 106us/step - loss: 0.2047 - accuracy: 0.8963 - val_loss: 1.2500 - val_accuracy: 0.7179
Epoch 886/1000
270/270 [==============================] - 0s 108us/step - loss: 0.2020 - accuracy: 0.8963 - val_loss: 1.2648 - val_accuracy: 0.7094
Epoch 887/1000
270/270 [==============================] - 0s 103us/step - loss: 0.2004 - accuracy: 0.9000 - val_loss: 1.2643 - val_accuracy: 0.7179
Epoch 888/1000
270/270 [==============================] - 0s 107us/step - loss: 0.2022 - accuracy: 0.8963 - val_loss: 1.2662 - val_accuracy: 0.7265
Epoch 889/1000
270/270 [==============================] - 0s 111us/step - loss: 0.1998 - accuracy: 0.9074 - val_loss: 1.2539 - v

Epoch 939/1000
270/270 [==============================] - 0s 68us/step - loss: 0.2029 - accuracy: 0.8815 - val_loss: 1.2696 - val_accuracy: 0.7350
Epoch 940/1000
270/270 [==============================] - 0s 74us/step - loss: 0.2016 - accuracy: 0.8963 - val_loss: 1.2941 - val_accuracy: 0.7179
Epoch 941/1000
270/270 [==============================] - 0s 70us/step - loss: 0.2010 - accuracy: 0.9037 - val_loss: 1.2790 - val_accuracy: 0.7179
Epoch 942/1000
270/270 [==============================] - 0s 69us/step - loss: 0.2007 - accuracy: 0.8852 - val_loss: 1.2770 - val_accuracy: 0.7179
Epoch 943/1000
270/270 [==============================] - 0s 76us/step - loss: 0.2017 - accuracy: 0.9000 - val_loss: 1.3042 - val_accuracy: 0.7094
Epoch 944/1000
270/270 [==============================] - 0s 72us/step - loss: 0.2004 - accuracy: 0.9000 - val_loss: 1.2881 - val_accuracy: 0.7179
Epoch 945/1000
270/270 [==============================] - 0s 63us/step - loss: 0.2102 - accuracy: 0.8889 - val_loss: 1

In [47]:
acc_test_over = model1_over.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over*100))

117/117 [==============================] - 0s 92us/step
over-sampling test accuracy: 68.38%


In [21]:
pred = model1_over.predict_classes(X_test_over)
pred

array([0, 1, 1, 0, 2, 1, 0, 2, 2, 2, 0, 2, 1, 1, 1, 2, 2, 2, 1, 2, 1, 1,
       0, 1, 0, 1, 0, 2, 1, 2, 2, 2, 2, 2, 1, 0, 1, 0, 1, 1, 2, 1, 0, 1,
       1, 1, 1, 2, 0, 1, 0, 2, 1, 0, 1, 0, 2, 2, 1, 1, 1, 1, 0, 2, 2, 2,
       1, 1, 2, 2, 0, 2, 1, 1, 2, 2, 0, 1, 0, 0, 0, 1, 1, 2, 1, 0, 2, 1,
       0, 0, 1, 1, 1, 0, 2, 2, 1, 2, 1, 0, 2, 0, 1, 2, 1, 0, 2, 0, 1, 2,
       2, 1, 0, 1, 1, 0, 1])

In [22]:
dat['pred'] = pred
dat

,0,test,pred
0,NRS383,1,0
1,NRS254,1,1
2,NRS218,1,1
3,NRS215,0,0
4,BCH-SA-14,2,2
...,...,...,...
112,NRS227,1,0
113,NRS272,1,1
114,CFBRSa24,0,1
115,CFBRSa74,0,0


In [23]:
proba1 = model1_over.predict_proba(X_test_over)
dat_proba1 = pd.DataFrame(proba1)

In [24]:
dat_proba1

,0,1,2
0,0.966261,0.032558,1.181456e-03
1,0.003082,0.996395,5.229296e-04
2,0.013079,0.986894,2.717784e-05
3,0.999998,0.000002,1.848142e-09
4,0.000005,0.060949,9.390469e-01
...,...,...,...
112,0.953246,0.009274,3.748031e-02
113,0.010068,0.987253,2.679535e-03
114,0.478381,0.521614,4.744370e-06
115,0.999965,0.000035,7.578712e-09


In [25]:
dat_proba1.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba1.csv", index = False,
         header=None)

In [26]:
dat.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/1p006p.csv", index = False,
         header=None)

In [51]:
hist1_over = model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 270 samples, validate on 117 samples
Epoch 1/1000
270/270 [==============================] - 0s 204us/step - loss: 0.2069 - accuracy: 0.8889 - val_loss: 1.4136 - val_accuracy: 0.7094
Epoch 2/1000
270/270 [==============================] - 0s 109us/step - loss: 0.2023 - accuracy: 0.8926 - val_loss: 1.4354 - val_accuracy: 0.6838
Epoch 3/1000
270/270 [==============================] - 0s 146us/step - loss: 0.2058 - accuracy: 0.9000 - val_loss: 1.4402 - val_accuracy: 0.6667
Epoch 4/1000
270/270 [==============================] - 0s 113us/step - loss: 0.2036 - accuracy: 0.8889 - val_loss: 1.4350 - val_accuracy: 0.6838
Epoch 5/1000
270/270 [==============================] - 0s 104us/step - loss: 0.2064 - accuracy: 0.8963 - val_loss: 1.4157 - val_accuracy: 0.6838
Epoch 6/1000
270/270 [==============================] - 0s 114us/step - loss: 0.2121 - accuracy: 0.9037 - val_loss: 1.4330 - val_accuracy: 0.6667
Epoch 7/1000
270/270 [==============================] - 0s 127us/step - loss: 

270/270 [==============================] - 0s 171us/step - loss: 0.1987 - accuracy: 0.9000 - val_loss: 1.4669 - val_accuracy: 0.6838
Epoch 112/1000
270/270 [==============================] - 0s 168us/step - loss: 0.1996 - accuracy: 0.9000 - val_loss: 1.4639 - val_accuracy: 0.6923
Epoch 113/1000
270/270 [==============================] - 0s 169us/step - loss: 0.1982 - accuracy: 0.9000 - val_loss: 1.4658 - val_accuracy: 0.6923
Epoch 114/1000
270/270 [==============================] - 0s 181us/step - loss: 0.1985 - accuracy: 0.9000 - val_loss: 1.4712 - val_accuracy: 0.6923
Epoch 115/1000
270/270 [==============================] - 0s 215us/step - loss: 0.2015 - accuracy: 0.8963 - val_loss: 1.4773 - val_accuracy: 0.6838
Epoch 116/1000
270/270 [==============================] - 0s 130us/step - loss: 0.2004 - accuracy: 0.9000 - val_loss: 1.4657 - val_accuracy: 0.6923
Epoch 117/1000
270/270 [==============================] - 0s 168us/step - loss: 0.2013 - accuracy: 0.8963 - val_loss: 1.4798 - 

270/270 [==============================] - 0s 163us/step - loss: 0.2026 - accuracy: 0.8889 - val_loss: 1.5314 - val_accuracy: 0.6923
Epoch 222/1000
270/270 [==============================] - 0s 130us/step - loss: 0.2033 - accuracy: 0.9037 - val_loss: 1.5469 - val_accuracy: 0.6667
Epoch 223/1000
270/270 [==============================] - 0s 171us/step - loss: 0.2024 - accuracy: 0.9037 - val_loss: 1.5261 - val_accuracy: 0.6923
Epoch 224/1000
270/270 [==============================] - 0s 130us/step - loss: 0.1966 - accuracy: 0.8963 - val_loss: 1.5353 - val_accuracy: 0.7009
Epoch 225/1000
270/270 [==============================] - 0s 191us/step - loss: 0.1995 - accuracy: 0.8963 - val_loss: 1.5274 - val_accuracy: 0.7009
Epoch 226/1000
270/270 [==============================] - 0s 179us/step - loss: 0.2025 - accuracy: 0.9000 - val_loss: 1.5277 - val_accuracy: 0.6923
Epoch 227/1000
270/270 [==============================] - 0s 170us/step - loss: 0.2010 - accuracy: 0.8926 - val_loss: 1.5182 - 

270/270 [==============================] - 0s 115us/step - loss: 0.1979 - accuracy: 0.9000 - val_loss: 1.5667 - val_accuracy: 0.6923
Epoch 332/1000
270/270 [==============================] - 0s 107us/step - loss: 0.1980 - accuracy: 0.9000 - val_loss: 1.5833 - val_accuracy: 0.6923
Epoch 333/1000
270/270 [==============================] - 0s 119us/step - loss: 0.1981 - accuracy: 0.9000 - val_loss: 1.5701 - val_accuracy: 0.6923
Epoch 334/1000
270/270 [==============================] - 0s 119us/step - loss: 0.2009 - accuracy: 0.9037 - val_loss: 1.5716 - val_accuracy: 0.6923
Epoch 335/1000
270/270 [==============================] - 0s 153us/step - loss: 0.2051 - accuracy: 0.8815 - val_loss: 1.5693 - val_accuracy: 0.6923
Epoch 336/1000
270/270 [==============================] - 0s 117us/step - loss: 0.1977 - accuracy: 0.9000 - val_loss: 1.5827 - val_accuracy: 0.7009
Epoch 337/1000
270/270 [==============================] - 0s 124us/step - loss: 0.1996 - accuracy: 0.9000 - val_loss: 1.5718 - 

270/270 [==============================] - 0s 106us/step - loss: 0.1992 - accuracy: 0.8926 - val_loss: 1.6091 - val_accuracy: 0.6923
Epoch 442/1000
270/270 [==============================] - 0s 107us/step - loss: 0.1988 - accuracy: 0.9000 - val_loss: 1.6254 - val_accuracy: 0.7009
Epoch 443/1000
270/270 [==============================] - 0s 120us/step - loss: 0.1998 - accuracy: 0.8926 - val_loss: 1.6126 - val_accuracy: 0.6923
Epoch 444/1000
270/270 [==============================] - 0s 92us/step - loss: 0.1985 - accuracy: 0.8926 - val_loss: 1.6264 - val_accuracy: 0.6752
Epoch 445/1000
270/270 [==============================] - 0s 115us/step - loss: 0.1978 - accuracy: 0.8963 - val_loss: 1.6151 - val_accuracy: 0.6923
Epoch 446/1000
270/270 [==============================] - 0s 124us/step - loss: 0.1982 - accuracy: 0.9000 - val_loss: 1.6228 - val_accuracy: 0.7009
Epoch 447/1000
270/270 [==============================] - 0s 121us/step - loss: 0.2013 - accuracy: 0.9000 - val_loss: 1.6253 - v

270/270 [==============================] - 0s 109us/step - loss: 0.1984 - accuracy: 0.9000 - val_loss: 1.6602 - val_accuracy: 0.6923
Epoch 552/1000
270/270 [==============================] - 0s 121us/step - loss: 0.1984 - accuracy: 0.8926 - val_loss: 1.6640 - val_accuracy: 0.6923
Epoch 553/1000
270/270 [==============================] - 0s 101us/step - loss: 0.2010 - accuracy: 0.8926 - val_loss: 1.6530 - val_accuracy: 0.6752
Epoch 554/1000
270/270 [==============================] - 0s 129us/step - loss: 0.2017 - accuracy: 0.8741 - val_loss: 1.6575 - val_accuracy: 0.6923
Epoch 555/1000
270/270 [==============================] - 0s 108us/step - loss: 0.1977 - accuracy: 0.9000 - val_loss: 1.6552 - val_accuracy: 0.6923
Epoch 556/1000
270/270 [==============================] - 0s 105us/step - loss: 0.1988 - accuracy: 0.8963 - val_loss: 1.6645 - val_accuracy: 0.7009
Epoch 557/1000
270/270 [==============================] - 0s 107us/step - loss: 0.2063 - accuracy: 0.8778 - val_loss: 1.6396 - 

270/270 [==============================] - 0s 100us/step - loss: 0.2010 - accuracy: 0.8963 - val_loss: 1.7148 - val_accuracy: 0.7094
Epoch 662/1000
270/270 [==============================] - 0s 90us/step - loss: 0.1990 - accuracy: 0.9000 - val_loss: 1.7014 - val_accuracy: 0.6923
Epoch 663/1000
270/270 [==============================] - 0s 90us/step - loss: 0.2015 - accuracy: 0.9000 - val_loss: 1.7034 - val_accuracy: 0.6923
Epoch 664/1000
270/270 [==============================] - 0s 105us/step - loss: 0.1967 - accuracy: 0.9000 - val_loss: 1.6962 - val_accuracy: 0.7009
Epoch 665/1000
270/270 [==============================] - 0s 95us/step - loss: 0.1999 - accuracy: 0.8963 - val_loss: 1.7036 - val_accuracy: 0.7009
Epoch 666/1000
270/270 [==============================] - 0s 118us/step - loss: 0.2008 - accuracy: 0.8963 - val_loss: 1.7005 - val_accuracy: 0.7009
Epoch 667/1000
270/270 [==============================] - 0s 101us/step - loss: 0.1960 - accuracy: 0.9000 - val_loss: 1.7109 - val

Epoch 772/1000
270/270 [==============================] - 0s 93us/step - loss: 0.2025 - accuracy: 0.8889 - val_loss: 1.7037 - val_accuracy: 0.7094
Epoch 773/1000
270/270 [==============================] - 0s 103us/step - loss: 0.1954 - accuracy: 0.8889 - val_loss: 1.7267 - val_accuracy: 0.7009
Epoch 774/1000
270/270 [==============================] - 0s 92us/step - loss: 0.1977 - accuracy: 0.8926 - val_loss: 1.7366 - val_accuracy: 0.7009
Epoch 775/1000
270/270 [==============================] - 0s 101us/step - loss: 0.1998 - accuracy: 0.8963 - val_loss: 1.7189 - val_accuracy: 0.7094
Epoch 776/1000
270/270 [==============================] - 0s 82us/step - loss: 0.1956 - accuracy: 0.8963 - val_loss: 1.7253 - val_accuracy: 0.7009
Epoch 777/1000
270/270 [==============================] - 0s 90us/step - loss: 0.1999 - accuracy: 0.8889 - val_loss: 1.7210 - val_accuracy: 0.6752
Epoch 778/1000
270/270 [==============================] - 0s 91us/step - loss: 0.2034 - accuracy: 0.8963 - val_loss:

270/270 [==============================] - 0s 89us/step - loss: 0.1951 - accuracy: 0.9000 - val_loss: 1.7490 - val_accuracy: 0.6923
Epoch 883/1000
270/270 [==============================] - 0s 93us/step - loss: 0.1998 - accuracy: 0.8963 - val_loss: 1.7594 - val_accuracy: 0.7009
Epoch 884/1000
270/270 [==============================] - 0s 85us/step - loss: 0.2005 - accuracy: 0.9037 - val_loss: 1.7722 - val_accuracy: 0.6923
Epoch 885/1000
270/270 [==============================] - 0s 93us/step - loss: 0.2003 - accuracy: 0.8963 - val_loss: 1.7415 - val_accuracy: 0.7094
Epoch 886/1000
270/270 [==============================] - 0s 110us/step - loss: 0.2001 - accuracy: 0.9000 - val_loss: 1.7464 - val_accuracy: 0.6923
Epoch 887/1000
270/270 [==============================] - 0s 91us/step - loss: 0.2026 - accuracy: 0.9000 - val_loss: 1.7376 - val_accuracy: 0.6923
Epoch 888/1000
270/270 [==============================] - 0s 94us/step - loss: 0.1984 - accuracy: 0.9000 - val_loss: 1.7526 - val_ac

Epoch 938/1000
270/270 [==============================] - 0s 98us/step - loss: 0.1964 - accuracy: 0.8963 - val_loss: 1.7708 - val_accuracy: 0.7094
Epoch 939/1000
270/270 [==============================] - 0s 98us/step - loss: 0.1995 - accuracy: 0.9000 - val_loss: 1.7719 - val_accuracy: 0.7009
Epoch 940/1000
270/270 [==============================] - 0s 126us/step - loss: 0.1996 - accuracy: 0.9000 - val_loss: 1.7495 - val_accuracy: 0.6923
Epoch 941/1000
270/270 [==============================] - 0s 101us/step - loss: 0.1998 - accuracy: 0.9000 - val_loss: 1.7539 - val_accuracy: 0.6923
Epoch 942/1000
270/270 [==============================] - 0s 96us/step - loss: 0.1987 - accuracy: 0.9000 - val_loss: 1.7643 - val_accuracy: 0.6923
Epoch 943/1000
270/270 [==============================] - 0s 89us/step - loss: 0.1970 - accuracy: 0.9000 - val_loss: 1.7565 - val_accuracy: 0.6923
Epoch 944/1000
270/270 [==============================] - 0s 106us/step - loss: 0.1988 - accuracy: 0.8963 - val_loss

In [52]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over.history['accuracy'])*100))

over-sampling train accuracy: 89.54%


In [18]:
df_proba = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_2.xlsx",
                        sheet_name=0,
                        index_col=None)

In [19]:
df_proba

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabs_qual,CFBRSa26,0,0,0.758914,0.241086,4.638713e-07
1,p002ykpresabs_qual,NRS109,2,2,0.005361,0.016236,9.784034e-01
2,p002ykpresabs_qual,NRS112,0,0,0.726623,0.273376,1.520979e-06
3,p002ykpresabs_qual,NRS216,1,1,0.138322,0.861665,1.334123e-05
4,p002ykpresabs_qual,NRS021,0,0,0.882176,0.117824,1.414530e-10
...,...,...,...,...,...,...,...
4279,pyopresabsSTCC_qual,NRS148,2,2,0.000007,0.000099,9.998934e-01
4280,pyopresabsSTCC_qual,NRS255,2,2,0.000257,0.002048,9.976944e-01
4281,pyopresabsSTCC_qual,NRS205,2,2,0.000011,0.000045,9.999435e-01
4282,pyopresabsSTCC_qual,NRS255,2,2,0.000257,0.002048,9.976944e-01


In [20]:
y_prob = df_proba[df_proba['phage']=='p0006presabs_qual'].iloc[:,-3:]
y_prob = y_prob.to_numpy()
y_prob

array([[9.66260970e-01, 3.25575960e-02, 1.18145640e-03],
       [3.08213550e-03, 9.96394930e-01, 5.22929600e-04],
       [1.30788350e-02, 9.86894000e-01, 2.71778390e-05],
       [9.99998200e-01, 1.79803540e-06, 1.84814210e-09],
       [4.63356300e-06, 6.09485320e-02, 9.39046860e-01],
       [5.16118370e-02, 6.10886700e-01, 3.37501530e-01],
       [9.99708830e-01, 1.52814060e-04, 1.38412880e-04],
       [1.30061850e-01, 9.73072800e-02, 7.72630900e-01],
       [3.44520700e-08, 1.09946450e-20, 1.00000000e+00],
       [8.17404000e-03, 5.94106900e-06, 9.91820000e-01],
       [9.99510650e-01, 2.20254270e-04, 2.69011940e-04],
       [5.46364400e-10, 8.41722650e-04, 9.99158260e-01],
       [1.54542230e-02, 9.84524900e-01, 2.09116440e-05],
       [4.78380920e-01, 5.21614400e-01, 4.74437000e-06],
       [1.46845470e-01, 8.51896170e-01, 1.25834920e-03],
       [4.63356300e-06, 6.09485320e-02, 9.39046860e-01],
       [8.46771200e-05, 4.93230300e-03, 9.94982960e-01],
       [1.78620400e-04, 5.72109

In [15]:
## Retrieved from https://github.com/scikit-learn/scikit-learn/issues/3298
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelBinarizer

def rocauc_ovo(truth, pred, average="macro", multi_class="ovo"):

    lb = LabelBinarizer()
    lb.fit(truth)

    truth = lb.transform(truth)   
    
    return roc_auc_score(truth, pred, average=average, multi_class=multi_class)

In [22]:
ovo1 = rocauc_ovo(y_test_over, y_prob, average="macro", multi_class="ovo")
ovo1

0.8645627876397107

In [16]:
def rocauc_ovr(truth, pred, average="macro", multi_class="ovr"):

    lb = LabelBinarizer()
    lb.fit(truth)

    truth = lb.transform(truth)   

    return roc_auc_score(truth, pred, average=average, multi_class=multi_class)

In [24]:
ovr1 = rocauc_ovr(y_test_over, y_prob, average="macro", multi_class="ovr")
ovr1

0.8645627876397107

In [25]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=234,
                                                    stratify=y_over)

In [26]:
dat2 = pd.DataFrame(X_test_over[:,0])
dat2['test'] = y_test_over

In [27]:
dat2

,0,test
0,BCH-SA-11,2
1,NRS161,1
2,BCH-SA-14,2
3,BCH-SA-11,2
4,CFBRSa49,1
...,...,...
112,NRS064,2
113,NRS266,2
114,NRS222,0
115,GA27,2


In [28]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [31]:
model1_over2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [32]:
model1_over2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [33]:
model1_over2.fit(X_train_over, y_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 270 samples, validate on 117 samples
Epoch 1/1000
270/270 [==============================] - 0s 577us/step - loss: 1.1979 - accuracy: 0.3444 - val_loss: 1.0965 - val_accuracy: 0.3419
Epoch 2/1000
270/270 [==============================] - 0s 123us/step - loss: 1.0690 - accuracy: 0.4259 - val_loss: 1.0104 - val_accuracy: 0.5214
Epoch 3/1000
270/270 [==============================] - 0s 170us/step - loss: 0.9959 - accuracy: 0.5296 - val_loss: 0.9627 - val_accuracy: 0.5983
Epoch 4/1000
270/270 [==============================] - 0s 197us/step - loss: 0.9607 - accuracy: 0.5852 - val_loss: 0.9298 - val_accuracy: 0.6239
Epoch 5/1000
270/270 [==============================] - 0s 160us/step - loss: 0.9278 - accuracy: 0.5852 - val_loss: 0.9057 - val_accuracy: 0.6068
Epoch 6/1000
270/270 [==============================] - 0s 195us/step - loss: 0.9040 - accuracy: 0.5815 - val_loss: 0.8895 - val_accuracy: 0.6068
Epoch 7/1000
270/270 [==============================] - 0s 190us/step - loss: 

Epoch 112/1000
270/270 [==============================] - 0s 138us/step - loss: 0.4206 - accuracy: 0.8333 - val_loss: 0.6684 - val_accuracy: 0.7179
Epoch 113/1000
270/270 [==============================] - 0s 115us/step - loss: 0.4206 - accuracy: 0.8370 - val_loss: 0.6700 - val_accuracy: 0.7179
Epoch 114/1000
270/270 [==============================] - 0s 158us/step - loss: 0.4226 - accuracy: 0.8407 - val_loss: 0.6626 - val_accuracy: 0.7265
Epoch 115/1000
270/270 [==============================] - 0s 141us/step - loss: 0.4227 - accuracy: 0.8148 - val_loss: 0.6855 - val_accuracy: 0.6838
Epoch 116/1000
270/270 [==============================] - 0s 110us/step - loss: 0.4176 - accuracy: 0.8333 - val_loss: 0.6629 - val_accuracy: 0.7179
Epoch 117/1000
270/270 [==============================] - 0s 90us/step - loss: 0.4169 - accuracy: 0.8444 - val_loss: 0.6715 - val_accuracy: 0.7179
Epoch 118/1000
270/270 [==============================] - 0s 97us/step - loss: 0.4152 - accuracy: 0.8370 - val_lo

270/270 [==============================] - 0s 104us/step - loss: 0.3238 - accuracy: 0.8630 - val_loss: 0.7074 - val_accuracy: 0.7350
Epoch 223/1000
270/270 [==============================] - 0s 92us/step - loss: 0.3262 - accuracy: 0.8556 - val_loss: 0.7252 - val_accuracy: 0.7436
Epoch 224/1000
270/270 [==============================] - 0s 97us/step - loss: 0.3214 - accuracy: 0.8630 - val_loss: 0.7120 - val_accuracy: 0.7350
Epoch 225/1000
270/270 [==============================] - 0s 90us/step - loss: 0.3234 - accuracy: 0.8593 - val_loss: 0.7127 - val_accuracy: 0.7350
Epoch 226/1000
270/270 [==============================] - 0s 82us/step - loss: 0.3239 - accuracy: 0.8630 - val_loss: 0.7218 - val_accuracy: 0.7436
Epoch 227/1000
270/270 [==============================] - 0s 75us/step - loss: 0.3225 - accuracy: 0.8667 - val_loss: 0.7114 - val_accuracy: 0.7350
Epoch 228/1000
270/270 [==============================] - 0s 84us/step - loss: 0.3225 - accuracy: 0.8630 - val_loss: 0.7134 - val_ac

Epoch 278/1000
270/270 [==============================] - 0s 64us/step - loss: 0.3006 - accuracy: 0.8667 - val_loss: 0.7434 - val_accuracy: 0.7778
Epoch 279/1000
270/270 [==============================] - 0s 67us/step - loss: 0.2997 - accuracy: 0.8667 - val_loss: 0.7456 - val_accuracy: 0.7521
Epoch 280/1000
270/270 [==============================] - 0s 69us/step - loss: 0.3006 - accuracy: 0.8704 - val_loss: 0.7374 - val_accuracy: 0.7692
Epoch 281/1000
270/270 [==============================] - 0s 69us/step - loss: 0.3008 - accuracy: 0.8667 - val_loss: 0.7380 - val_accuracy: 0.7778
Epoch 282/1000
270/270 [==============================] - 0s 61us/step - loss: 0.2987 - accuracy: 0.8704 - val_loss: 0.7538 - val_accuracy: 0.7692
Epoch 283/1000
270/270 [==============================] - 0s 66us/step - loss: 0.3003 - accuracy: 0.8741 - val_loss: 0.7469 - val_accuracy: 0.7692
Epoch 284/1000
270/270 [==============================] - 0s 58us/step - loss: 0.2974 - accuracy: 0.8667 - val_loss: 0

Epoch 389/1000
270/270 [==============================] - 0s 95us/step - loss: 0.2729 - accuracy: 0.8741 - val_loss: 0.7961 - val_accuracy: 0.7778
Epoch 390/1000
270/270 [==============================] - 0s 106us/step - loss: 0.2726 - accuracy: 0.8778 - val_loss: 0.7987 - val_accuracy: 0.7778
Epoch 391/1000
270/270 [==============================] - 0s 98us/step - loss: 0.2795 - accuracy: 0.8741 - val_loss: 0.7865 - val_accuracy: 0.7778
Epoch 392/1000
270/270 [==============================] - 0s 155us/step - loss: 0.2737 - accuracy: 0.8667 - val_loss: 0.8113 - val_accuracy: 0.7521
Epoch 393/1000
270/270 [==============================] - 0s 140us/step - loss: 0.2765 - accuracy: 0.8741 - val_loss: 0.7921 - val_accuracy: 0.7778
Epoch 394/1000
270/270 [==============================] - 0s 156us/step - loss: 0.2734 - accuracy: 0.8778 - val_loss: 0.7904 - val_accuracy: 0.7692
Epoch 395/1000
270/270 [==============================] - 0s 64us/step - loss: 0.2743 - accuracy: 0.8741 - val_los

...

270/270 [==============================] - 0s 79us/step - loss: 0.2458 - accuracy: 0.8778 - val_loss: 0.9333 - val_accuracy: 0.7436
Epoch 720/1000
270/270 [==============================] - 0s 94us/step - loss: 0.2558 - accuracy: 0.8815 - val_loss: 0.9421 - val_accuracy: 0.7265
Epoch 721/1000
270/270 [==============================] - 0s 89us/step - loss: 0.2519 - accuracy: 0.8704 - val_loss: 0.9454 - val_accuracy: 0.7692
Epoch 722/1000
270/270 [==============================] - 0s 92us/step - loss: 0.2493 - accuracy: 0.8741 - val_loss: 0.9406 - val_accuracy: 0.7692
Epoch 723/1000
270/270 [==============================] - 0s 85us/step - loss: 0.2545 - accuracy: 0.8704 - val_loss: 0.9487 - val_accuracy: 0.7436
Epoch 724/1000
270/270 [==============================] - 0s 88us/step - loss: 0.2497 - accuracy: 0.8667 - val_loss: 0.9350 - val_accuracy: 0.7692
Epoch 725/1000
270/270 [==============================] - 0s 84us/step - loss: 0.2538 - accuracy: 0.8704 - val_loss: 0.9363 - val_acc

Epoch 775/1000
270/270 [==============================] - 0s 83us/step - loss: 0.2523 - accuracy: 0.8778 - val_loss: 0.9684 - val_accuracy: 0.7436
Epoch 776/1000
270/270 [==============================] - 0s 83us/step - loss: 0.2515 - accuracy: 0.8741 - val_loss: 0.9545 - val_accuracy: 0.7778
Epoch 777/1000
270/270 [==============================] - 0s 86us/step - loss: 0.2522 - accuracy: 0.8815 - val_loss: 0.9758 - val_accuracy: 0.7436
Epoch 778/1000
270/270 [==============================] - 0s 85us/step - loss: 0.2502 - accuracy: 0.8741 - val_loss: 0.9611 - val_accuracy: 0.7692
Epoch 779/1000
270/270 [==============================] - 0s 89us/step - loss: 0.2474 - accuracy: 0.8815 - val_loss: 0.9653 - val_accuracy: 0.7436
Epoch 780/1000
270/270 [==============================] - 0s 91us/step - loss: 0.2535 - accuracy: 0.8741 - val_loss: 0.9834 - val_accuracy: 0.7350
Epoch 781/1000
270/270 [==============================] - 0s 86us/step - loss: 0.2553 - accuracy: 0.8778 - val_loss: 0

Epoch 831/1000
270/270 [==============================] - 0s 86us/step - loss: 0.2472 - accuracy: 0.8778 - val_loss: 0.9750 - val_accuracy: 0.7692
Epoch 832/1000
270/270 [==============================] - 0s 85us/step - loss: 0.2483 - accuracy: 0.8741 - val_loss: 0.9930 - val_accuracy: 0.7436
Epoch 833/1000
270/270 [==============================] - 0s 97us/step - loss: 0.2462 - accuracy: 0.8778 - val_loss: 0.9752 - val_accuracy: 0.7436
Epoch 834/1000
270/270 [==============================] - 0s 84us/step - loss: 0.2446 - accuracy: 0.8741 - val_loss: 0.9744 - val_accuracy: 0.7692
Epoch 835/1000
270/270 [==============================] - 0s 86us/step - loss: 0.2484 - accuracy: 0.8741 - val_loss: 0.9874 - val_accuracy: 0.7436
Epoch 836/1000
270/270 [==============================] - 0s 89us/step - loss: 0.2446 - accuracy: 0.8778 - val_loss: 0.9776 - val_accuracy: 0.7778
Epoch 837/1000
270/270 [==============================] - 0s 91us/step - loss: 0.2467 - accuracy: 0.8741 - val_loss: 0

Epoch 887/1000
270/270 [==============================] - 0s 88us/step - loss: 0.2441 - accuracy: 0.8778 - val_loss: 0.9924 - val_accuracy: 0.7436
Epoch 888/1000
270/270 [==============================] - 0s 89us/step - loss: 0.2497 - accuracy: 0.8704 - val_loss: 0.9947 - val_accuracy: 0.7436
Epoch 889/1000
270/270 [==============================] - 0s 90us/step - loss: 0.2499 - accuracy: 0.8704 - val_loss: 0.9785 - val_accuracy: 0.7692
Epoch 890/1000
270/270 [==============================] - 0s 88us/step - loss: 0.2512 - accuracy: 0.8778 - val_loss: 0.9963 - val_accuracy: 0.7692
Epoch 891/1000
270/270 [==============================] - 0s 89us/step - loss: 0.2475 - accuracy: 0.8778 - val_loss: 0.9944 - val_accuracy: 0.7436
Epoch 892/1000
270/270 [==============================] - 0s 83us/step - loss: 0.2465 - accuracy: 0.8815 - val_loss: 0.9765 - val_accuracy: 0.7607
Epoch 893/1000
270/270 [==============================] - 0s 92us/step - loss: 0.2462 - accuracy: 0.8778 - val_loss: 0

270/270 [==============================] - 0s 91us/step - loss: 0.2450 - accuracy: 0.8778 - val_loss: 1.0142 - val_accuracy: 0.7607
Epoch 998/1000
270/270 [==============================] - 0s 89us/step - loss: 0.2516 - accuracy: 0.8704 - val_loss: 1.0112 - val_accuracy: 0.7692
Epoch 999/1000
270/270 [==============================] - 0s 89us/step - loss: 0.2606 - accuracy: 0.8481 - val_loss: 1.0379 - val_accuracy: 0.7436
Epoch 1000/1000
270/270 [==============================] - 0s 91us/step - loss: 0.2516 - accuracy: 0.8704 - val_loss: 1.0123 - val_accuracy: 0.7778


In [69]:
acc_test_over2 = model1_over2.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over2*100))

117/117 [==============================] - 0s 58us/step
over-sampling test accuracy: 76.07%


In [34]:
pred2 = model1_over2.predict_classes(X_test_over)
pred2

array([2, 1, 2, 2, 1, 2, 2, 1, 0, 0, 2, 0, 0, 0, 0, 2, 2, 0, 2, 1, 2, 1,
       0, 0, 0, 1, 2, 1, 2, 1, 0, 1, 2, 1, 1, 0, 1, 1, 2, 2, 0, 0, 0, 0,
       2, 0, 2, 2, 2, 1, 0, 0, 2, 2, 0, 0, 1, 0, 1, 1, 2, 2, 2, 1, 1, 0,
       1, 2, 1, 0, 0, 0, 1, 2, 0, 1, 0, 2, 0, 0, 1, 2, 1, 2, 0, 0, 1, 0,
       0, 0, 2, 0, 1, 0, 1, 0, 2, 2, 1, 2, 0, 1, 1, 0, 1, 0, 1, 2, 1, 2,
       0, 1, 2, 2, 0, 2, 1])

In [35]:
dat2['pred'] = pred2
dat2

,0,test,pred
0,BCH-SA-11,2,2
1,NRS161,1,1
2,BCH-SA-14,2,2
3,BCH-SA-11,2,2
4,CFBRSa49,1,1
...,...,...,...
112,NRS064,2,2
113,NRS266,2,2
114,NRS222,0,0
115,GA27,2,2


In [36]:
proba2 = model1_over2.predict_proba(X_test_over)
dat_proba2 = pd.DataFrame(proba2)

In [37]:
dat_proba2

,0,1,2
0,1.401598e-01,0.221760,6.380803e-01
1,1.279516e-03,0.611688,3.870320e-01
2,1.149810e-06,0.000631,9.993679e-01
3,1.401598e-01,0.221760,6.380803e-01
4,1.517423e-02,0.984826,8.666426e-16
...,...,...,...
112,5.866610e-10,0.000001,9.999988e-01
113,1.139936e-02,0.000391,9.882096e-01
114,9.997405e-01,0.000205,5.487098e-05
115,1.858575e-04,0.002584,9.972298e-01


In [38]:
dat_proba2.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba2.csv", index = False,
         header=None)

In [39]:
dat2.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/2p006p.csv", index = False,
         header=None)

In [74]:
hist1_over2 = model1_over2.fit(X_train_over, y_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 270 samples, validate on 117 samples
Epoch 1/1000
270/270 [==============================] - 0s 124us/step - loss: 0.2463 - accuracy: 0.8778 - val_loss: 1.0701 - val_accuracy: 0.7350
Epoch 2/1000
270/270 [==============================] - 0s 128us/step - loss: 0.2485 - accuracy: 0.8667 - val_loss: 1.0554 - val_accuracy: 0.7607
Epoch 3/1000
270/270 [==============================] - 0s 140us/step - loss: 0.2415 - accuracy: 0.8704 - val_loss: 1.0761 - val_accuracy: 0.7350
Epoch 4/1000
270/270 [==============================] - 0s 114us/step - loss: 0.2467 - accuracy: 0.8778 - val_loss: 1.0695 - val_accuracy: 0.7350
Epoch 5/1000
270/270 [==============================] - 0s 111us/step - loss: 0.2420 - accuracy: 0.8778 - val_loss: 1.0789 - val_accuracy: 0.7350
Epoch 6/1000
270/270 [==============================] - 0s 113us/step - loss: 0.2430 - accuracy: 0.8778 - val_loss: 1.0704 - val_accuracy: 0.7350
Epoch 7/1000
270/270 [==============================] - 0s 98us/step - loss: 0

270/270 [==============================] - 0s 88us/step - loss: 0.2414 - accuracy: 0.8778 - val_loss: 1.1067 - val_accuracy: 0.7350
Epoch 112/1000
270/270 [==============================] - 0s 107us/step - loss: 0.2471 - accuracy: 0.8593 - val_loss: 1.1269 - val_accuracy: 0.7179
Epoch 113/1000
270/270 [==============================] - 0s 109us/step - loss: 0.2483 - accuracy: 0.8741 - val_loss: 1.1060 - val_accuracy: 0.7607
Epoch 114/1000
270/270 [==============================] - 0s 101us/step - loss: 0.2431 - accuracy: 0.8778 - val_loss: 1.1115 - val_accuracy: 0.7607
Epoch 115/1000
270/270 [==============================] - 0s 118us/step - loss: 0.2447 - accuracy: 0.8778 - val_loss: 1.1049 - val_accuracy: 0.7607
Epoch 116/1000
270/270 [==============================] - 0s 112us/step - loss: 0.2427 - accuracy: 0.8778 - val_loss: 1.1030 - val_accuracy: 0.7607
Epoch 117/1000
270/270 [==============================] - 0s 114us/step - loss: 0.2453 - accuracy: 0.8667 - val_loss: 1.0939 - v

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.142766). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


270/270 [==============================] - 0s 213us/step - loss: 0.2422 - accuracy: 0.8778 - val_loss: 1.0816 - val_accuracy: 0.7607
Epoch 141/1000
270/270 [==============================] - 0s 122us/step - loss: 0.2460 - accuracy: 0.8778 - val_loss: 1.0932 - val_accuracy: 0.7350
Epoch 142/1000
270/270 [==============================] - 0s 162us/step - loss: 0.2436 - accuracy: 0.8778 - val_loss: 1.0966 - val_accuracy: 0.7350
Epoch 143/1000
270/270 [==============================] - 0s 143us/step - loss: 0.2460 - accuracy: 0.8667 - val_loss: 1.0916 - val_accuracy: 0.7692
Epoch 144/1000
270/270 [==============================] - 0s 153us/step - loss: 0.2659 - accuracy: 0.8556 - val_loss: 1.1049 - val_accuracy: 0.7350
Epoch 145/1000
270/270 [==============================] - 0s 158us/step - loss: 0.2449 - accuracy: 0.8593 - val_loss: 1.1355 - val_accuracy: 0.6752
Epoch 146/1000
270/270 [==============================] - 0s 134us/step - loss: 0.2513 - accuracy: 0.8519 - val_loss: 1.1057 - 

270/270 [==============================] - 0s 148us/step - loss: 0.2416 - accuracy: 0.8852 - val_loss: 1.1503 - val_accuracy: 0.7350
Epoch 251/1000
270/270 [==============================] - 0s 155us/step - loss: 0.2462 - accuracy: 0.8630 - val_loss: 1.1571 - val_accuracy: 0.7350
Epoch 252/1000
270/270 [==============================] - 0s 139us/step - loss: 0.2447 - accuracy: 0.8704 - val_loss: 1.1312 - val_accuracy: 0.7607
Epoch 253/1000
270/270 [==============================] - 0s 142us/step - loss: 0.2462 - accuracy: 0.8778 - val_loss: 1.1435 - val_accuracy: 0.7607
Epoch 254/1000
270/270 [==============================] - 0s 162us/step - loss: 0.2454 - accuracy: 0.8741 - val_loss: 1.1601 - val_accuracy: 0.7350
Epoch 255/1000
270/270 [==============================] - 0s 127us/step - loss: 0.2472 - accuracy: 0.8741 - val_loss: 1.1455 - val_accuracy: 0.7607
Epoch 256/1000
270/270 [==============================] - 0s 135us/step - loss: 0.2420 - accuracy: 0.8778 - val_loss: 1.1464 - 

...

Epoch 635/1000
270/270 [==============================] - 0s 118us/step - loss: 0.2444 - accuracy: 0.8778 - val_loss: 1.2224 - val_accuracy: 0.7607
Epoch 636/1000
270/270 [==============================] - 0s 205us/step - loss: 0.2446 - accuracy: 0.8778 - val_loss: 1.2201 - val_accuracy: 0.7350
Epoch 637/1000
270/270 [==============================] - 0s 149us/step - loss: 0.2420 - accuracy: 0.8778 - val_loss: 1.2116 - val_accuracy: 0.7607
Epoch 638/1000
270/270 [==============================] - 0s 108us/step - loss: 0.2481 - accuracy: 0.8741 - val_loss: 1.2260 - val_accuracy: 0.7350
Epoch 639/1000
270/270 [==============================] - 0s 126us/step - loss: 0.2393 - accuracy: 0.8778 - val_loss: 1.2258 - val_accuracy: 0.7350
Epoch 640/1000
270/270 [==============================] - 0s 113us/step - loss: 0.2452 - accuracy: 0.8593 - val_loss: 1.2332 - val_accuracy: 0.7350
Epoch 641/1000
270/270 [==============================] - 0s 112us/step - loss: 0.2394 - accuracy: 0.8815 - val_

270/270 [==============================] - 0s 100us/step - loss: 0.2419 - accuracy: 0.8778 - val_loss: 1.2416 - val_accuracy: 0.7350
Epoch 691/1000
270/270 [==============================] - 0s 98us/step - loss: 0.2430 - accuracy: 0.8778 - val_loss: 1.2475 - val_accuracy: 0.7350
Epoch 692/1000
270/270 [==============================] - 0s 112us/step - loss: 0.2385 - accuracy: 0.8778 - val_loss: 1.2389 - val_accuracy: 0.7692
Epoch 693/1000
270/270 [==============================] - 0s 104us/step - loss: 0.2471 - accuracy: 0.8667 - val_loss: 1.2434 - val_accuracy: 0.7436
Epoch 694/1000
270/270 [==============================] - 0s 103us/step - loss: 0.2426 - accuracy: 0.8741 - val_loss: 1.2480 - val_accuracy: 0.7607
Epoch 695/1000
270/270 [==============================] - 0s 99us/step - loss: 0.2454 - accuracy: 0.8667 - val_loss: 1.2587 - val_accuracy: 0.7350
Epoch 696/1000
270/270 [==============================] - 0s 98us/step - loss: 0.2414 - accuracy: 0.8778 - val_loss: 1.2439 - val

270/270 [==============================] - 0s 100us/step - loss: 0.2452 - accuracy: 0.8481 - val_loss: 1.2998 - val_accuracy: 0.7350
Epoch 801/1000
270/270 [==============================] - 0s 112us/step - loss: 0.2413 - accuracy: 0.8704 - val_loss: 1.2855 - val_accuracy: 0.7607
Epoch 802/1000
270/270 [==============================] - 0s 106us/step - loss: 0.2395 - accuracy: 0.8778 - val_loss: 1.2889 - val_accuracy: 0.7607
Epoch 803/1000
270/270 [==============================] - 0s 128us/step - loss: 0.2445 - accuracy: 0.8741 - val_loss: 1.2931 - val_accuracy: 0.7350
Epoch 804/1000
270/270 [==============================] - 0s 112us/step - loss: 0.2401 - accuracy: 0.8741 - val_loss: 1.2872 - val_accuracy: 0.7607
Epoch 805/1000
270/270 [==============================] - 0s 84us/step - loss: 0.2409 - accuracy: 0.8778 - val_loss: 1.2870 - val_accuracy: 0.7607
Epoch 806/1000
270/270 [==============================] - 0s 104us/step - loss: 0.2415 - accuracy: 0.8778 - val_loss: 1.2888 - v

270/270 [==============================] - 0s 112us/step - loss: 0.2431 - accuracy: 0.8741 - val_loss: 1.3304 - val_accuracy: 0.7607
Epoch 911/1000
270/270 [==============================] - 0s 102us/step - loss: 0.2388 - accuracy: 0.8778 - val_loss: 1.3280 - val_accuracy: 0.7607
Epoch 912/1000
270/270 [==============================] - 0s 91us/step - loss: 0.2398 - accuracy: 0.8778 - val_loss: 1.3256 - val_accuracy: 0.7607
Epoch 913/1000
270/270 [==============================] - 0s 107us/step - loss: 0.2403 - accuracy: 0.8778 - val_loss: 1.3242 - val_accuracy: 0.7350
Epoch 914/1000
270/270 [==============================] - 0s 89us/step - loss: 0.2418 - accuracy: 0.8778 - val_loss: 1.3329 - val_accuracy: 0.7350
Epoch 915/1000
270/270 [==============================] - 0s 94us/step - loss: 0.2413 - accuracy: 0.8778 - val_loss: 1.3311 - val_accuracy: 0.7350
Epoch 916/1000
270/270 [==============================] - 0s 91us/step - loss: 0.2475 - accuracy: 0.8556 - val_loss: 1.3246 - val_

Epoch 966/1000
270/270 [==============================] - 0s 108us/step - loss: 0.2452 - accuracy: 0.8519 - val_loss: 1.3512 - val_accuracy: 0.7350
Epoch 967/1000
270/270 [==============================] - 0s 92us/step - loss: 0.2457 - accuracy: 0.8741 - val_loss: 1.3396 - val_accuracy: 0.7350
Epoch 968/1000
270/270 [==============================] - 0s 90us/step - loss: 0.2421 - accuracy: 0.8778 - val_loss: 1.3538 - val_accuracy: 0.7350
Epoch 969/1000
270/270 [==============================] - 0s 89us/step - loss: 0.2411 - accuracy: 0.8741 - val_loss: 1.3357 - val_accuracy: 0.7607
Epoch 970/1000
270/270 [==============================] - 0s 95us/step - loss: 0.2462 - accuracy: 0.8556 - val_loss: 1.3461 - val_accuracy: 0.7179
Epoch 971/1000
270/270 [==============================] - 0s 91us/step - loss: 0.2449 - accuracy: 0.8741 - val_loss: 1.3437 - val_accuracy: 0.7350
Epoch 972/1000
270/270 [==============================] - 0s 90us/step - loss: 0.2438 - accuracy: 0.8778 - val_loss: 

In [75]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over2.history['accuracy'])*100))

over-sampling train accuracy: 87.47%


In [29]:
df_proba2 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_2.xlsx",
                        sheet_name=1,
                        index_col=None)

In [30]:
df_proba2

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabs_qual,NRS148,2,2,0.000056,1.748042e-03,9.981960e-01
1,p002ykpresabs_qual,BCH-SA-03,1,0,0.712007,2.879924e-01,9.646217e-07
2,p002ykpresabs_qual,NRS218,1,1,0.006222,9.937732e-01,4.482882e-06
3,p002ykpresabs_qual,NRS036,0,0,0.882617,1.173831e-01,2.310933e-10
4,p002ykpresabs_qual,NRS386,1,0,0.571179,4.288184e-01,2.444667e-06
...,...,...,...,...,...,...,...
4279,pyopresabsSTCC_qual,NRS112,1,1,0.001860,9.979747e-01,1.653396e-04
4280,pyopresabsSTCC_qual,SR1065,0,0,0.982940,1.705227e-02,7.349168e-06
4281,pyopresabsSTCC_qual,NRS203,0,0,0.997093,1.962516e-03,9.441347e-04
4282,pyopresabsSTCC_qual,CFBREBSa129,0,0,1.000000,3.031141e-13,3.208205e-09


In [31]:
y_prob2 = df_proba2[df_proba2['phage']=='p0006presabs_qual'].iloc[:,-3:]
y_prob2 = y_prob2.to_numpy()
y_prob2

array([[1.40159790e-01, 2.21759930e-01, 6.38080300e-01],
       [1.27951560e-03, 6.11688500e-01, 3.87032000e-01],
       [1.14981000e-06, 6.30983500e-04, 9.99367900e-01],
       [1.40159790e-01, 2.21759930e-01, 6.38080300e-01],
       [1.51742340e-02, 9.84825800e-01, 8.66642550e-16],
       [2.08744730e-05, 1.13796390e-04, 9.99865300e-01],
       [2.66073170e-05, 6.04231000e-04, 9.99369200e-01],
       [1.14000120e-02, 9.88586100e-01, 1.39110100e-05],
       [9.95590000e-01, 4.16077400e-03, 2.49183070e-04],
       [6.00208340e-01, 3.99788350e-01, 3.31982280e-06],
       [7.53085200e-07, 9.24056500e-12, 9.99999300e-01],
       [6.00208340e-01, 3.99788350e-01, 3.31982280e-06],
       [9.31678650e-01, 2.58136280e-04, 6.80632900e-02],
       [6.23062600e-01, 3.74073740e-01, 2.86369300e-03],
       [9.99999900e-01, 9.80837950e-08, 1.29155950e-10],
       [1.20944480e-10, 3.64184950e-08, 1.00000000e+00],
       [1.73290820e-02, 7.19616700e-04, 9.81951360e-01],
       [6.00208340e-01, 3.99788

In [32]:
ovo2 = rocauc_ovo(y_test_over, y_prob2, average="macro", multi_class="ovo")
ovo2

0.8908612754766602

In [33]:
ovr2 = rocauc_ovr(y_test_over, y_prob2, average="macro", multi_class="ovr")
ovr2

0.8908612754766602

In [34]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=345,
                                                    stratify=y_over)

In [35]:
dat3 = pd.DataFrame(X_test_over[:,0])
dat3['test'] = y_test_over

In [36]:
dat3

,0,test
0,SR4187,0
1,NRS177,0
2,NRS109,2
3,CFBREBSa131,2
4,SR4152,1
...,...,...
112,NRS110,2
113,CFBRSa25,1
114,834N,0
115,CFBREBSa114,1


In [37]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [38]:
model1_over3 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [45]:
model1_over3.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [46]:
model1_over3.fit(X_train_over, y_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 270 samples, validate on 117 samples
Epoch 1/1000
270/270 [==============================] - 0s 667us/step - loss: 1.1835 - accuracy: 0.3667 - val_loss: 1.0829 - val_accuracy: 0.3761
Epoch 2/1000
270/270 [==============================] - 0s 123us/step - loss: 1.0273 - accuracy: 0.4630 - val_loss: 1.0095 - val_accuracy: 0.4786
Epoch 3/1000
270/270 [==============================] - 0s 162us/step - loss: 0.9828 - accuracy: 0.5407 - val_loss: 0.9744 - val_accuracy: 0.5470
Epoch 4/1000
270/270 [==============================] - 0s 133us/step - loss: 0.9499 - accuracy: 0.5667 - val_loss: 0.9436 - val_accuracy: 0.5897
Epoch 5/1000
270/270 [==============================] - 0s 115us/step - loss: 0.9216 - accuracy: 0.5852 - val_loss: 0.9224 - val_accuracy: 0.6154
Epoch 6/1000
270/270 [==============================] - 0s 140us/step - loss: 0.8981 - accuracy: 0.6148 - val_loss: 0.9089 - val_accuracy: 0.6154
Epoch 7/1000
270/270 [==============================] - 0s 183us/step - loss: 

Epoch 112/1000
270/270 [==============================] - 0s 86us/step - loss: 0.4132 - accuracy: 0.8444 - val_loss: 0.6592 - val_accuracy: 0.7179
Epoch 113/1000
270/270 [==============================] - 0s 76us/step - loss: 0.4117 - accuracy: 0.8407 - val_loss: 0.6639 - val_accuracy: 0.7265
Epoch 114/1000
270/270 [==============================] - 0s 86us/step - loss: 0.4112 - accuracy: 0.8444 - val_loss: 0.6481 - val_accuracy: 0.7179
Epoch 115/1000
270/270 [==============================] - 0s 101us/step - loss: 0.4081 - accuracy: 0.8407 - val_loss: 0.6580 - val_accuracy: 0.7265
Epoch 116/1000
270/270 [==============================] - 0s 127us/step - loss: 0.4101 - accuracy: 0.8481 - val_loss: 0.6671 - val_accuracy: 0.7009
Epoch 117/1000
270/270 [==============================] - 0s 108us/step - loss: 0.4070 - accuracy: 0.8481 - val_loss: 0.6515 - val_accuracy: 0.7179
Epoch 118/1000
270/270 [==============================] - 0s 124us/step - loss: 0.4079 - accuracy: 0.8481 - val_los

Epoch 223/1000
270/270 [==============================] - 0s 104us/step - loss: 0.3241 - accuracy: 0.8741 - val_loss: 0.6830 - val_accuracy: 0.7094
Epoch 224/1000
270/270 [==============================] - 0s 112us/step - loss: 0.3356 - accuracy: 0.8741 - val_loss: 0.6771 - val_accuracy: 0.7094
Epoch 225/1000
270/270 [==============================] - 0s 109us/step - loss: 0.3278 - accuracy: 0.8593 - val_loss: 0.6974 - val_accuracy: 0.7179
Epoch 226/1000
270/270 [==============================] - 0s 106us/step - loss: 0.3241 - accuracy: 0.8741 - val_loss: 0.6817 - val_accuracy: 0.7094
Epoch 227/1000
270/270 [==============================] - 0s 137us/step - loss: 0.3218 - accuracy: 0.8704 - val_loss: 0.6901 - val_accuracy: 0.7094
Epoch 228/1000
270/270 [==============================] - 0s 113us/step - loss: 0.3291 - accuracy: 0.8593 - val_loss: 0.6760 - val_accuracy: 0.7094
Epoch 229/1000
270/270 [==============================] - 0s 97us/step - loss: 0.3237 - accuracy: 0.8741 - val_l

270/270 [==============================] - 0s 91us/step - loss: 0.2907 - accuracy: 0.8778 - val_loss: 0.7389 - val_accuracy: 0.7094
Epoch 334/1000
270/270 [==============================] - 0s 109us/step - loss: 0.2895 - accuracy: 0.8778 - val_loss: 0.7241 - val_accuracy: 0.7094
Epoch 335/1000
270/270 [==============================] - 0s 138us/step - loss: 0.2880 - accuracy: 0.8593 - val_loss: 0.7372 - val_accuracy: 0.7094
Epoch 336/1000
270/270 [==============================] - 0s 86us/step - loss: 0.2891 - accuracy: 0.8741 - val_loss: 0.7423 - val_accuracy: 0.7094
Epoch 337/1000
270/270 [==============================] - 0s 85us/step - loss: 0.2944 - accuracy: 0.8704 - val_loss: 0.7293 - val_accuracy: 0.7179
Epoch 338/1000
270/270 [==============================] - 0s 87us/step - loss: 0.2879 - accuracy: 0.8778 - val_loss: 0.7434 - val_accuracy: 0.7094
Epoch 339/1000
270/270 [==============================] - 0s 81us/step - loss: 0.2885 - accuracy: 0.8778 - val_loss: 0.7425 - val_a

Epoch 389/1000
270/270 [==============================] - 0s 103us/step - loss: 0.2779 - accuracy: 0.8778 - val_loss: 0.7721 - val_accuracy: 0.7094
Epoch 390/1000
270/270 [==============================] - 0s 112us/step - loss: 0.2754 - accuracy: 0.8778 - val_loss: 0.7593 - val_accuracy: 0.7094
Epoch 391/1000
270/270 [==============================] - 0s 85us/step - loss: 0.2748 - accuracy: 0.8778 - val_loss: 0.7573 - val_accuracy: 0.7094
Epoch 392/1000
270/270 [==============================] - 0s 122us/step - loss: 0.2803 - accuracy: 0.8667 - val_loss: 0.7534 - val_accuracy: 0.7094
Epoch 393/1000
270/270 [==============================] - 0s 97us/step - loss: 0.2797 - accuracy: 0.8778 - val_loss: 0.7963 - val_accuracy: 0.7179
Epoch 394/1000
270/270 [==============================] - 0s 90us/step - loss: 0.2794 - accuracy: 0.8778 - val_loss: 0.7730 - val_accuracy: 0.7265
Epoch 395/1000
270/270 [==============================] - 0s 93us/step - loss: 0.2762 - accuracy: 0.8778 - val_loss

...

Epoch 612/1000
270/270 [==============================] - 0s 88us/step - loss: 0.2538 - accuracy: 0.8778 - val_loss: 0.8497 - val_accuracy: 0.7094
Epoch 613/1000
270/270 [==============================] - 0s 84us/step - loss: 0.2542 - accuracy: 0.8815 - val_loss: 0.8698 - val_accuracy: 0.7179
Epoch 614/1000
270/270 [==============================] - 0s 94us/step - loss: 0.2535 - accuracy: 0.8852 - val_loss: 0.8742 - val_accuracy: 0.7009
Epoch 615/1000
270/270 [==============================] - 0s 92us/step - loss: 0.2526 - accuracy: 0.8667 - val_loss: 0.8679 - val_accuracy: 0.7009
Epoch 616/1000
270/270 [==============================] - 0s 83us/step - loss: 0.2546 - accuracy: 0.8852 - val_loss: 0.8690 - val_accuracy: 0.7009
Epoch 617/1000
270/270 [==============================] - 0s 87us/step - loss: 0.2537 - accuracy: 0.8852 - val_loss: 0.8804 - val_accuracy: 0.7009
Epoch 618/1000
270/270 [==============================] - 0s 84us/step - loss: 0.2539 - accuracy: 0.8815 - val_loss: 0

Epoch 668/1000
270/270 [==============================] - 0s 90us/step - loss: 0.2492 - accuracy: 0.8852 - val_loss: 0.8828 - val_accuracy: 0.7009
Epoch 669/1000
270/270 [==============================] - 0s 89us/step - loss: 0.2489 - accuracy: 0.8852 - val_loss: 0.8932 - val_accuracy: 0.7009
Epoch 670/1000
270/270 [==============================] - 0s 84us/step - loss: 0.2556 - accuracy: 0.8778 - val_loss: 0.9023 - val_accuracy: 0.6923
Epoch 671/1000
270/270 [==============================] - 0s 84us/step - loss: 0.2568 - accuracy: 0.8852 - val_loss: 0.8824 - val_accuracy: 0.7179
Epoch 672/1000
270/270 [==============================] - 0s 81us/step - loss: 0.2500 - accuracy: 0.8815 - val_loss: 0.8822 - val_accuracy: 0.7009
Epoch 673/1000
270/270 [==============================] - 0s 88us/step - loss: 0.2488 - accuracy: 0.8852 - val_loss: 0.9017 - val_accuracy: 0.7009
Epoch 674/1000
270/270 [==============================] - 0s 97us/step - loss: 0.2494 - accuracy: 0.8852 - val_loss: 0

Epoch 779/1000
270/270 [==============================] - 0s 84us/step - loss: 0.2474 - accuracy: 0.8852 - val_loss: 0.9475 - val_accuracy: 0.7179
Epoch 780/1000
270/270 [==============================] - 0s 89us/step - loss: 0.2458 - accuracy: 0.8889 - val_loss: 0.9185 - val_accuracy: 0.7009
Epoch 781/1000
270/270 [==============================] - 0s 84us/step - loss: 0.2478 - accuracy: 0.8815 - val_loss: 0.9443 - val_accuracy: 0.6923
Epoch 782/1000
270/270 [==============================] - 0s 87us/step - loss: 0.2458 - accuracy: 0.8852 - val_loss: 0.9422 - val_accuracy: 0.7009
Epoch 783/1000
270/270 [==============================] - 0s 85us/step - loss: 0.2498 - accuracy: 0.8852 - val_loss: 0.9695 - val_accuracy: 0.7179
Epoch 784/1000
270/270 [==============================] - 0s 88us/step - loss: 0.2452 - accuracy: 0.8852 - val_loss: 0.9292 - val_accuracy: 0.7179
Epoch 785/1000
270/270 [==============================] - 0s 83us/step - loss: 0.2457 - accuracy: 0.8852 - val_loss: 0

270/270 [==============================] - 0s 86us/step - loss: 0.2413 - accuracy: 0.8889 - val_loss: 0.9931 - val_accuracy: 0.7179
Epoch 890/1000
270/270 [==============================] - 0s 87us/step - loss: 0.2426 - accuracy: 0.8852 - val_loss: 0.9909 - val_accuracy: 0.6923
Epoch 891/1000
270/270 [==============================] - 0s 103us/step - loss: 0.2478 - accuracy: 0.8815 - val_loss: 1.0005 - val_accuracy: 0.7094
Epoch 892/1000
270/270 [==============================] - 0s 93us/step - loss: 0.2470 - accuracy: 0.8815 - val_loss: 0.9774 - val_accuracy: 0.7179
Epoch 893/1000
270/270 [==============================] - 0s 84us/step - loss: 0.2507 - accuracy: 0.8630 - val_loss: 0.9862 - val_accuracy: 0.7009
Epoch 894/1000
270/270 [==============================] - 0s 90us/step - loss: 0.2404 - accuracy: 0.8852 - val_loss: 0.9930 - val_accuracy: 0.7009
Epoch 895/1000
270/270 [==============================] - 0s 85us/step - loss: 0.2421 - accuracy: 0.8852 - val_loss: 0.9856 - val_ac

Epoch 945/1000
270/270 [==============================] - 0s 84us/step - loss: 0.2425 - accuracy: 0.8852 - val_loss: 1.0030 - val_accuracy: 0.7094
Epoch 946/1000
270/270 [==============================] - 0s 91us/step - loss: 0.2460 - accuracy: 0.8704 - val_loss: 0.9973 - val_accuracy: 0.7009
Epoch 947/1000
270/270 [==============================] - 0s 82us/step - loss: 0.2447 - accuracy: 0.8741 - val_loss: 1.0051 - val_accuracy: 0.7094
Epoch 948/1000
270/270 [==============================] - 0s 83us/step - loss: 0.2417 - accuracy: 0.8852 - val_loss: 0.9925 - val_accuracy: 0.7009
Epoch 949/1000
270/270 [==============================] - 0s 96us/step - loss: 0.2422 - accuracy: 0.8778 - val_loss: 0.9877 - val_accuracy: 0.6923
Epoch 950/1000
270/270 [==============================] - 0s 100us/step - loss: 0.2419 - accuracy: 0.8778 - val_loss: 0.9976 - val_accuracy: 0.7094
Epoch 951/1000
270/270 [==============================] - 0s 86us/step - loss: 0.2402 - accuracy: 0.8852 - val_loss: 

In [99]:
acc_test_over3 = model1_over3.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over3*100))

117/117 [==============================] - 0s 83us/step
over-sampling test accuracy: 70.94%


In [47]:
pred3 = model1_over3.predict_classes(X_test_over)
pred3

array([1, 0, 2, 2, 1, 1, 1, 2, 2, 2, 2, 0, 2, 0, 1, 0, 2, 1, 1, 0, 0, 1,
       2, 2, 1, 1, 1, 2, 1, 1, 0, 2, 1, 1, 1, 2, 0, 2, 2, 2, 1, 1, 0, 2,
       2, 2, 1, 2, 1, 1, 1, 0, 2, 2, 1, 0, 2, 2, 0, 0, 1, 2, 1, 1, 0, 2,
       0, 2, 0, 0, 1, 0, 0, 2, 2, 1, 2, 0, 2, 1, 1, 2, 1, 2, 1, 1, 2, 0,
       0, 1, 2, 0, 1, 1, 2, 2, 1, 2, 2, 1, 1, 0, 1, 2, 0, 1, 1, 0, 2, 1,
       1, 0, 1, 1, 0, 1, 1])

In [48]:
dat3['pred'] = pred3
dat3

,0,test,pred
0,SR4187,0,1
1,NRS177,0,0
2,NRS109,2,2
3,CFBREBSa131,2,2
4,SR4152,1,1
...,...,...,...
112,NRS110,2,1
113,CFBRSa25,1,1
114,834N,0,0
115,CFBREBSa114,1,1


In [49]:
proba3 = model1_over3.predict_proba(X_test_over)
dat_proba3 = pd.DataFrame(proba3)

In [50]:
dat_proba3

,0,1,2
0,4.274309e-01,5.300288e-01,0.042540
1,5.455379e-01,4.544108e-01,0.000051
2,4.960730e-07,2.832813e-05,0.999971
3,4.014884e-07,2.203532e-07,0.999999
4,8.429662e-02,6.361464e-01,0.279557
...,...,...,...
112,4.274309e-01,5.300288e-01,0.042540
113,2.138136e-01,7.792639e-01,0.006922
114,9.998555e-01,1.507464e-05,0.000129
115,4.274309e-01,5.300288e-01,0.042540


In [51]:
dat_proba3.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba3.csv", index = False,
         header=None)

In [52]:
dat3.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/3p006p.csv", index = False,
         header=None)

In [105]:
hist1_over3 = model1_over3.fit(X_train_over, y_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 270 samples, validate on 117 samples
Epoch 1/1000
270/270 [==============================] - 0s 116us/step - loss: 0.2357 - accuracy: 0.8852 - val_loss: 1.2774 - val_accuracy: 0.7094
Epoch 2/1000
270/270 [==============================] - 0s 122us/step - loss: 0.2376 - accuracy: 0.8815 - val_loss: 1.2652 - val_accuracy: 0.7094
Epoch 3/1000
270/270 [==============================] - 0s 153us/step - loss: 0.2366 - accuracy: 0.8926 - val_loss: 1.2628 - val_accuracy: 0.7094
Epoch 4/1000
270/270 [==============================] - 0s 126us/step - loss: 0.2400 - accuracy: 0.8704 - val_loss: 1.2772 - val_accuracy: 0.7009
Epoch 5/1000
270/270 [==============================] - 0s 129us/step - loss: 0.2374 - accuracy: 0.8778 - val_loss: 1.2627 - val_accuracy: 0.7094
Epoch 6/1000
270/270 [==============================] - 0s 132us/step - loss: 0.2374 - accuracy: 0.8815 - val_loss: 1.2927 - val_accuracy: 0.7009
Epoch 7/1000
270/270 [==============================] - 0s 124us/step - loss: 

Epoch 112/1000
270/270 [==============================] - 0s 87us/step - loss: 0.2371 - accuracy: 0.8667 - val_loss: 1.3390 - val_accuracy: 0.7094
Epoch 113/1000
270/270 [==============================] - 0s 60us/step - loss: 0.2359 - accuracy: 0.8852 - val_loss: 1.3525 - val_accuracy: 0.7094
Epoch 114/1000
270/270 [==============================] - 0s 82us/step - loss: 0.2351 - accuracy: 0.8852 - val_loss: 1.3412 - val_accuracy: 0.7179
Epoch 115/1000
270/270 [==============================] - 0s 77us/step - loss: 0.2425 - accuracy: 0.8593 - val_loss: 1.3228 - val_accuracy: 0.7009
Epoch 116/1000
270/270 [==============================] - 0s 68us/step - loss: 0.2373 - accuracy: 0.8704 - val_loss: 1.3626 - val_accuracy: 0.7179
Epoch 117/1000
270/270 [==============================] - 0s 77us/step - loss: 0.2433 - accuracy: 0.8815 - val_loss: 1.3620 - val_accuracy: 0.7179
Epoch 118/1000
270/270 [==============================] - 0s 60us/step - loss: 0.2369 - accuracy: 0.8926 - val_loss: 1

270/270 [==============================] - 0s 139us/step - loss: 0.2362 - accuracy: 0.8815 - val_loss: 1.4123 - val_accuracy: 0.7094
Epoch 223/1000
270/270 [==============================] - 0s 108us/step - loss: 0.2345 - accuracy: 0.8778 - val_loss: 1.4337 - val_accuracy: 0.7009
Epoch 224/1000
270/270 [==============================] - 0s 102us/step - loss: 0.2387 - accuracy: 0.8852 - val_loss: 1.4149 - val_accuracy: 0.7265
Epoch 225/1000
270/270 [==============================] - 0s 158us/step - loss: 0.2346 - accuracy: 0.8852 - val_loss: 1.4290 - val_accuracy: 0.7094
Epoch 226/1000
270/270 [==============================] - 0s 117us/step - loss: 0.2346 - accuracy: 0.8852 - val_loss: 1.4168 - val_accuracy: 0.7179
Epoch 227/1000
270/270 [==============================] - 0s 101us/step - loss: 0.2355 - accuracy: 0.8704 - val_loss: 1.4152 - val_accuracy: 0.7265
Epoch 228/1000
270/270 [==============================] - 0s 141us/step - loss: 0.2352 - accuracy: 0.8926 - val_loss: 1.4181 - 

270/270 [==============================] - 0s 91us/step - loss: 0.2413 - accuracy: 0.8815 - val_loss: 1.4985 - val_accuracy: 0.7179
Epoch 333/1000
270/270 [==============================] - 0s 100us/step - loss: 0.2363 - accuracy: 0.8815 - val_loss: 1.4606 - val_accuracy: 0.7009
Epoch 334/1000
270/270 [==============================] - 0s 98us/step - loss: 0.2359 - accuracy: 0.8815 - val_loss: 1.4809 - val_accuracy: 0.7094
Epoch 335/1000
270/270 [==============================] - 0s 100us/step - loss: 0.2373 - accuracy: 0.8815 - val_loss: 1.4963 - val_accuracy: 0.7009
Epoch 336/1000
270/270 [==============================] - 0s 109us/step - loss: 0.2333 - accuracy: 0.8963 - val_loss: 1.4775 - val_accuracy: 0.7094
Epoch 337/1000
270/270 [==============================] - 0s 93us/step - loss: 0.2369 - accuracy: 0.8815 - val_loss: 1.4813 - val_accuracy: 0.7094
Epoch 338/1000
270/270 [==============================] - 0s 104us/step - loss: 0.2379 - accuracy: 0.8815 - val_loss: 1.4884 - val

Epoch 443/1000
270/270 [==============================] - 0s 103us/step - loss: 0.2369 - accuracy: 0.8741 - val_loss: 1.5641 - val_accuracy: 0.7179
Epoch 444/1000
270/270 [==============================] - 0s 91us/step - loss: 0.2350 - accuracy: 0.8778 - val_loss: 1.5703 - val_accuracy: 0.7094
Epoch 445/1000
270/270 [==============================] - 0s 99us/step - loss: 0.2350 - accuracy: 0.8815 - val_loss: 1.5721 - val_accuracy: 0.7009
Epoch 446/1000
270/270 [==============================] - 0s 99us/step - loss: 0.2339 - accuracy: 0.8852 - val_loss: 1.5610 - val_accuracy: 0.6923
Epoch 447/1000
270/270 [==============================] - 0s 103us/step - loss: 0.2361 - accuracy: 0.8778 - val_loss: 1.5544 - val_accuracy: 0.7009
Epoch 448/1000
270/270 [==============================] - 0s 113us/step - loss: 0.2337 - accuracy: 0.8852 - val_loss: 1.5676 - val_accuracy: 0.7179
Epoch 449/1000
270/270 [==============================] - 0s 90us/step - loss: 0.2327 - accuracy: 0.8852 - val_loss

Epoch 554/1000
270/270 [==============================] - 0s 90us/step - loss: 0.2355 - accuracy: 0.8852 - val_loss: 1.6133 - val_accuracy: 0.7265
Epoch 555/1000
270/270 [==============================] - 0s 89us/step - loss: 0.2369 - accuracy: 0.8815 - val_loss: 1.5810 - val_accuracy: 0.7094
Epoch 556/1000
270/270 [==============================] - 0s 89us/step - loss: 0.2352 - accuracy: 0.8852 - val_loss: 1.6033 - val_accuracy: 0.7009
Epoch 557/1000
270/270 [==============================] - 0s 89us/step - loss: 0.2331 - accuracy: 0.8778 - val_loss: 1.5986 - val_accuracy: 0.7009
Epoch 558/1000
270/270 [==============================] - 0s 98us/step - loss: 0.2350 - accuracy: 0.8815 - val_loss: 1.6089 - val_accuracy: 0.7009
Epoch 559/1000
270/270 [==============================] - 0s 115us/step - loss: 0.2334 - accuracy: 0.8852 - val_loss: 1.6087 - val_accuracy: 0.6923
Epoch 560/1000
270/270 [==============================] - 0s 90us/step - loss: 0.2396 - accuracy: 0.8815 - val_loss: 

270/270 [==============================] - 0s 126us/step - loss: 0.2331 - accuracy: 0.8852 - val_loss: 1.6622 - val_accuracy: 0.6923
Epoch 665/1000
270/270 [==============================] - 0s 154us/step - loss: 0.2435 - accuracy: 0.8852 - val_loss: 1.6448 - val_accuracy: 0.7265
Epoch 666/1000
270/270 [==============================] - 0s 157us/step - loss: 0.2366 - accuracy: 0.8741 - val_loss: 1.6669 - val_accuracy: 0.7094
Epoch 667/1000
270/270 [==============================] - 0s 116us/step - loss: 0.2332 - accuracy: 0.8852 - val_loss: 1.6657 - val_accuracy: 0.7009
Epoch 668/1000
270/270 [==============================] - 0s 133us/step - loss: 0.2325 - accuracy: 0.8852 - val_loss: 1.6472 - val_accuracy: 0.7009
Epoch 669/1000
270/270 [==============================] - 0s 114us/step - loss: 0.2331 - accuracy: 0.8852 - val_loss: 1.6472 - val_accuracy: 0.7009
Epoch 670/1000
270/270 [==============================] - 0s 182us/step - loss: 0.2330 - accuracy: 0.8852 - val_loss: 1.6569 - 

270/270 [==============================] - 0s 137us/step - loss: 0.2322 - accuracy: 0.8778 - val_loss: 1.7395 - val_accuracy: 0.7009
Epoch 775/1000
270/270 [==============================] - 0s 165us/step - loss: 0.2355 - accuracy: 0.8815 - val_loss: 1.7202 - val_accuracy: 0.7265
Epoch 776/1000
270/270 [==============================] - 0s 168us/step - loss: 0.2384 - accuracy: 0.8667 - val_loss: 1.7230 - val_accuracy: 0.7094
Epoch 777/1000
270/270 [==============================] - 0s 188us/step - loss: 0.2340 - accuracy: 0.8815 - val_loss: 1.7227 - val_accuracy: 0.7179
Epoch 778/1000
270/270 [==============================] - 0s 142us/step - loss: 0.2340 - accuracy: 0.8778 - val_loss: 1.7409 - val_accuracy: 0.7179
Epoch 779/1000
270/270 [==============================] - 0s 167us/step - loss: 0.2362 - accuracy: 0.8741 - val_loss: 1.7320 - val_accuracy: 0.7179
Epoch 780/1000
270/270 [==============================] - 0s 225us/step - loss: 0.2347 - accuracy: 0.8815 - val_loss: 1.7242 - 

270/270 [==============================] - 0s 128us/step - loss: 0.2360 - accuracy: 0.8815 - val_loss: 1.7633 - val_accuracy: 0.7179
Epoch 885/1000
270/270 [==============================] - 0s 117us/step - loss: 0.2329 - accuracy: 0.8852 - val_loss: 1.7840 - val_accuracy: 0.7009
Epoch 886/1000
270/270 [==============================] - 0s 121us/step - loss: 0.2331 - accuracy: 0.8852 - val_loss: 1.7689 - val_accuracy: 0.7179
Epoch 887/1000
270/270 [==============================] - 0s 112us/step - loss: 0.2357 - accuracy: 0.8667 - val_loss: 1.7720 - val_accuracy: 0.7009
Epoch 888/1000
270/270 [==============================] - 0s 98us/step - loss: 0.2335 - accuracy: 0.8815 - val_loss: 1.7767 - val_accuracy: 0.7009
Epoch 889/1000
270/270 [==============================] - 0s 96us/step - loss: 0.2327 - accuracy: 0.8852 - val_loss: 1.7880 - val_accuracy: 0.7179
Epoch 890/1000
270/270 [==============================] - 0s 92us/step - loss: 0.2320 - accuracy: 0.8852 - val_loss: 1.7823 - val

Epoch 994/1000
270/270 [==============================] - 0s 114us/step - loss: 0.2332 - accuracy: 0.8815 - val_loss: 1.8388 - val_accuracy: 0.7179
Epoch 995/1000
270/270 [==============================] - 0s 170us/step - loss: 0.2353 - accuracy: 0.8704 - val_loss: 1.8408 - val_accuracy: 0.7179
Epoch 996/1000
270/270 [==============================] - 0s 166us/step - loss: 0.2319 - accuracy: 0.8815 - val_loss: 1.8504 - val_accuracy: 0.7265
Epoch 997/1000
270/270 [==============================] - 0s 140us/step - loss: 0.2359 - accuracy: 0.8778 - val_loss: 1.8453 - val_accuracy: 0.7350
Epoch 998/1000
270/270 [==============================] - 0s 112us/step - loss: 0.2349 - accuracy: 0.8889 - val_loss: 1.8474 - val_accuracy: 0.7265
Epoch 999/1000
270/270 [==============================] - 0s 121us/step - loss: 0.2432 - accuracy: 0.8815 - val_loss: 1.8582 - val_accuracy: 0.7350
Epoch 1000/1000
270/270 [==============================] - 0s 105us/step - loss: 0.2321 - accuracy: 0.8852 - val

In [106]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over3.history['accuracy'])*100))

over-sampling train accuracy: 88.10%


In [39]:
df_proba3 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_2.xlsx",
                        sheet_name=2,
                        index_col=None)

In [40]:
df_proba3

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabs_qual,NRS109,2,2,0.004477,0.013518,9.820048e-01
1,p002ykpresabs_qual,NRS109,2,2,0.004477,0.013518,9.820048e-01
2,p002ykpresabs_qual,NRS222,0,0,0.851725,0.148269,5.980786e-06
3,p002ykpresabs_qual,NRS109,2,2,0.004477,0.013518,9.820048e-01
4,p002ykpresabs_qual,GA50245,0,0,0.812055,0.187945,1.161034e-07
...,...,...,...,...,...,...,...
4279,pyopresabsSTCC_qual,NRS255,2,2,0.000633,0.000928,9.984396e-01
4280,pyopresabsSTCC_qual,NRS255,2,2,0.000633,0.000928,9.984396e-01
4281,pyopresabsSTCC_qual,NRS266,1,1,0.025932,0.974061,7.323514e-06
4282,pyopresabsSTCC_qual,NRS001,1,1,0.000597,0.999403,3.675362e-10


In [41]:
y_prob3 = df_proba3[df_proba3['phage']=='p0006presabs_qual'].iloc[:,-3:]
y_prob3 = y_prob3.to_numpy()
y_prob3

array([[4.27430900e-01, 5.30028760e-01, 4.25402930e-02],
       [5.45537900e-01, 4.54410760e-01, 5.13159760e-05],
       [4.96073000e-07, 2.83281330e-05, 9.99971150e-01],
       [4.01488450e-07, 2.20353210e-07, 9.99999400e-01],
       [8.42966200e-02, 6.36146370e-01, 2.79557050e-01],
       [1.76221920e-01, 8.23777600e-01, 5.11233600e-07],
       [7.96645300e-05, 9.99894000e-01, 2.63388570e-05],
       [4.01488450e-07, 2.20353210e-07, 9.99999400e-01],
       [9.78356300e-03, 6.08098800e-02, 9.29406600e-01],
       [4.01488450e-07, 2.20353210e-07, 9.99999400e-01],
       [3.21095080e-04, 4.97451100e-03, 9.94704400e-01],
       [7.13727000e-01, 2.86093320e-01, 1.79655240e-04],
       [5.89314900e-04, 1.65953450e-03, 9.97751200e-01],
       [1.00000000e+00, 1.61318460e-08, 1.56131460e-11],
       [8.88902800e-02, 9.11102300e-01, 7.38604300e-06],
       [8.91226230e-01, 1.04236215e-01, 4.53766300e-03],
       [3.61728000e-03, 8.88966100e-04, 9.95493770e-01],
       [4.27430900e-01, 5.30028

In [42]:
ovo3 = rocauc_ovo(y_test_over, y_prob3, average="macro", multi_class="ovo")
ovo3

0.87146614069691

In [43]:
ovr3 = rocauc_ovr(y_test_over, y_prob3, average="macro", multi_class="ovr")
ovr3

0.87146614069691

In [44]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=456,
                                                    stratify=y_over)

In [45]:
dat4 = pd.DataFrame(X_test_over[:,0])
dat4['test'] = y_test_over

In [46]:
dat4

,0,test
0,CFBRSa25,1
1,CFBRSa07,0
2,NRS247,0
3,NY439,2
4,CFBREBSa110,1
...,...,...
112,SR1129,0
113,NRS172,0
114,NRS205,2
115,NY439,2


In [47]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [48]:
model1_over4 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
])

In [58]:
model1_over4.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [59]:
model1_over4.fit(X_train_over, y_train_over,
          batch_size=64, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 270 samples, validate on 117 samples
Epoch 1/1000
270/270 [==============================] - 0s 612us/step - loss: 1.0966 - accuracy: 0.3852 - val_loss: 1.0931 - val_accuracy: 0.4274
Epoch 2/1000
270/270 [==============================] - 0s 84us/step - loss: 1.0420 - accuracy: 0.4815 - val_loss: 1.0589 - val_accuracy: 0.4359
Epoch 3/1000
270/270 [==============================] - 0s 93us/step - loss: 0.9998 - accuracy: 0.5148 - val_loss: 1.0312 - val_accuracy: 0.4786
Epoch 4/1000
270/270 [==============================] - 0s 142us/step - loss: 0.9656 - accuracy: 0.5630 - val_loss: 1.0052 - val_accuracy: 0.4786
Epoch 5/1000
270/270 [==============================] - 0s 81us/step - loss: 0.9350 - accuracy: 0.6037 - val_loss: 0.9836 - val_accuracy: 0.5214
Epoch 6/1000
270/270 [==============================] - 0s 77us/step - loss: 0.9053 - accuracy: 0.6296 - val_loss: 0.9618 - val_accuracy: 0.4957
Epoch 7/1000
270/270 [==============================] - 0s 113us/step - loss: 0.87

Epoch 57/1000
270/270 [==============================] - 0s 101us/step - loss: 0.4839 - accuracy: 0.8037 - val_loss: 0.9471 - val_accuracy: 0.7009
Epoch 58/1000
270/270 [==============================] - 0s 59us/step - loss: 0.4739 - accuracy: 0.8111 - val_loss: 0.9316 - val_accuracy: 0.7009
Epoch 59/1000
270/270 [==============================] - 0s 60us/step - loss: 0.4733 - accuracy: 0.8111 - val_loss: 0.9249 - val_accuracy: 0.6923
Epoch 60/1000
270/270 [==============================] - 0s 70us/step - loss: 0.4770 - accuracy: 0.8185 - val_loss: 0.9006 - val_accuracy: 0.7094
Epoch 61/1000
270/270 [==============================] - 0s 81us/step - loss: 0.4658 - accuracy: 0.8185 - val_loss: 0.9202 - val_accuracy: 0.7009
Epoch 62/1000
270/270 [==============================] - 0s 67us/step - loss: 0.4573 - accuracy: 0.8037 - val_loss: 0.9248 - val_accuracy: 0.7009
Epoch 63/1000
270/270 [==============================] - 0s 80us/step - loss: 0.4525 - accuracy: 0.8185 - val_loss: 0.9276 

Epoch 113/1000
270/270 [==============================] - 0s 56us/step - loss: 0.3433 - accuracy: 0.8778 - val_loss: 1.1334 - val_accuracy: 0.6923
Epoch 114/1000
270/270 [==============================] - 0s 56us/step - loss: 0.3397 - accuracy: 0.8593 - val_loss: 1.1258 - val_accuracy: 0.6923
Epoch 115/1000
270/270 [==============================] - 0s 80us/step - loss: 0.3343 - accuracy: 0.8556 - val_loss: 1.1001 - val_accuracy: 0.7094
Epoch 116/1000
270/270 [==============================] - 0s 63us/step - loss: 0.3290 - accuracy: 0.8704 - val_loss: 1.0716 - val_accuracy: 0.7094
Epoch 117/1000
270/270 [==============================] - 0s 61us/step - loss: 0.3345 - accuracy: 0.8704 - val_loss: 1.0637 - val_accuracy: 0.7265
Epoch 118/1000
270/270 [==============================] - 0s 107us/step - loss: 0.3307 - accuracy: 0.8667 - val_loss: 1.0686 - val_accuracy: 0.7265
Epoch 119/1000
270/270 [==============================] - 0s 92us/step - loss: 0.3300 - accuracy: 0.8667 - val_loss: 

Epoch 224/1000
270/270 [==============================] - 0s 113us/step - loss: 0.2719 - accuracy: 0.8815 - val_loss: 1.3416 - val_accuracy: 0.7179
Epoch 225/1000
270/270 [==============================] - 0s 81us/step - loss: 0.2620 - accuracy: 0.8815 - val_loss: 1.3346 - val_accuracy: 0.6923
Epoch 226/1000
270/270 [==============================] - 0s 59us/step - loss: 0.2788 - accuracy: 0.8593 - val_loss: 1.3114 - val_accuracy: 0.6581
Epoch 227/1000
270/270 [==============================] - 0s 60us/step - loss: 0.2660 - accuracy: 0.8630 - val_loss: 1.3227 - val_accuracy: 0.7179
Epoch 228/1000
270/270 [==============================] - 0s 81us/step - loss: 0.2743 - accuracy: 0.8815 - val_loss: 1.3205 - val_accuracy: 0.7179
Epoch 229/1000
270/270 [==============================] - 0s 57us/step - loss: 0.2714 - accuracy: 0.8704 - val_loss: 1.2467 - val_accuracy: 0.6410
Epoch 230/1000
270/270 [==============================] - 0s 58us/step - loss: 0.2881 - accuracy: 0.8556 - val_loss: 

Epoch 280/1000
270/270 [==============================] - 0s 74us/step - loss: 0.2606 - accuracy: 0.8593 - val_loss: 1.3611 - val_accuracy: 0.6667
Epoch 281/1000
270/270 [==============================] - 0s 75us/step - loss: 0.2670 - accuracy: 0.8704 - val_loss: 1.3884 - val_accuracy: 0.7179
Epoch 282/1000
270/270 [==============================] - 0s 106us/step - loss: 0.2616 - accuracy: 0.8815 - val_loss: 1.4228 - val_accuracy: 0.7265
Epoch 283/1000
270/270 [==============================] - 0s 93us/step - loss: 0.2681 - accuracy: 0.8667 - val_loss: 1.4577 - val_accuracy: 0.6923
Epoch 284/1000
270/270 [==============================] - 0s 123us/step - loss: 0.2703 - accuracy: 0.8630 - val_loss: 1.4164 - val_accuracy: 0.7265
Epoch 285/1000
270/270 [==============================] - 0s 77us/step - loss: 0.2626 - accuracy: 0.8815 - val_loss: 1.3828 - val_accuracy: 0.7265
Epoch 286/1000
270/270 [==============================] - 0s 104us/step - loss: 0.2586 - accuracy: 0.8741 - val_loss

...

Epoch 726/1000
270/270 [==============================] - 0s 69us/step - loss: 0.2467 - accuracy: 0.8815 - val_loss: 1.6320 - val_accuracy: 0.7265
Epoch 727/1000
270/270 [==============================] - 0s 66us/step - loss: 0.2430 - accuracy: 0.8704 - val_loss: 1.6426 - val_accuracy: 0.7265
Epoch 728/1000
270/270 [==============================] - 0s 70us/step - loss: 0.2432 - accuracy: 0.8778 - val_loss: 1.6205 - val_accuracy: 0.7265
Epoch 729/1000
270/270 [==============================] - 0s 72us/step - loss: 0.2429 - accuracy: 0.8815 - val_loss: 1.6359 - val_accuracy: 0.7265
Epoch 730/1000
270/270 [==============================] - 0s 83us/step - loss: 0.2457 - accuracy: 0.8815 - val_loss: 1.6421 - val_accuracy: 0.7265
Epoch 731/1000
270/270 [==============================] - 0s 68us/step - loss: 0.2451 - accuracy: 0.8667 - val_loss: 1.6571 - val_accuracy: 0.7350
Epoch 732/1000
270/270 [==============================] - 0s 71us/step - loss: 0.2423 - accuracy: 0.8778 - val_loss: 1

Epoch 782/1000
270/270 [==============================] - 0s 53us/step - loss: 0.2446 - accuracy: 0.8778 - val_loss: 1.6807 - val_accuracy: 0.7265
Epoch 783/1000
270/270 [==============================] - 0s 57us/step - loss: 0.2522 - accuracy: 0.8630 - val_loss: 1.6948 - val_accuracy: 0.6752
Epoch 784/1000
270/270 [==============================] - 0s 50us/step - loss: 0.2444 - accuracy: 0.8741 - val_loss: 1.6143 - val_accuracy: 0.7265
Epoch 785/1000
270/270 [==============================] - 0s 51us/step - loss: 0.2515 - accuracy: 0.8815 - val_loss: 1.6427 - val_accuracy: 0.7265
Epoch 786/1000
270/270 [==============================] - 0s 57us/step - loss: 0.2485 - accuracy: 0.8815 - val_loss: 1.6718 - val_accuracy: 0.7265
Epoch 787/1000
270/270 [==============================] - 0s 53us/step - loss: 0.2482 - accuracy: 0.8741 - val_loss: 1.6725 - val_accuracy: 0.6410
Epoch 788/1000
270/270 [==============================] - 0s 62us/step - loss: 0.2572 - accuracy: 0.8519 - val_loss: 1

Epoch 893/1000
270/270 [==============================] - 0s 66us/step - loss: 0.2431 - accuracy: 0.8815 - val_loss: 1.6944 - val_accuracy: 0.7265
Epoch 894/1000
270/270 [==============================] - 0s 67us/step - loss: 0.2473 - accuracy: 0.8815 - val_loss: 1.7257 - val_accuracy: 0.7265
Epoch 895/1000
270/270 [==============================] - 0s 74us/step - loss: 0.2439 - accuracy: 0.8815 - val_loss: 1.6807 - val_accuracy: 0.7265
Epoch 896/1000
270/270 [==============================] - 0s 75us/step - loss: 0.2418 - accuracy: 0.8815 - val_loss: 1.6734 - val_accuracy: 0.7265
Epoch 897/1000
270/270 [==============================] - 0s 67us/step - loss: 0.2415 - accuracy: 0.8741 - val_loss: 1.6997 - val_accuracy: 0.7350
Epoch 898/1000
270/270 [==============================] - 0s 66us/step - loss: 0.2404 - accuracy: 0.8778 - val_loss: 1.7312 - val_accuracy: 0.7094
Epoch 899/1000
270/270 [==============================] - 0s 69us/step - loss: 0.2483 - accuracy: 0.8667 - val_loss: 1

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.132567). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


Epoch 985/1000
270/270 [==============================] - 0s 367us/step - loss: 0.2530 - accuracy: 0.8741 - val_loss: 1.7495 - val_accuracy: 0.7265
Epoch 986/1000
270/270 [==============================] - 0s 134us/step - loss: 0.2454 - accuracy: 0.8815 - val_loss: 1.7050 - val_accuracy: 0.7265
Epoch 987/1000
270/270 [==============================] - 0s 101us/step - loss: 0.2426 - accuracy: 0.8852 - val_loss: 1.6997 - val_accuracy: 0.7265
Epoch 988/1000
270/270 [==============================] - 0s 81us/step - loss: 0.2453 - accuracy: 0.8704 - val_loss: 1.7363 - val_accuracy: 0.6838
Epoch 989/1000
270/270 [==============================] - 0s 678us/step - loss: 0.2423 - accuracy: 0.8852 - val_loss: 1.7653 - val_accuracy: 0.7265
Epoch 990/1000
270/270 [==============================] - 0s 94us/step - loss: 0.2423 - accuracy: 0.8815 - val_loss: 1.7585 - val_accuracy: 0.7265
Epoch 991/1000
270/270 [==============================] - 0s 91us/step - loss: 0.2414 - accuracy: 0.8778 - val_los

In [122]:
acc_test_over4 = model1_over4.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over4*100))

117/117 [==============================] - 0s 115us/step
over-sampling test accuracy: 72.65%


In [60]:
pred4 = model1_over4.predict_classes(X_test_over)
pred4

array([1, 0, 0, 2, 1, 0, 2, 0, 2, 1, 0, 2, 2, 1, 1, 2, 2, 2, 0, 1, 1, 2,
       1, 1, 1, 0, 2, 2, 1, 2, 2, 1, 0, 1, 2, 2, 1, 0, 1, 1, 1, 2, 0, 0,
       1, 2, 0, 1, 0, 1, 0, 1, 1, 2, 1, 0, 0, 1, 2, 1, 1, 1, 1, 2, 1, 1,
       1, 2, 1, 2, 2, 1, 2, 1, 2, 1, 1, 2, 2, 0, 1, 1, 1, 0, 2, 2, 0, 1,
       2, 2, 2, 2, 1, 2, 2, 2, 1, 1, 1, 1, 2, 2, 1, 1, 2, 1, 0, 2, 1, 1,
       0, 1, 0, 1, 2, 2, 2])

In [61]:
dat4['pred'] = pred4
dat4

,0,test,pred
0,CFBRSa25,1,1
1,CFBRSa07,0,0
2,NRS247,0,0
3,NY439,2,2
4,CFBREBSa110,1,1
...,...,...,...
112,SR1129,0,0
113,NRS172,0,1
114,NRS205,2,2
115,NY439,2,2


In [62]:
proba4 = model1_over4.predict_proba(X_test_over)
dat_proba4 = pd.DataFrame(proba4)

In [63]:
dat_proba4

,0,1,2
0,0.127998,8.664240e-01,5.577853e-03
1,0.990138,9.755548e-03,1.060881e-04
2,0.759890,2.400113e-01,9.896529e-05
3,0.000066,3.849521e-04,9.995494e-01
4,0.325551,5.760931e-01,9.835556e-02
...,...,...,...
112,0.999644,3.559579e-04,7.032389e-14
113,0.000018,9.999825e-01,3.212149e-08
114,0.000001,6.725805e-11,9.999989e-01
115,0.000066,3.849521e-04,9.995494e-01


In [64]:
dat_proba4.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba4.csv", index = False,
         header=None)

In [65]:
dat4.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/4p006p.csv", index = False,
         header=None)

In [126]:
hist1_over4 = model1_over4.fit(X_train_over, y_train_over,
          batch_size=64, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 270 samples, validate on 117 samples
Epoch 1/1000
270/270 [==============================] - 0s 96us/step - loss: 0.2567 - accuracy: 0.8704 - val_loss: 1.6927 - val_accuracy: 0.6838
Epoch 2/1000
270/270 [==============================] - 0s 74us/step - loss: 0.2453 - accuracy: 0.8778 - val_loss: 1.6684 - val_accuracy: 0.6838
Epoch 3/1000
270/270 [==============================] - 0s 111us/step - loss: 0.2507 - accuracy: 0.8778 - val_loss: 1.6384 - val_accuracy: 0.6496
Epoch 4/1000
270/270 [==============================] - 0s 105us/step - loss: 0.2443 - accuracy: 0.8815 - val_loss: 1.6457 - val_accuracy: 0.7265
Epoch 5/1000
270/270 [==============================] - 0s 90us/step - loss: 0.2530 - accuracy: 0.8815 - val_loss: 1.6767 - val_accuracy: 0.7265
Epoch 6/1000
270/270 [==============================] - 0s 94us/step - loss: 0.2512 - accuracy: 0.8815 - val_loss: 1.6787 - val_accuracy: 0.7265
Epoch 7/1000
270/270 [==============================] - 0s 77us/step - loss: 0.243

270/270 [==============================] - 0s 61us/step - loss: 0.2433 - accuracy: 0.8778 - val_loss: 1.7308 - val_accuracy: 0.6752
Epoch 113/1000
270/270 [==============================] - 0s 61us/step - loss: 0.2474 - accuracy: 0.8815 - val_loss: 1.7148 - val_accuracy: 0.6838
Epoch 114/1000
270/270 [==============================] - 0s 60us/step - loss: 0.2433 - accuracy: 0.8593 - val_loss: 1.6865 - val_accuracy: 0.7265
Epoch 115/1000
270/270 [==============================] - 0s 55us/step - loss: 0.2442 - accuracy: 0.8815 - val_loss: 1.6781 - val_accuracy: 0.7265
Epoch 116/1000
270/270 [==============================] - 0s 73us/step - loss: 0.2446 - accuracy: 0.8815 - val_loss: 1.6634 - val_accuracy: 0.7265
Epoch 117/1000
270/270 [==============================] - 0s 71us/step - loss: 0.2437 - accuracy: 0.8815 - val_loss: 1.6922 - val_accuracy: 0.7265
Epoch 118/1000
270/270 [==============================] - 0s 76us/step - loss: 0.2437 - accuracy: 0.8815 - val_loss: 1.7455 - val_acc

Epoch 168/1000
270/270 [==============================] - 0s 90us/step - loss: 0.2494 - accuracy: 0.8704 - val_loss: 1.7135 - val_accuracy: 0.7009
Epoch 169/1000
270/270 [==============================] - 0s 87us/step - loss: 0.2458 - accuracy: 0.8741 - val_loss: 1.6888 - val_accuracy: 0.7265
Epoch 170/1000
270/270 [==============================] - 0s 81us/step - loss: 0.2450 - accuracy: 0.8704 - val_loss: 1.7029 - val_accuracy: 0.6838
Epoch 171/1000
270/270 [==============================] - 0s 72us/step - loss: 0.2434 - accuracy: 0.8778 - val_loss: 1.6842 - val_accuracy: 0.7265
Epoch 172/1000
270/270 [==============================] - 0s 107us/step - loss: 0.2472 - accuracy: 0.8741 - val_loss: 1.6668 - val_accuracy: 0.7265
Epoch 173/1000
270/270 [==============================] - 0s 74us/step - loss: 0.2486 - accuracy: 0.8778 - val_loss: 1.6729 - val_accuracy: 0.7265
Epoch 174/1000
270/270 [==============================] - 0s 88us/step - loss: 0.2450 - accuracy: 0.8778 - val_loss: 

Epoch 224/1000
270/270 [==============================] - 0s 76us/step - loss: 0.2454 - accuracy: 0.8741 - val_loss: 1.7247 - val_accuracy: 0.6838
Epoch 225/1000
270/270 [==============================] - 0s 55us/step - loss: 0.2438 - accuracy: 0.8852 - val_loss: 1.7416 - val_accuracy: 0.6838
Epoch 226/1000
270/270 [==============================] - 0s 58us/step - loss: 0.2440 - accuracy: 0.8815 - val_loss: 1.7388 - val_accuracy: 0.7265
Epoch 227/1000
270/270 [==============================] - 0s 63us/step - loss: 0.2465 - accuracy: 0.8815 - val_loss: 1.7482 - val_accuracy: 0.7179
Epoch 228/1000
270/270 [==============================] - 0s 70us/step - loss: 0.2434 - accuracy: 0.8778 - val_loss: 1.7637 - val_accuracy: 0.6838
Epoch 229/1000
270/270 [==============================] - 0s 72us/step - loss: 0.2482 - accuracy: 0.8667 - val_loss: 1.7443 - val_accuracy: 0.6838
Epoch 230/1000
270/270 [==============================] - 0s 74us/step - loss: 0.2507 - accuracy: 0.8593 - val_loss: 1

Epoch 280/1000
270/270 [==============================] - 0s 68us/step - loss: 0.2476 - accuracy: 0.8815 - val_loss: 1.7545 - val_accuracy: 0.7265
Epoch 281/1000
270/270 [==============================] - 0s 68us/step - loss: 0.2479 - accuracy: 0.8741 - val_loss: 1.7702 - val_accuracy: 0.7265
Epoch 282/1000
270/270 [==============================] - 0s 76us/step - loss: 0.2441 - accuracy: 0.8815 - val_loss: 1.7952 - val_accuracy: 0.7265
Epoch 283/1000
270/270 [==============================] - 0s 75us/step - loss: 0.2442 - accuracy: 0.8815 - val_loss: 1.7946 - val_accuracy: 0.7265
Epoch 284/1000
270/270 [==============================] - 0s 72us/step - loss: 0.2429 - accuracy: 0.8815 - val_loss: 1.7834 - val_accuracy: 0.7265
Epoch 285/1000
270/270 [==============================] - 0s 67us/step - loss: 0.2431 - accuracy: 0.8741 - val_loss: 1.8004 - val_accuracy: 0.6838
Epoch 286/1000
270/270 [==============================] - 0s 73us/step - loss: 0.2454 - accuracy: 0.8667 - val_loss: 1

...

Epoch 556/1000
270/270 [==============================] - 0s 75us/step - loss: 0.2511 - accuracy: 0.8704 - val_loss: 1.9140 - val_accuracy: 0.6838
Epoch 557/1000
270/270 [==============================] - 0s 76us/step - loss: 0.2410 - accuracy: 0.8778 - val_loss: 1.9409 - val_accuracy: 0.6838
Epoch 558/1000
270/270 [==============================] - 0s 156us/step - loss: 0.2456 - accuracy: 0.8741 - val_loss: 1.9463 - val_accuracy: 0.7009
Epoch 559/1000
270/270 [==============================] - ETA: 0s - loss: 0.1183 - accuracy: 0.96 - 0s 168us/step - loss: 0.2474 - accuracy: 0.8704 - val_loss: 1.9037 - val_accuracy: 0.7436
Epoch 560/1000
270/270 [==============================] - 0s 104us/step - loss: 0.2483 - accuracy: 0.8741 - val_loss: 1.8544 - val_accuracy: 0.7265
Epoch 561/1000
270/270 [==============================] - 0s 156us/step - loss: 0.2571 - accuracy: 0.8741 - val_loss: 1.8289 - val_accuracy: 0.7265
Epoch 562/1000
270/270 [==============================] - 0s 75us/step -

270/270 [==============================] - 0s 203us/step - loss: 0.2438 - accuracy: 0.8815 - val_loss: 1.9764 - val_accuracy: 0.7265
Epoch 667/1000
270/270 [==============================] - 0s 157us/step - loss: 0.2420 - accuracy: 0.8815 - val_loss: 1.9914 - val_accuracy: 0.7265
Epoch 668/1000
270/270 [==============================] - 0s 123us/step - loss: 0.2431 - accuracy: 0.8815 - val_loss: 1.9809 - val_accuracy: 0.7265
Epoch 669/1000
270/270 [==============================] - 0s 77us/step - loss: 0.2426 - accuracy: 0.8778 - val_loss: 1.9937 - val_accuracy: 0.6838
Epoch 670/1000
270/270 [==============================] - 0s 137us/step - loss: 0.2412 - accuracy: 0.8852 - val_loss: 1.9996 - val_accuracy: 0.6838
Epoch 671/1000
270/270 [==============================] - 0s 70us/step - loss: 0.2424 - accuracy: 0.8778 - val_loss: 1.9978 - val_accuracy: 0.6838
Epoch 672/1000
270/270 [==============================] - 0s 157us/step - loss: 0.2402 - accuracy: 0.8852 - val_loss: 1.9647 - va

270/270 [==============================] - 0s 154us/step - loss: 0.2551 - accuracy: 0.8815 - val_loss: 2.0837 - val_accuracy: 0.7265
Epoch 777/1000
270/270 [==============================] - 0s 167us/step - loss: 0.2482 - accuracy: 0.8815 - val_loss: 2.0467 - val_accuracy: 0.7265
Epoch 778/1000
270/270 [==============================] - 0s 164us/step - loss: 0.2467 - accuracy: 0.8815 - val_loss: 2.0041 - val_accuracy: 0.7265
Epoch 779/1000
270/270 [==============================] - 0s 96us/step - loss: 0.2483 - accuracy: 0.8778 - val_loss: 2.0152 - val_accuracy: 0.7265
Epoch 780/1000
270/270 [==============================] - 0s 98us/step - loss: 0.2465 - accuracy: 0.8778 - val_loss: 2.0325 - val_accuracy: 0.7265
Epoch 781/1000
270/270 [==============================] - 0s 113us/step - loss: 0.2448 - accuracy: 0.8852 - val_loss: 2.0682 - val_accuracy: 0.6838
Epoch 782/1000
270/270 [==============================] - 0s 128us/step - loss: 0.2451 - accuracy: 0.8778 - val_loss: 2.0677 - va

270/270 [==============================] - 0s 80us/step - loss: 0.2444 - accuracy: 0.8778 - val_loss: 2.0712 - val_accuracy: 0.7265
Epoch 887/1000
270/270 [==============================] - 0s 91us/step - loss: 0.2455 - accuracy: 0.8815 - val_loss: 2.0988 - val_accuracy: 0.7265
Epoch 888/1000
270/270 [==============================] - 0s 58us/step - loss: 0.2467 - accuracy: 0.8815 - val_loss: 2.1106 - val_accuracy: 0.7265
Epoch 889/1000
270/270 [==============================] - 0s 59us/step - loss: 0.2431 - accuracy: 0.8815 - val_loss: 2.0987 - val_accuracy: 0.7265
Epoch 890/1000
270/270 [==============================] - 0s 60us/step - loss: 0.2451 - accuracy: 0.8741 - val_loss: 2.0956 - val_accuracy: 0.7265
Epoch 891/1000
270/270 [==============================] - 0s 67us/step - loss: 0.2461 - accuracy: 0.8778 - val_loss: 2.0971 - val_accuracy: 0.7265
Epoch 892/1000
270/270 [==============================] - 0s 66us/step - loss: 0.2443 - accuracy: 0.8778 - val_loss: 2.0976 - val_acc

270/270 [==============================] - 0s 106us/step - loss: 0.2511 - accuracy: 0.8815 - val_loss: 2.1962 - val_accuracy: 0.7265
Epoch 997/1000
270/270 [==============================] - 0s 108us/step - loss: 0.2428 - accuracy: 0.8815 - val_loss: 2.2082 - val_accuracy: 0.7265
Epoch 998/1000
270/270 [==============================] - 0s 90us/step - loss: 0.2489 - accuracy: 0.8481 - val_loss: 2.2088 - val_accuracy: 0.6496
Epoch 999/1000
270/270 [==============================] - 0s 72us/step - loss: 0.2450 - accuracy: 0.8741 - val_loss: 2.2094 - val_accuracy: 0.7265
Epoch 1000/1000
270/270 [==============================] - 0s 70us/step - loss: 0.2415 - accuracy: 0.8815 - val_loss: 2.2115 - val_accuracy: 0.7265


In [127]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over4.history['accuracy'])*100))

over-sampling train accuracy: 87.63%


In [49]:
df_proba4 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_2.xlsx",
                        sheet_name=3,
                        index_col=None)

In [50]:
df_proba4

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabs_qual,NRS110,1,1,0.000003,0.999997,5.870196e-13
1,p002ykpresabs_qual,NRS216,1,1,0.039254,0.960745,9.078969e-07
2,p002ykpresabs_qual,NRS386,1,1,0.326752,0.673248,1.061032e-07
3,p002ykpresabs_qual,CFBRSa25,0,0,0.611084,0.388916,7.664974e-07
4,p002ykpresabs_qual,BCH-SA-03,1,0,0.611084,0.388916,7.664974e-07
...,...,...,...,...,...,...,...
4279,pyopresabsSTCC_qual,NRS236,1,1,0.000052,0.999768,1.803156e-04
4280,pyopresabsSTCC_qual,NRS029,0,1,0.322350,0.677496,1.533154e-04
4281,pyopresabsSTCC_qual,NRS148,2,2,0.000006,0.000026,9.999682e-01
4282,pyopresabsSTCC_qual,CFBRSa28,0,0,0.999288,0.000176,5.361527e-04


In [51]:
y_prob4 = df_proba4[df_proba4['phage']=='p0006presabs_qual'].iloc[:,-3:]
y_prob4 = y_prob4.to_numpy()
y_prob4

array([[1.27998140e-01, 8.66424000e-01, 5.57785340e-03],
       [9.90138300e-01, 9.75554800e-03, 1.06088126e-04],
       [7.59889660e-01, 2.40011300e-01, 9.89652900e-05],
       [6.56236100e-05, 3.84952150e-04, 9.99549450e-01],
       [3.25551450e-01, 5.76093100e-01, 9.83555600e-02],
       [1.00000000e+00, 1.19036870e-11, 1.36924656e-11],
       [8.18993100e-11, 1.04420610e-11, 1.00000000e+00],
       [9.99537940e-01, 4.62038150e-04, 5.37932000e-10],
       [1.45181120e-01, 1.13316300e-01, 7.41502500e-01],
       [3.25551450e-01, 5.76093100e-01, 9.83555600e-02],
       [9.92524800e-01, 7.43914160e-03, 3.60887900e-05],
       [6.19076200e-04, 3.13929620e-01, 6.85451300e-01],
       [1.45181120e-01, 1.13316300e-01, 7.41502500e-01],
       [9.09975850e-04, 9.99086600e-01, 3.47870420e-06],
       [3.20201500e-04, 9.99228100e-01, 4.51702100e-04],
       [5.00588700e-08, 1.13294950e-01, 8.86705000e-01],
       [5.95785150e-15, 1.28512440e-13, 1.00000000e+00],
       [3.50018400e-07, 1.60826

In [52]:
ovo4 = rocauc_ovo(y_test_over, y_prob4, average="macro", multi_class="ovo")
ovo4

0.8474687705456937

In [53]:
ovr4 = rocauc_ovr(y_test_over, y_prob4, average="macro", multi_class="ovr")
ovr4

0.8474687705456937

In [54]:
ovos = [ovo1, ovo2, ovo3, ovo4]
np.mean(ovos)

0.8685897435897436

In [55]:
np.std(ovos)

0.015545149319474582

In [56]:
ovrs = [ovr1, ovr2, ovr3, ovr4]
np.mean(ovrs)

0.8685897435897436

In [57]:
np.std(ovrs)

0.015545149319474582

In [128]:
accs = [acc_test_over, acc_test_over2, acc_test_over3, acc_test_over4]

In [129]:
mean = np.mean(accs)
print('over-sampling test accuracy mean: %.2f%%' % (mean*100))

over-sampling test accuracy mean: 72.01%


In [130]:
std = np.std(accs)
print('over-sampling test accuracy standard deviation:', std)

over-sampling test accuracy standard deviation: 0.027941658229038786


In [131]:
accs_train = [np.mean(hist1_over.history['accuracy']), np.mean(hist1_over2.history['accuracy']), np.mean(hist1_over3.history['accuracy']),
             np.mean(hist1_over4.history['accuracy'])]

In [132]:
mean_train = np.mean(accs_train)
print('over-sampling train accuracy mean: %.2f%%' % (mean_train*100))

over-sampling train accuracy mean: 88.18%


In [133]:
std_train = np.std(accs_train)
print('over-sampling train accuracy standard deviation:', std_train)

over-sampling train accuracy standard deviation: 0.008183083


In [ ]:
############ Feature selection using lasso ##########

In [17]:
## Retrieved from https://towardsdatascience.com/feature-selection-using-regularisation-a3678b71e499
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
import numpy as np

In [18]:
selection = SelectFromModel(LogisticRegression(C=1, penalty='l1', solver='liblinear'))
selection.fit(X_over[:,1:], y_over)

SelectFromModel(estimator=LogisticRegression(C=1, class_weight=None, dual=False,
                                             fit_intercept=True,
                                             intercept_scaling=1, l1_ratio=None,
                                             max_iter=100, multi_class='auto',
                                             n_jobs=None, penalty='l1',
                                             random_state=None,
                                             solver='liblinear', tol=0.0001,
                                             verbose=0, warm_start=False),
                max_features=None, norm_order=1, prefit=False, threshold=None)

In [19]:
names = np.array(df_clean.columns).tolist()
names.remove('pheno')

In [20]:
X_train_features_over = np.vstack((names, X_over[:,1:]))
X_train_features_over = pd.DataFrame(X_train_features_over)

In [21]:
sel_features = X_train_features_over.columns[(selection.get_support())]
print('total features: {}'.format((X_train_features_over.shape[1])))
print('selected features: {}'.format(len(sel_features)))

total features: 94
selected features: 61


In [22]:
cols = sel_features.values
cols.reshape((1, -1))

array([[ 1,  2,  3,  4,  5,  6,  7,  9, 10, 11, 12, 13, 15, 16, 17, 18,
        20, 21, 22, 23, 24, 25, 27, 29, 31, 32, 34, 38, 40, 41, 42, 45,
        46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61,
        64, 67, 80, 81, 82, 83, 84, 85, 86, 87, 88, 91, 93]])

In [23]:
names_arr = np.array(names)
names_arr[cols]

array(['TTAATTTAATAGA', 'TTAACATAATAAT', 'TGCAATCTCTTTAT',
       'TATTATGTTAATG', 'TACATACCGAT', 'GTGTATCATAAT', 'GCTGTTGAAATGGC',
       'GAGTCCTGTT', 'GAGTCCTGTTT',
       'GACAAACATGTATTAGCGTTATGTCGCGAACATCATAACCAGCAACATGCGATTGGCGTTAAGTCGTTTGATGATAAATATCACTTGCATGACTCGTGG',
       'CTTTTTCACCTGT', 'CTTGTGAATTTAG', 'CGCCATTATGTT', 'CAGAAAAGCGT',
       'ACAATTACTATATTT', 'X1_102334_C_T', 'X1_143918_T_C',
       'X1_144092_A_G', 'X1_144113_T_C', 'X1_238886_T_C', 'X1_287706_A_G',
       'X1_327716_C_T', 'X1_449085_T_A', 'X1_569187_A_G', 'X1_655318_C_T',
       'X1_655324_T_C', 'X1_655369_T_A', 'X1_698337_T_C', 'X1_791797_A_T',
       'X1_810661_T_C', 'X1_829088_C_T', 'kdpA', 'group_5991',
       'group_3022', 'sdrE_1', 'group_1251', 'group_482', 'group_6857',
       'group_1087', 'group_2822', 'group_1748', 'clpA', 'group_3572',
       'group_3937', 'group_1320', 'group_34', 'dus_1', 'group_9051',
       'group_6900', 'group_7995', 'group_11094', 'group_7803',
       'group_870', 'grou

In [24]:
###### keep selected variables as a new dataframe
df_sel = df_clean.loc[:,names_arr[cols]].copy()
df_sel['pheno'] = df_clean['pheno']

In [25]:
df_sel['strain'] = X.iloc[:,0]

In [26]:
df_sel

,TTAATTTAATAGA,TTAACATAATAAT,TGCAATCTCTTTAT,TATTATGTTAATG,TACATACCGAT,GTGTATCATAAT,GCTGTTGAAATGGC,GAGTCCTGTT,GAGTCCTGTTT,GACAAACATGTATTAGCGTTATGTCGCGAACATCATAACCAGCAACATGCGATTGGCGTTAAGTCGTTTGATGATAAATATCACTTGCATGACTCGTGG,...,group_465,group_2236,group_7824,group_7845,group_10140,group_10143,group_7172,group_9586,pheno,strain
0,1,1,1,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,1,107
1,1,1,1,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,109
2,1,1,1,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,1,115
3,1,1,1,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,1,120335
4,1,1,1,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,1,120337
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,1,1,1,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,1,SR4152
249,1,1,1,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,SR4153
250,1,1,1,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,SR4155
251,1,1,1,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,SR4156


In [27]:
X_sel = df_sel.loc[:, df_sel.columns != 'pheno']
y_sel = df_sel['pheno']
print(X_sel.shape, y_sel.shape, df_sel.shape)

(253, 62) (253,) (253, 63)


In [28]:
df_sel['pheno'].value_counts()

0    129
1     85
2     39
Name: pheno, dtype: int64

In [29]:
# over-sampling
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_sel_over, y_sel_over = overS.fit_resample(X_sel, y_sel)
print(sorted(Counter(y_sel_over).items()))

[(0, 129), (1, 129), (2, 129)]


/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [30]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_sel_train_over, X_sel_test_over, y_sel_train_over, y_sel_test_over = train_test_split(X_sel_over, y_sel_over,
                                                    test_size = 0.3,
                                                    random_state=567,
                                                    stratify=y_sel_over)

In [31]:
dat5 = pd.DataFrame(X_sel_test_over[:,-1])
dat5['test'] = y_sel_test_over

In [32]:
dat5

,0,test
0,NRS245,1
1,NY439,2
2,CA544,1
3,CA541,2
4,EUH15,1
...,...,...
112,NRS112,0
113,CFBRSa51,2
114,NRS383,1
115,NRS247,0


In [33]:
X_sel_train_over = X_sel_train_over[:,:-1]
X_sel_test_over = X_sel_test_over[:,:-1]

In [83]:
#### neural network on over-sampling data
model2_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [84]:
model2_over.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [85]:
model2_over.fit(X_sel_train_over, y_sel_train_over,
          batch_size=64, epochs=1000,
          validation_data=(X_sel_test_over, y_sel_test_over))

Train on 270 samples, validate on 117 samples
Epoch 1/1000
270/270 [==============================] - 0s 402us/step - loss: 1.0975 - accuracy: 0.3815 - val_loss: 1.0901 - val_accuracy: 0.3590
Epoch 2/1000
270/270 [==============================] - 0s 61us/step - loss: 1.0489 - accuracy: 0.4852 - val_loss: 1.0577 - val_accuracy: 0.4359
Epoch 3/1000
270/270 [==============================] - 0s 76us/step - loss: 1.0154 - accuracy: 0.5222 - val_loss: 1.0306 - val_accuracy: 0.5043
Epoch 4/1000
270/270 [==============================] - 0s 98us/step - loss: 0.9875 - accuracy: 0.5593 - val_loss: 1.0091 - val_accuracy: 0.5556
Epoch 5/1000
270/270 [==============================] - 0s 96us/step - loss: 0.9637 - accuracy: 0.5741 - val_loss: 0.9904 - val_accuracy: 0.5983
Epoch 6/1000
270/270 [==============================] - 0s 102us/step - loss: 0.9450 - accuracy: 0.6185 - val_loss: 0.9731 - val_accuracy: 0.6154
Epoch 7/1000
270/270 [==============================] - 0s 94us/step - loss: 0.926

Epoch 57/1000
270/270 [==============================] - 0s 73us/step - loss: 0.6067 - accuracy: 0.7481 - val_loss: 0.8222 - val_accuracy: 0.6667
Epoch 58/1000
270/270 [==============================] - 0s 75us/step - loss: 0.6019 - accuracy: 0.7333 - val_loss: 0.8199 - val_accuracy: 0.6581
Epoch 59/1000
270/270 [==============================] - 0s 103us/step - loss: 0.5998 - accuracy: 0.7333 - val_loss: 0.8196 - val_accuracy: 0.6581
Epoch 60/1000
270/270 [==============================] - 0s 70us/step - loss: 0.5971 - accuracy: 0.7333 - val_loss: 0.8193 - val_accuracy: 0.6581
Epoch 61/1000
270/270 [==============================] - 0s 63us/step - loss: 0.5939 - accuracy: 0.7333 - val_loss: 0.8190 - val_accuracy: 0.6667
Epoch 62/1000
270/270 [==============================] - 0s 65us/step - loss: 0.5909 - accuracy: 0.7333 - val_loss: 0.8181 - val_accuracy: 0.6667
Epoch 63/1000
270/270 [==============================] - 0s 62us/step - loss: 0.5877 - accuracy: 0.7333 - val_loss: 0.8172 

Epoch 168/1000
270/270 [==============================] - 0s 69us/step - loss: 0.4185 - accuracy: 0.8481 - val_loss: 0.8517 - val_accuracy: 0.7094
Epoch 169/1000
270/270 [==============================] - 0s 107us/step - loss: 0.4201 - accuracy: 0.8481 - val_loss: 0.8424 - val_accuracy: 0.7179
Epoch 170/1000
270/270 [==============================] - 0s 91us/step - loss: 0.4171 - accuracy: 0.8296 - val_loss: 0.8433 - val_accuracy: 0.7265
Epoch 171/1000
270/270 [==============================] - 0s 98us/step - loss: 0.4151 - accuracy: 0.8481 - val_loss: 0.8502 - val_accuracy: 0.7179
Epoch 172/1000
270/270 [==============================] - 0s 121us/step - loss: 0.4147 - accuracy: 0.8481 - val_loss: 0.8532 - val_accuracy: 0.7179
Epoch 173/1000
270/270 [==============================] - 0s 113us/step - loss: 0.4141 - accuracy: 0.8444 - val_loss: 0.8475 - val_accuracy: 0.7009
Epoch 174/1000
270/270 [==============================] - 0s 93us/step - loss: 0.4130 - accuracy: 0.8444 - val_loss

270/270 [==============================] - 0s 60us/step - loss: 0.3455 - accuracy: 0.8593 - val_loss: 0.9078 - val_accuracy: 0.7265
Epoch 279/1000
270/270 [==============================] - 0s 55us/step - loss: 0.3466 - accuracy: 0.8630 - val_loss: 0.9050 - val_accuracy: 0.7265
Epoch 280/1000
270/270 [==============================] - 0s 64us/step - loss: 0.3449 - accuracy: 0.8630 - val_loss: 0.9102 - val_accuracy: 0.7265
Epoch 281/1000
270/270 [==============================] - 0s 65us/step - loss: 0.3444 - accuracy: 0.8630 - val_loss: 0.9196 - val_accuracy: 0.7265
Epoch 282/1000
270/270 [==============================] - 0s 57us/step - loss: 0.3455 - accuracy: 0.8630 - val_loss: 0.9212 - val_accuracy: 0.7265
Epoch 283/1000
270/270 [==============================] - 0s 73us/step - loss: 0.3439 - accuracy: 0.8630 - val_loss: 0.9114 - val_accuracy: 0.7350
Epoch 284/1000
270/270 [==============================] - 0s 70us/step - loss: 0.3448 - accuracy: 0.8481 - val_loss: 0.9024 - val_acc

Epoch 334/1000
270/270 [==============================] - 0s 50us/step - loss: 0.3286 - accuracy: 0.8630 - val_loss: 0.9422 - val_accuracy: 0.7265
Epoch 335/1000
270/270 [==============================] - 0s 56us/step - loss: 0.3250 - accuracy: 0.8630 - val_loss: 0.9345 - val_accuracy: 0.7350
Epoch 336/1000
270/270 [==============================] - 0s 59us/step - loss: 0.3243 - accuracy: 0.8667 - val_loss: 0.9375 - val_accuracy: 0.7350
Epoch 337/1000
270/270 [==============================] - 0s 52us/step - loss: 0.3242 - accuracy: 0.8667 - val_loss: 0.9427 - val_accuracy: 0.7350
Epoch 338/1000
270/270 [==============================] - 0s 63us/step - loss: 0.3239 - accuracy: 0.8667 - val_loss: 0.9416 - val_accuracy: 0.7350
Epoch 339/1000
270/270 [==============================] - 0s 56us/step - loss: 0.3236 - accuracy: 0.8704 - val_loss: 0.9340 - val_accuracy: 0.7350
Epoch 340/1000
270/270 [==============================] - 0s 59us/step - loss: 0.3240 - accuracy: 0.8667 - val_loss: 0

...

Epoch 614/1000
270/270 [==============================] - 0s 80us/step - loss: 0.2789 - accuracy: 0.8704 - val_loss: 1.0374 - val_accuracy: 0.7436
Epoch 615/1000
270/270 [==============================] - 0s 76us/step - loss: 0.2790 - accuracy: 0.8704 - val_loss: 1.0406 - val_accuracy: 0.7436
Epoch 616/1000
270/270 [==============================] - 0s 92us/step - loss: 0.2792 - accuracy: 0.8667 - val_loss: 1.0589 - val_accuracy: 0.7436
Epoch 617/1000
270/270 [==============================] - 0s 105us/step - loss: 0.2808 - accuracy: 0.8667 - val_loss: 1.0616 - val_accuracy: 0.7436
Epoch 618/1000
270/270 [==============================] - 0s 106us/step - loss: 0.2776 - accuracy: 0.8704 - val_loss: 1.0435 - val_accuracy: 0.7436
Epoch 619/1000
270/270 [==============================] - 0s 91us/step - loss: 0.2777 - accuracy: 0.8667 - val_loss: 1.0422 - val_accuracy: 0.7436
Epoch 620/1000
270/270 [==============================] - 0s 108us/step - loss: 0.2775 - accuracy: 0.8704 - val_loss

Epoch 725/1000
270/270 [==============================] - 0s 56us/step - loss: 0.2725 - accuracy: 0.8704 - val_loss: 1.0911 - val_accuracy: 0.7350
Epoch 726/1000
270/270 [==============================] - 0s 42us/step - loss: 0.2741 - accuracy: 0.8667 - val_loss: 1.0959 - val_accuracy: 0.7436
Epoch 727/1000
270/270 [==============================] - 0s 46us/step - loss: 0.2708 - accuracy: 0.8704 - val_loss: 1.1033 - val_accuracy: 0.7436
Epoch 728/1000
270/270 [==============================] - 0s 45us/step - loss: 0.2745 - accuracy: 0.8667 - val_loss: 1.1272 - val_accuracy: 0.7436
Epoch 729/1000
270/270 [==============================] - 0s 48us/step - loss: 0.2737 - accuracy: 0.8704 - val_loss: 1.1239 - val_accuracy: 0.7436
Epoch 730/1000
270/270 [==============================] - 0s 39us/step - loss: 0.2716 - accuracy: 0.8704 - val_loss: 1.0998 - val_accuracy: 0.7436
Epoch 731/1000
270/270 [==============================] - 0s 46us/step - loss: 0.2719 - accuracy: 0.8667 - val_loss: 1

Epoch 781/1000
270/270 [==============================] - 0s 82us/step - loss: 0.2676 - accuracy: 0.8704 - val_loss: 1.0904 - val_accuracy: 0.7350
Epoch 782/1000
270/270 [==============================] - 0s 63us/step - loss: 0.2676 - accuracy: 0.8667 - val_loss: 1.0959 - val_accuracy: 0.7350
Epoch 783/1000
270/270 [==============================] - 0s 56us/step - loss: 0.2677 - accuracy: 0.8704 - val_loss: 1.1123 - val_accuracy: 0.7350
Epoch 784/1000
270/270 [==============================] - 0s 55us/step - loss: 0.2680 - accuracy: 0.8704 - val_loss: 1.0975 - val_accuracy: 0.7350
Epoch 785/1000
270/270 [==============================] - 0s 77us/step - loss: 0.2679 - accuracy: 0.8704 - val_loss: 1.0913 - val_accuracy: 0.7350
Epoch 786/1000
270/270 [==============================] - 0s 69us/step - loss: 0.2672 - accuracy: 0.8704 - val_loss: 1.0923 - val_accuracy: 0.7350
Epoch 787/1000
270/270 [==============================] - 0s 81us/step - loss: 0.2677 - accuracy: 0.8630 - val_loss: 1

Epoch 837/1000
270/270 [==============================] - 0s 56us/step - loss: 0.2670 - accuracy: 0.8630 - val_loss: 1.1412 - val_accuracy: 0.7350
Epoch 838/1000
270/270 [==============================] - 0s 42us/step - loss: 0.2666 - accuracy: 0.8704 - val_loss: 1.1368 - val_accuracy: 0.7350
Epoch 839/1000
270/270 [==============================] - 0s 44us/step - loss: 0.2653 - accuracy: 0.8704 - val_loss: 1.1338 - val_accuracy: 0.7350
Epoch 840/1000
270/270 [==============================] - 0s 50us/step - loss: 0.2655 - accuracy: 0.8704 - val_loss: 1.1406 - val_accuracy: 0.7350
Epoch 841/1000
270/270 [==============================] - 0s 51us/step - loss: 0.2654 - accuracy: 0.8704 - val_loss: 1.1339 - val_accuracy: 0.7350
Epoch 842/1000
270/270 [==============================] - 0s 58us/step - loss: 0.2659 - accuracy: 0.8704 - val_loss: 1.1290 - val_accuracy: 0.7350
Epoch 843/1000
270/270 [==============================] - 0s 54us/step - loss: 0.2650 - accuracy: 0.8704 - val_loss: 1

Epoch 893/1000
270/270 [==============================] - 0s 81us/step - loss: 0.2669 - accuracy: 0.8704 - val_loss: 1.1492 - val_accuracy: 0.7436
Epoch 894/1000
270/270 [==============================] - 0s 87us/step - loss: 0.2653 - accuracy: 0.8704 - val_loss: 1.1440 - val_accuracy: 0.7350
Epoch 895/1000
270/270 [==============================] - 0s 79us/step - loss: 0.2637 - accuracy: 0.8704 - val_loss: 1.1580 - val_accuracy: 0.7350
Epoch 896/1000
270/270 [==============================] - 0s 72us/step - loss: 0.2660 - accuracy: 0.8704 - val_loss: 1.1590 - val_accuracy: 0.7350
Epoch 897/1000
270/270 [==============================] - 0s 83us/step - loss: 0.2661 - accuracy: 0.8704 - val_loss: 1.1466 - val_accuracy: 0.7350
Epoch 898/1000
270/270 [==============================] - 0s 87us/step - loss: 0.2645 - accuracy: 0.8704 - val_loss: 1.1381 - val_accuracy: 0.7521
Epoch 899/1000
270/270 [==============================] - 0s 59us/step - loss: 0.2663 - accuracy: 0.8630 - val_loss: 1

In [166]:
acc_test2_over = model2_over.evaluate(X_sel_test_over, y_sel_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test2_over*100))

117/117 [==============================] - 0s 64us/step
over-sampling test accuracy: 75.21%


In [86]:
pred5 = model2_over.predict_classes(X_sel_test_over)
pred5

array([1, 2, 1, 1, 1, 2, 1, 0, 0, 2, 0, 2, 2, 1, 2, 2, 2, 2, 2, 0, 2, 1,
       0, 2, 2, 2, 1, 1, 1, 1, 2, 1, 1, 0, 2, 0, 0, 2, 0, 0, 2, 2, 1, 1,
       1, 2, 2, 0, 2, 1, 0, 2, 2, 2, 1, 2, 0, 0, 1, 1, 0, 2, 1, 2, 0, 0,
       2, 1, 0, 2, 2, 1, 0, 1, 2, 1, 2, 0, 1, 0, 2, 0, 0, 2, 0, 2, 1, 0,
       1, 0, 0, 2, 0, 0, 1, 2, 2, 2, 0, 2, 2, 1, 2, 0, 0, 1, 0, 1, 0, 1,
       2, 0, 0, 1, 0, 0, 0])

In [87]:
dat5['pred'] = pred5
dat5

,0,test,pred
0,NRS245,1,1
1,NY439,2,2
2,CA544,1,1
3,CA541,2,1
4,EUH15,1,1
...,...,...,...
112,NRS112,0,0
113,CFBRSa51,2,1
114,NRS383,1,0
115,NRS247,0,0


In [88]:
proba5 = model2_over.predict_proba(X_sel_test_over)
dat_proba5 = pd.DataFrame(proba5)

In [89]:
dat_proba5

,0,1,2
0,0.113622,0.870446,0.015932
1,0.052619,0.000876,0.946505
2,0.100919,0.583063,0.316018
3,0.100919,0.583063,0.316018
4,0.346191,0.644950,0.008858
...,...,...,...
112,0.909846,0.089374,0.000780
113,0.100919,0.583063,0.316018
114,0.909846,0.089374,0.000780
115,0.960853,0.039130,0.000017


In [90]:
dat_proba5.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba5.csv", index = False,
         header=None)

In [91]:
dat5.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/5p006p.csv", index = False,
         header=None)

In [170]:
hist2_over = model2_over.fit(X_sel_train_over, y_sel_train_over,
          batch_size=64, epochs=1000,
          validation_data=(X_sel_test_over, y_sel_test_over))

Train on 270 samples, validate on 117 samples
Epoch 1/1000
270/270 [==============================] - 0s 82us/step - loss: 0.2655 - accuracy: 0.8778 - val_loss: 1.2036 - val_accuracy: 0.7607
Epoch 2/1000
270/270 [==============================] - 0s 82us/step - loss: 0.2650 - accuracy: 0.8704 - val_loss: 1.2063 - val_accuracy: 0.7607
Epoch 3/1000
270/270 [==============================] - 0s 73us/step - loss: 0.2649 - accuracy: 0.8667 - val_loss: 1.2027 - val_accuracy: 0.7607
Epoch 4/1000
270/270 [==============================] - 0s 92us/step - loss: 0.2650 - accuracy: 0.8704 - val_loss: 1.2005 - val_accuracy: 0.7607
Epoch 5/1000
270/270 [==============================] - 0s 97us/step - loss: 0.2648 - accuracy: 0.8704 - val_loss: 1.1975 - val_accuracy: 0.7607
Epoch 6/1000
270/270 [==============================] - 0s 82us/step - loss: 0.2640 - accuracy: 0.8704 - val_loss: 1.2035 - val_accuracy: 0.7607
Epoch 7/1000
270/270 [==============================] - 0s 83us/step - loss: 0.2643 

Epoch 57/1000
270/270 [==============================] - 0s 65us/step - loss: 0.2641 - accuracy: 0.8519 - val_loss: 1.1957 - val_accuracy: 0.7607
Epoch 58/1000
270/270 [==============================] - 0s 53us/step - loss: 0.2624 - accuracy: 0.8704 - val_loss: 1.1994 - val_accuracy: 0.7521
Epoch 59/1000
270/270 [==============================] - 0s 72us/step - loss: 0.2651 - accuracy: 0.8704 - val_loss: 1.2114 - val_accuracy: 0.7521
Epoch 60/1000
270/270 [==============================] - 0s 81us/step - loss: 0.2654 - accuracy: 0.8704 - val_loss: 1.2125 - val_accuracy: 0.7607
Epoch 61/1000
270/270 [==============================] - 0s 79us/step - loss: 0.2626 - accuracy: 0.8704 - val_loss: 1.2161 - val_accuracy: 0.7607
Epoch 62/1000
270/270 [==============================] - 0s 82us/step - loss: 0.2617 - accuracy: 0.8704 - val_loss: 1.2081 - val_accuracy: 0.7607
Epoch 63/1000
270/270 [==============================] - 0s 52us/step - loss: 0.2650 - accuracy: 0.8630 - val_loss: 1.2006 -

270/270 [==============================] - 0s 89us/step - loss: 0.2616 - accuracy: 0.8704 - val_loss: 1.2103 - val_accuracy: 0.7607
Epoch 114/1000
270/270 [==============================] - 0s 86us/step - loss: 0.2615 - accuracy: 0.8704 - val_loss: 1.2112 - val_accuracy: 0.7607
Epoch 115/1000
270/270 [==============================] - 0s 87us/step - loss: 0.2602 - accuracy: 0.8704 - val_loss: 1.2177 - val_accuracy: 0.7607
Epoch 116/1000
270/270 [==============================] - 0s 91us/step - loss: 0.2628 - accuracy: 0.8704 - val_loss: 1.2270 - val_accuracy: 0.7607
Epoch 117/1000
270/270 [==============================] - 0s 99us/step - loss: 0.2595 - accuracy: 0.8593 - val_loss: 1.2148 - val_accuracy: 0.7607
Epoch 118/1000
270/270 [==============================] - 0s 113us/step - loss: 0.2622 - accuracy: 0.8704 - val_loss: 1.2074 - val_accuracy: 0.7607
Epoch 119/1000
270/270 [==============================] - 0s 73us/step - loss: 0.2619 - accuracy: 0.8704 - val_loss: 1.2085 - val_ac

Epoch 224/1000
270/270 [==============================] - 0s 153us/step - loss: 0.2586 - accuracy: 0.8704 - val_loss: 1.2622 - val_accuracy: 0.7607
Epoch 225/1000
270/270 [==============================] - 0s 108us/step - loss: 0.2599 - accuracy: 0.8704 - val_loss: 1.2603 - val_accuracy: 0.7607
Epoch 226/1000
270/270 [==============================] - 0s 116us/step - loss: 0.2581 - accuracy: 0.8704 - val_loss: 1.2580 - val_accuracy: 0.7607
Epoch 227/1000
270/270 [==============================] - 0s 200us/step - loss: 0.2606 - accuracy: 0.8667 - val_loss: 1.2640 - val_accuracy: 0.7778
Epoch 228/1000
270/270 [==============================] - 0s 108us/step - loss: 0.2594 - accuracy: 0.8630 - val_loss: 1.2759 - val_accuracy: 0.7607
Epoch 229/1000
270/270 [==============================] - 0s 69us/step - loss: 0.2622 - accuracy: 0.8704 - val_loss: 1.2824 - val_accuracy: 0.7607
Epoch 230/1000
270/270 [==============================] - 0s 73us/step - loss: 0.2589 - accuracy: 0.8704 - val_lo

270/270 [==============================] - 0s 101us/step - loss: 0.2575 - accuracy: 0.8704 - val_loss: 1.3136 - val_accuracy: 0.7607
Epoch 335/1000
270/270 [==============================] - 0s 58us/step - loss: 0.2571 - accuracy: 0.8778 - val_loss: 1.3113 - val_accuracy: 0.7607
Epoch 336/1000
270/270 [==============================] - 0s 69us/step - loss: 0.2573 - accuracy: 0.8704 - val_loss: 1.3088 - val_accuracy: 0.7607
Epoch 337/1000
270/270 [==============================] - 0s 67us/step - loss: 0.2593 - accuracy: 0.8704 - val_loss: 1.3204 - val_accuracy: 0.7607
Epoch 338/1000
270/270 [==============================] - 0s 59us/step - loss: 0.2591 - accuracy: 0.8704 - val_loss: 1.3163 - val_accuracy: 0.7607
Epoch 339/1000
270/270 [==============================] - 0s 69us/step - loss: 0.2569 - accuracy: 0.8704 - val_loss: 1.3029 - val_accuracy: 0.7607
Epoch 340/1000
270/270 [==============================] - 0s 66us/step - loss: 0.2603 - accuracy: 0.8704 - val_loss: 1.3004 - val_ac

...

Epoch 556/1000
270/270 [==============================] - 0s 102us/step - loss: 0.2592 - accuracy: 0.8630 - val_loss: 1.3938 - val_accuracy: 0.7778
Epoch 557/1000
270/270 [==============================] - 0s 109us/step - loss: 0.2575 - accuracy: 0.8704 - val_loss: 1.4009 - val_accuracy: 0.7607
Epoch 558/1000
270/270 [==============================] - 0s 113us/step - loss: 0.2555 - accuracy: 0.8667 - val_loss: 1.4006 - val_accuracy: 0.7521
Epoch 559/1000
270/270 [==============================] - 0s 73us/step - loss: 0.2568 - accuracy: 0.8630 - val_loss: 1.4039 - val_accuracy: 0.7350
Epoch 560/1000
270/270 [==============================] - 0s 69us/step - loss: 0.2591 - accuracy: 0.8667 - val_loss: 1.4016 - val_accuracy: 0.7436
Epoch 561/1000
270/270 [==============================] - 0s 66us/step - loss: 0.2573 - accuracy: 0.8630 - val_loss: 1.3805 - val_accuracy: 0.7607
Epoch 562/1000
270/270 [==============================] - 0s 67us/step - loss: 0.2576 - accuracy: 0.8704 - val_loss

270/270 [==============================] - 0s 71us/step - loss: 0.2566 - accuracy: 0.8704 - val_loss: 1.4172 - val_accuracy: 0.7521
Epoch 667/1000
270/270 [==============================] - 0s 79us/step - loss: 0.2554 - accuracy: 0.8704 - val_loss: 1.4249 - val_accuracy: 0.7521
Epoch 668/1000
270/270 [==============================] - 0s 100us/step - loss: 0.2555 - accuracy: 0.8704 - val_loss: 1.4238 - val_accuracy: 0.7607
Epoch 669/1000
270/270 [==============================] - 0s 82us/step - loss: 0.2550 - accuracy: 0.8667 - val_loss: 1.4182 - val_accuracy: 0.7607
Epoch 670/1000
270/270 [==============================] - 0s 153us/step - loss: 0.2538 - accuracy: 0.8704 - val_loss: 1.4200 - val_accuracy: 0.7778
Epoch 671/1000
270/270 [==============================] - 0s 74us/step - loss: 0.2591 - accuracy: 0.8630 - val_loss: 1.4253 - val_accuracy: 0.7778
Epoch 672/1000
270/270 [==============================] - 0s 80us/step - loss: 0.2571 - accuracy: 0.8556 - val_loss: 1.4340 - val_a

270/270 [==============================] - 0s 80us/step - loss: 0.2549 - accuracy: 0.8704 - val_loss: 1.4336 - val_accuracy: 0.7607
Epoch 777/1000
270/270 [==============================] - 0s 90us/step - loss: 0.2546 - accuracy: 0.8704 - val_loss: 1.4295 - val_accuracy: 0.7607
Epoch 778/1000
270/270 [==============================] - 0s 87us/step - loss: 0.2547 - accuracy: 0.8556 - val_loss: 1.4309 - val_accuracy: 0.7607
Epoch 779/1000
270/270 [==============================] - 0s 83us/step - loss: 0.2535 - accuracy: 0.8704 - val_loss: 1.4423 - val_accuracy: 0.7607
Epoch 780/1000
270/270 [==============================] - 0s 64us/step - loss: 0.2589 - accuracy: 0.8704 - val_loss: 1.4569 - val_accuracy: 0.7607
Epoch 781/1000
270/270 [==============================] - 0s 122us/step - loss: 0.2569 - accuracy: 0.8704 - val_loss: 1.4416 - val_accuracy: 0.7521
Epoch 782/1000
270/270 [==============================] - 0s 189us/step - loss: 0.2558 - accuracy: 0.8704 - val_loss: 1.4402 - val_a

Epoch 832/1000
270/270 [==============================] - 0s 66us/step - loss: 0.2596 - accuracy: 0.8704 - val_loss: 1.4656 - val_accuracy: 0.7607
Epoch 833/1000
270/270 [==============================] - 0s 71us/step - loss: 0.2565 - accuracy: 0.8741 - val_loss: 1.4611 - val_accuracy: 0.7607
Epoch 834/1000
270/270 [==============================] - 0s 69us/step - loss: 0.2560 - accuracy: 0.8741 - val_loss: 1.4588 - val_accuracy: 0.7692
Epoch 835/1000
270/270 [==============================] - 0s 84us/step - loss: 0.2553 - accuracy: 0.8704 - val_loss: 1.4620 - val_accuracy: 0.7692
Epoch 836/1000
270/270 [==============================] - 0s 67us/step - loss: 0.2553 - accuracy: 0.8704 - val_loss: 1.4701 - val_accuracy: 0.7607
Epoch 837/1000
270/270 [==============================] - 0s 76us/step - loss: 0.2547 - accuracy: 0.8593 - val_loss: 1.4667 - val_accuracy: 0.7607
Epoch 838/1000
270/270 [==============================] - 0s 54us/step - loss: 0.2528 - accuracy: 0.8704 - val_loss: 1

Epoch 943/1000
270/270 [==============================] - 0s 81us/step - loss: 0.2593 - accuracy: 0.8630 - val_loss: 1.4909 - val_accuracy: 0.7863
Epoch 944/1000
270/270 [==============================] - 0s 110us/step - loss: 0.2576 - accuracy: 0.8667 - val_loss: 1.5030 - val_accuracy: 0.7692
Epoch 945/1000
270/270 [==============================] - 0s 92us/step - loss: 0.2553 - accuracy: 0.8704 - val_loss: 1.5172 - val_accuracy: 0.7692
Epoch 946/1000
270/270 [==============================] - 0s 77us/step - loss: 0.2553 - accuracy: 0.8630 - val_loss: 1.5241 - val_accuracy: 0.7692
Epoch 947/1000
270/270 [==============================] - 0s 104us/step - loss: 0.2552 - accuracy: 0.8704 - val_loss: 1.5146 - val_accuracy: 0.7692
Epoch 948/1000
270/270 [==============================] - 0s 74us/step - loss: 0.2540 - accuracy: 0.8667 - val_loss: 1.5092 - val_accuracy: 0.7863
Epoch 949/1000
270/270 [==============================] - 0s 73us/step - loss: 0.2544 - accuracy: 0.8556 - val_loss:

In [171]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist2_over.history['accuracy'])*100))

over-sampling train accuracy: 86.80%


In [34]:
df_proba5 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_lasso_2.xlsx",
                        sheet_name=0,
                        index_col=None)

In [35]:
df_proba5

,phage,strain,phenotype,prediction,0,1,2
0,p0006kpresabs_qual,NRS245,1,2,1.345807e-02,2.164788e-01,7.700630e-01
1,p0006kpresabs_qual,NY439,2,2,2.674153e-02,9.294230e-04,9.723290e-01
2,p0006kpresabs_qual,CA544,1,0,4.147484e-01,3.626331e-01,2.226184e-01
3,p0006kpresabs_qual,CA541,2,0,4.147484e-01,3.626331e-01,2.226184e-01
4,p0006kpresabs_qual,EUH15,1,0,4.147484e-01,3.626331e-01,2.226184e-01
...,...,...,...,...,...,...,...
984,p0017Skpresabs_qual,CA541,1,1,3.723218e-01,6.276781e-01,1.945911e-08
985,p0017Skpresabs_qual,SR4152,1,0,7.372800e-01,2.627200e-01,4.197748e-08
986,p0017Skpresabs_qual,NRS110,2,2,4.194510e-08,7.508231e-09,1.000000e+00
987,p0017Skpresabs_qual,CFBRSa70,0,0,7.372800e-01,2.627200e-01,4.197748e-08


In [36]:
y_prob5 = df_proba5[df_proba5['phage']=='p0006presabs_qual'].iloc[:,-3:]
y_prob5 = y_prob5.to_numpy()
y_prob5

array([[1.13621786e-01, 8.70445970e-01, 1.59322710e-02],
       [5.26185630e-02, 8.76052770e-04, 9.46505300e-01],
       [1.00919420e-01, 5.83062770e-01, 3.16017840e-01],
       [1.00919420e-01, 5.83062770e-01, 3.16017840e-01],
       [3.46191400e-01, 6.44950200e-01, 8.85843800e-03],
       [3.66983870e-03, 3.28549300e-03, 9.93044700e-01],
       [9.85246600e-02, 8.94542750e-01, 6.93248720e-03],
       [5.15567700e-01, 3.64493520e-01, 1.19938820e-01],
       [9.99999900e-01, 1.54699700e-07, 3.24824200e-11],
       [2.66017580e-02, 3.57331580e-06, 9.73394630e-01],
       [9.99998900e-01, 1.04783080e-06, 1.13333650e-09],
       [4.92268240e-06, 5.65362630e-03, 9.94341500e-01],
       [9.46227900e-05, 1.15727100e-03, 9.98748060e-01],
       [4.30122540e-02, 9.56769100e-01, 2.18604080e-04],
       [9.30907900e-05, 3.41646720e-03, 9.96490400e-01],
       [6.26257300e-03, 8.80636200e-03, 9.84931100e-01],
       [3.39146220e-02, 1.89631710e-03, 9.64189100e-01],
       [3.14550400e-04, 2.12784

In [37]:
ovo5 = rocauc_ovo(y_sel_test_over, y_prob5, average="macro", multi_class="ovo")
ovo5

0.8566732412886259

In [38]:
ovr5 = rocauc_ovr(y_sel_test_over, y_prob5, average="macro", multi_class="ovr")
ovr5

0.8566732412886259

In [39]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_sel_train_over, X_sel_test_over, y_sel_train_over, y_sel_test_over = train_test_split(X_sel_over, y_sel_over,
                                                    test_size = 0.3,
                                                    random_state=678,
                                                    stratify=y_sel_over)

In [40]:
dat6 = pd.DataFrame(X_sel_test_over[:,-1])
dat6['test'] = y_sel_test_over

In [41]:
dat6

,0,test
0,NRS249,2
1,NRS188,1
2,NRS232,2
3,NY439,2
4,GA27,2
...,...,...
112,SR3569,0
113,NRS204,0
114,NRS203,0
115,CFBRSa25,1


In [42]:
X_sel_train_over = X_sel_train_over[:,:-1]
X_sel_test_over = X_sel_test_over[:,:-1]

In [43]:
model2_over2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train_over.shape[1],)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
])

In [97]:
model2_over2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [98]:
model2_over2.fit(X_sel_train_over, y_sel_train_over,
          batch_size=64, epochs=1000,
          validation_data=(X_sel_test_over, y_sel_test_over))

Train on 270 samples, validate on 117 samples
Epoch 1/1000
270/270 [==============================] - 0s 491us/step - loss: 1.1870 - accuracy: 0.3333 - val_loss: 1.1303 - val_accuracy: 0.3333
Epoch 2/1000
270/270 [==============================] - 0s 67us/step - loss: 1.1037 - accuracy: 0.3519 - val_loss: 1.0828 - val_accuracy: 0.3248
Epoch 3/1000
270/270 [==============================] - 0s 115us/step - loss: 1.0495 - accuracy: 0.4407 - val_loss: 1.0529 - val_accuracy: 0.4274
Epoch 4/1000
270/270 [==============================] - 0s 79us/step - loss: 1.0117 - accuracy: 0.4778 - val_loss: 1.0308 - val_accuracy: 0.4444
Epoch 5/1000
270/270 [==============================] - 0s 85us/step - loss: 0.9818 - accuracy: 0.5370 - val_loss: 1.0150 - val_accuracy: 0.4701
Epoch 6/1000
270/270 [==============================] - 0s 142us/step - loss: 0.9560 - accuracy: 0.5741 - val_loss: 1.0006 - val_accuracy: 0.5299
Epoch 7/1000
270/270 [==============================] - 0s 106us/step - loss: 0.9

Epoch 112/1000
270/270 [==============================] - 0s 99us/step - loss: 0.3621 - accuracy: 0.8519 - val_loss: 0.8012 - val_accuracy: 0.6667
Epoch 113/1000
270/270 [==============================] - 0s 138us/step - loss: 0.3661 - accuracy: 0.8556 - val_loss: 0.8349 - val_accuracy: 0.6667
Epoch 114/1000
270/270 [==============================] - 0s 119us/step - loss: 0.3684 - accuracy: 0.8630 - val_loss: 0.8045 - val_accuracy: 0.6667
Epoch 115/1000
270/270 [==============================] - 0s 150us/step - loss: 0.3544 - accuracy: 0.8593 - val_loss: 0.7862 - val_accuracy: 0.6496
Epoch 116/1000
270/270 [==============================] - 0s 111us/step - loss: 0.3568 - accuracy: 0.8556 - val_loss: 0.7935 - val_accuracy: 0.6496
Epoch 117/1000
270/270 [==============================] - 0s 160us/step - loss: 0.3528 - accuracy: 0.8593 - val_loss: 0.8007 - val_accuracy: 0.6667
Epoch 118/1000
270/270 [==============================] - 0s 443us/step - loss: 0.3505 - accuracy: 0.8593 - val_l

Epoch 223/1000
270/270 [==============================] - 0s 62us/step - loss: 0.2723 - accuracy: 0.8778 - val_loss: 0.9922 - val_accuracy: 0.6496
Epoch 224/1000
270/270 [==============================] - 0s 63us/step - loss: 0.2748 - accuracy: 0.8593 - val_loss: 0.9765 - val_accuracy: 0.6667
Epoch 225/1000
270/270 [==============================] - 0s 58us/step - loss: 0.2734 - accuracy: 0.8778 - val_loss: 1.0033 - val_accuracy: 0.6496
Epoch 226/1000
270/270 [==============================] - 0s 80us/step - loss: 0.2724 - accuracy: 0.8815 - val_loss: 1.0305 - val_accuracy: 0.6496
Epoch 227/1000
270/270 [==============================] - 0s 52us/step - loss: 0.2740 - accuracy: 0.8778 - val_loss: 1.0093 - val_accuracy: 0.6496
Epoch 228/1000
270/270 [==============================] - 0s 57us/step - loss: 0.2734 - accuracy: 0.8778 - val_loss: 0.9898 - val_accuracy: 0.6496
Epoch 229/1000
270/270 [==============================] - 0s 50us/step - loss: 0.2732 - accuracy: 0.8815 - val_loss: 0

Epoch 279/1000
270/270 [==============================] - 0s 57us/step - loss: 0.2641 - accuracy: 0.8704 - val_loss: 1.0599 - val_accuracy: 0.6667
Epoch 280/1000
270/270 [==============================] - 0s 83us/step - loss: 0.2593 - accuracy: 0.8704 - val_loss: 1.0614 - val_accuracy: 0.6581
Epoch 281/1000
270/270 [==============================] - 0s 68us/step - loss: 0.2619 - accuracy: 0.8741 - val_loss: 1.0729 - val_accuracy: 0.6581
Epoch 282/1000
270/270 [==============================] - 0s 64us/step - loss: 0.2628 - accuracy: 0.8778 - val_loss: 1.0935 - val_accuracy: 0.6581
Epoch 283/1000
270/270 [==============================] - 0s 145us/step - loss: 0.2607 - accuracy: 0.8889 - val_loss: 1.1381 - val_accuracy: 0.6496
Epoch 284/1000
270/270 [==============================] - 0s 104us/step - loss: 0.2682 - accuracy: 0.8815 - val_loss: 1.1551 - val_accuracy: 0.6581
Epoch 285/1000
270/270 [==============================] - 0s 150us/step - loss: 0.2704 - accuracy: 0.8704 - val_loss

270/270 [==============================] - 0s 73us/step - loss: 0.2583 - accuracy: 0.8741 - val_loss: 1.2103 - val_accuracy: 0.6581
Epoch 390/1000
270/270 [==============================] - 0s 82us/step - loss: 0.2569 - accuracy: 0.8852 - val_loss: 1.2475 - val_accuracy: 0.6581
Epoch 391/1000
270/270 [==============================] - 0s 95us/step - loss: 0.2560 - accuracy: 0.8815 - val_loss: 1.2450 - val_accuracy: 0.6581
Epoch 392/1000
270/270 [==============================] - 0s 68us/step - loss: 0.2532 - accuracy: 0.8778 - val_loss: 1.2071 - val_accuracy: 0.6667
Epoch 393/1000
270/270 [==============================] - 0s 80us/step - loss: 0.2547 - accuracy: 0.8667 - val_loss: 1.2227 - val_accuracy: 0.6667
Epoch 394/1000
270/270 [==============================] - 0s 93us/step - loss: 0.2509 - accuracy: 0.8815 - val_loss: 1.2259 - val_accuracy: 0.6581
Epoch 395/1000
270/270 [==============================] - 0s 72us/step - loss: 0.2501 - accuracy: 0.8815 - val_loss: 1.2174 - val_acc

...

Epoch 667/1000
270/270 [==============================] - 0s 59us/step - loss: 0.2482 - accuracy: 0.8815 - val_loss: 1.3237 - val_accuracy: 0.6667
Epoch 668/1000
270/270 [==============================] - 0s 63us/step - loss: 0.2456 - accuracy: 0.8815 - val_loss: 1.3074 - val_accuracy: 0.6667
Epoch 669/1000
270/270 [==============================] - 0s 96us/step - loss: 0.2465 - accuracy: 0.8630 - val_loss: 1.3101 - val_accuracy: 0.6752
Epoch 670/1000
270/270 [==============================] - 0s 74us/step - loss: 0.2433 - accuracy: 0.8778 - val_loss: 1.3393 - val_accuracy: 0.6752
Epoch 671/1000
270/270 [==============================] - 0s 73us/step - loss: 0.2447 - accuracy: 0.8778 - val_loss: 1.3493 - val_accuracy: 0.6752
Epoch 672/1000
270/270 [==============================] - 0s 73us/step - loss: 0.2460 - accuracy: 0.8704 - val_loss: 1.3597 - val_accuracy: 0.6838
Epoch 673/1000
270/270 [==============================] - 0s 72us/step - loss: 0.2429 - accuracy: 0.8704 - val_loss: 1

Epoch 723/1000
270/270 [==============================] - 0s 112us/step - loss: 0.2440 - accuracy: 0.8815 - val_loss: 1.4160 - val_accuracy: 0.6667
Epoch 724/1000
270/270 [==============================] - 0s 64us/step - loss: 0.2453 - accuracy: 0.8815 - val_loss: 1.3846 - val_accuracy: 0.6667
Epoch 725/1000
270/270 [==============================] - 0s 102us/step - loss: 0.2444 - accuracy: 0.8741 - val_loss: 1.3250 - val_accuracy: 0.6667
Epoch 726/1000
270/270 [==============================] - 0s 101us/step - loss: 0.2450 - accuracy: 0.8778 - val_loss: 1.3532 - val_accuracy: 0.6496
Epoch 727/1000
270/270 [==============================] - 0s 93us/step - loss: 0.2450 - accuracy: 0.8778 - val_loss: 1.4083 - val_accuracy: 0.6581
Epoch 728/1000
270/270 [==============================] - 0s 104us/step - loss: 0.2452 - accuracy: 0.8852 - val_loss: 1.3934 - val_accuracy: 0.6581
Epoch 729/1000
270/270 [==============================] - 0s 103us/step - loss: 0.2457 - accuracy: 0.8815 - val_lo

Epoch 779/1000
270/270 [==============================] - 0s 67us/step - loss: 0.2530 - accuracy: 0.8741 - val_loss: 1.3355 - val_accuracy: 0.6838
Epoch 780/1000
270/270 [==============================] - 0s 68us/step - loss: 0.2563 - accuracy: 0.8741 - val_loss: 1.3634 - val_accuracy: 0.6838
Epoch 781/1000
270/270 [==============================] - 0s 66us/step - loss: 0.2469 - accuracy: 0.8630 - val_loss: 1.4036 - val_accuracy: 0.6667
Epoch 782/1000
270/270 [==============================] - 0s 76us/step - loss: 0.2506 - accuracy: 0.8815 - val_loss: 1.4163 - val_accuracy: 0.6581
Epoch 783/1000
270/270 [==============================] - 0s 67us/step - loss: 0.2498 - accuracy: 0.8815 - val_loss: 1.3832 - val_accuracy: 0.6667
Epoch 784/1000
270/270 [==============================] - 0s 71us/step - loss: 0.2403 - accuracy: 0.8889 - val_loss: 1.3366 - val_accuracy: 0.6752
Epoch 785/1000
270/270 [==============================] - 0s 77us/step - loss: 0.2519 - accuracy: 0.8704 - val_loss: 1

270/270 [==============================] - 0s 78us/step - loss: 0.2406 - accuracy: 0.8778 - val_loss: 1.4418 - val_accuracy: 0.6581
Epoch 890/1000
270/270 [==============================] - 0s 62us/step - loss: 0.2485 - accuracy: 0.8778 - val_loss: 1.4625 - val_accuracy: 0.6667
Epoch 891/1000
270/270 [==============================] - 0s 69us/step - loss: 0.2480 - accuracy: 0.8741 - val_loss: 1.4313 - val_accuracy: 0.6667
Epoch 892/1000
270/270 [==============================] - 0s 54us/step - loss: 0.2420 - accuracy: 0.8815 - val_loss: 1.3800 - val_accuracy: 0.6667
Epoch 893/1000
270/270 [==============================] - 0s 60us/step - loss: 0.2475 - accuracy: 0.8741 - val_loss: 1.3859 - val_accuracy: 0.6325
Epoch 894/1000
270/270 [==============================] - 0s 55us/step - loss: 0.2506 - accuracy: 0.8667 - val_loss: 1.4404 - val_accuracy: 0.6667
Epoch 895/1000
270/270 [==============================] - 0s 65us/step - loss: 0.2420 - accuracy: 0.8778 - val_loss: 1.4838 - val_acc

Epoch 945/1000
270/270 [==============================] - 0s 78us/step - loss: 0.2432 - accuracy: 0.8741 - val_loss: 1.3904 - val_accuracy: 0.6752
Epoch 946/1000
270/270 [==============================] - 0s 71us/step - loss: 0.2443 - accuracy: 0.8741 - val_loss: 1.3904 - val_accuracy: 0.6667
Epoch 947/1000
270/270 [==============================] - 0s 70us/step - loss: 0.2466 - accuracy: 0.8815 - val_loss: 1.4409 - val_accuracy: 0.6581
Epoch 948/1000
270/270 [==============================] - 0s 71us/step - loss: 0.2418 - accuracy: 0.8741 - val_loss: 1.4780 - val_accuracy: 0.6667
Epoch 949/1000
270/270 [==============================] - 0s 88us/step - loss: 0.2462 - accuracy: 0.8741 - val_loss: 1.4665 - val_accuracy: 0.6667
Epoch 950/1000
270/270 [==============================] - 0s 77us/step - loss: 0.2460 - accuracy: 0.8815 - val_loss: 1.4370 - val_accuracy: 0.6667
Epoch 951/1000
270/270 [==============================] - 0s 65us/step - loss: 0.2437 - accuracy: 0.8778 - val_loss: 1

In [187]:
acc_test2_over2 = model2_over2.evaluate(X_sel_test_over, y_sel_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test2_over2*100))

117/117 [==============================] - 0s 96us/step
over-sampling test accuracy: 72.65%


In [99]:
pred6 = model2_over2.predict_classes(X_sel_test_over)
pred6

array([2, 1, 2, 2, 2, 2, 2, 0, 1, 2, 0, 1, 2, 2, 1, 2, 1, 1, 0, 0, 0, 1,
       1, 0, 2, 0, 1, 0, 0, 1, 0, 0, 1, 1, 2, 0, 1, 2, 1, 1, 2, 2, 0, 0,
       0, 2, 1, 1, 2, 2, 0, 1, 2, 2, 0, 1, 1, 2, 1, 1, 1, 2, 1, 2, 1, 0,
       0, 1, 0, 1, 0, 2, 1, 2, 2, 2, 2, 0, 0, 1, 2, 2, 1, 1, 1, 0, 1, 1,
       0, 1, 0, 1, 1, 1, 0, 1, 2, 0, 1, 1, 2, 2, 0, 1, 2, 0, 1, 0, 1, 2,
       0, 0, 0, 0, 2, 1, 2])

In [100]:
dat6['pred'] = pred6
dat6

,0,test,pred
0,NRS249,2,2
1,NRS188,1,1
2,NRS232,2,2
3,NY439,2,2
4,GA27,2,2
...,...,...,...
112,SR3569,0,0
113,NRS204,0,0
114,NRS203,0,2
115,CFBRSa25,1,1


In [101]:
proba6 = model2_over2.predict_proba(X_sel_test_over)
dat_proba6 = pd.DataFrame(proba6)

In [102]:
dat_proba6

,0,1,2
0,1.482748e-06,3.357116e-03,0.996641
1,3.502702e-04,8.620075e-01,0.137642
2,6.214145e-08,1.826003e-04,0.999817
3,2.244555e-03,3.042703e-05,0.997725
4,2.452473e-08,5.293663e-03,0.994706
...,...,...,...
112,9.999979e-01,3.866272e-08,0.000002
113,8.595100e-01,1.400107e-01,0.000479
114,8.794755e-02,4.880720e-04,0.911564
115,2.951578e-01,7.014401e-01,0.003402


In [103]:
dat_proba6.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba6.csv", index = False,
         header=None)

In [104]:
dat6.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/6p006p.csv", index = False,
         header=None)

In [191]:
hist2_over2 = model2_over2.fit(X_sel_train_over, y_sel_train_over,
          batch_size=64, epochs=1000,
          validation_data=(X_sel_test_over, y_sel_test_over))

Train on 270 samples, validate on 117 samples
Epoch 1/1000
270/270 [==============================] - 0s 83us/step - loss: 0.2518 - accuracy: 0.8778 - val_loss: 1.3424 - val_accuracy: 0.7094
Epoch 2/1000
270/270 [==============================] - 0s 63us/step - loss: 0.2514 - accuracy: 0.8815 - val_loss: 1.3570 - val_accuracy: 0.7094
Epoch 3/1000
270/270 [==============================] - 0s 80us/step - loss: 0.2497 - accuracy: 0.8815 - val_loss: 1.3155 - val_accuracy: 0.7094
Epoch 4/1000
270/270 [==============================] - 0s 97us/step - loss: 0.2432 - accuracy: 0.8815 - val_loss: 1.2681 - val_accuracy: 0.7094
Epoch 5/1000
270/270 [==============================] - 0s 102us/step - loss: 0.2410 - accuracy: 0.8815 - val_loss: 1.2539 - val_accuracy: 0.7094
Epoch 6/1000
270/270 [==============================] - 0s 72us/step - loss: 0.2453 - accuracy: 0.8704 - val_loss: 1.2531 - val_accuracy: 0.7094
Epoch 7/1000
270/270 [==============================] - 0s 90us/step - loss: 0.2421

Epoch 57/1000
270/270 [==============================] - 0s 86us/step - loss: 0.2438 - accuracy: 0.8815 - val_loss: 1.3314 - val_accuracy: 0.7179
Epoch 58/1000
270/270 [==============================] - 0s 86us/step - loss: 0.2448 - accuracy: 0.8815 - val_loss: 1.3339 - val_accuracy: 0.7179
Epoch 59/1000
270/270 [==============================] - 0s 68us/step - loss: 0.2457 - accuracy: 0.8815 - val_loss: 1.3053 - val_accuracy: 0.7265
Epoch 60/1000
270/270 [==============================] - 0s 57us/step - loss: 0.2471 - accuracy: 0.8704 - val_loss: 1.2688 - val_accuracy: 0.7094
Epoch 61/1000
270/270 [==============================] - 0s 68us/step - loss: 0.2444 - accuracy: 0.8778 - val_loss: 1.2723 - val_accuracy: 0.7179
Epoch 62/1000
270/270 [==============================] - 0s 103us/step - loss: 0.2424 - accuracy: 0.8852 - val_loss: 1.2804 - val_accuracy: 0.7094
Epoch 63/1000
270/270 [==============================] - 0s 86us/step - loss: 0.2423 - accuracy: 0.8815 - val_loss: 1.2786 

Epoch 113/1000
270/270 [==============================] - 0s 77us/step - loss: 0.2407 - accuracy: 0.8778 - val_loss: 1.2981 - val_accuracy: 0.7179
Epoch 114/1000
270/270 [==============================] - 0s 82us/step - loss: 0.2419 - accuracy: 0.8778 - val_loss: 1.3106 - val_accuracy: 0.7094
Epoch 115/1000
270/270 [==============================] - 0s 86us/step - loss: 0.2439 - accuracy: 0.8704 - val_loss: 1.3146 - val_accuracy: 0.7179
Epoch 116/1000
270/270 [==============================] - 0s 62us/step - loss: 0.2523 - accuracy: 0.8778 - val_loss: 1.3018 - val_accuracy: 0.7265
Epoch 117/1000
270/270 [==============================] - 0s 75us/step - loss: 0.2492 - accuracy: 0.8704 - val_loss: 1.3238 - val_accuracy: 0.7179
Epoch 118/1000
270/270 [==============================] - 0s 60us/step - loss: 0.2412 - accuracy: 0.8815 - val_loss: 1.3570 - val_accuracy: 0.7179
Epoch 119/1000
270/270 [==============================] - 0s 65us/step - loss: 0.2460 - accuracy: 0.8815 - val_loss: 1

270/270 [==============================] - 0s 74us/step - loss: 0.2425 - accuracy: 0.8741 - val_loss: 1.3900 - val_accuracy: 0.7179
Epoch 224/1000
270/270 [==============================] - 0s 69us/step - loss: 0.2432 - accuracy: 0.8815 - val_loss: 1.4356 - val_accuracy: 0.7179
Epoch 225/1000
270/270 [==============================] - 0s 66us/step - loss: 0.2422 - accuracy: 0.8815 - val_loss: 1.4268 - val_accuracy: 0.7094
Epoch 226/1000
270/270 [==============================] - 0s 155us/step - loss: 0.2478 - accuracy: 0.8778 - val_loss: 1.3966 - val_accuracy: 0.7094
Epoch 227/1000
270/270 [==============================] - 0s 432us/step - loss: 0.2415 - accuracy: 0.8741 - val_loss: 1.3891 - val_accuracy: 0.7179
Epoch 228/1000
270/270 [==============================] - 0s 168us/step - loss: 0.2431 - accuracy: 0.8778 - val_loss: 1.3865 - val_accuracy: 0.7179
Epoch 229/1000
270/270 [==============================] - 0s 252us/step - loss: 0.2451 - accuracy: 0.8778 - val_loss: 1.4050 - val

Epoch 334/1000
270/270 [==============================] - 0s 66us/step - loss: 0.2445 - accuracy: 0.8852 - val_loss: 1.4156 - val_accuracy: 0.7179
Epoch 335/1000
270/270 [==============================] - 0s 71us/step - loss: 0.2468 - accuracy: 0.8815 - val_loss: 1.4405 - val_accuracy: 0.7094
Epoch 336/1000
270/270 [==============================] - 0s 93us/step - loss: 0.2469 - accuracy: 0.8741 - val_loss: 1.4365 - val_accuracy: 0.7179
Epoch 337/1000
270/270 [==============================] - 0s 67us/step - loss: 0.2463 - accuracy: 0.8778 - val_loss: 1.4237 - val_accuracy: 0.7179
Epoch 338/1000
270/270 [==============================] - 0s 69us/step - loss: 0.2426 - accuracy: 0.8741 - val_loss: 1.3908 - val_accuracy: 0.7265
Epoch 339/1000
270/270 [==============================] - 0s 67us/step - loss: 0.2468 - accuracy: 0.8778 - val_loss: 1.4081 - val_accuracy: 0.7265
Epoch 340/1000
270/270 [==============================] - 0s 71us/step - loss: 0.2498 - accuracy: 0.8667 - val_loss: 1

...

Epoch 611/1000
270/270 [==============================] - 0s 72us/step - loss: 0.2416 - accuracy: 0.8815 - val_loss: 1.5288 - val_accuracy: 0.7094
Epoch 612/1000
270/270 [==============================] - 0s 73us/step - loss: 0.2406 - accuracy: 0.8741 - val_loss: 1.5007 - val_accuracy: 0.7179
Epoch 613/1000
270/270 [==============================] - 0s 88us/step - loss: 0.2428 - accuracy: 0.8741 - val_loss: 1.5408 - val_accuracy: 0.7179
Epoch 614/1000
270/270 [==============================] - 0s 115us/step - loss: 0.2474 - accuracy: 0.8778 - val_loss: 1.5551 - val_accuracy: 0.7179
Epoch 615/1000
270/270 [==============================] - 0s 69us/step - loss: 0.2417 - accuracy: 0.8778 - val_loss: 1.5467 - val_accuracy: 0.7094
Epoch 616/1000
270/270 [==============================] - 0s 191us/step - loss: 0.2394 - accuracy: 0.8778 - val_loss: 1.5425 - val_accuracy: 0.7179
Epoch 617/1000
270/270 [==============================] - 0s 160us/step - loss: 0.2469 - accuracy: 0.8778 - val_loss

Epoch 722/1000
270/270 [==============================] - 0s 60us/step - loss: 0.2416 - accuracy: 0.8778 - val_loss: 1.5806 - val_accuracy: 0.7094
Epoch 723/1000
270/270 [==============================] - 0s 95us/step - loss: 0.2409 - accuracy: 0.8667 - val_loss: 1.5629 - val_accuracy: 0.7265
Epoch 724/1000
270/270 [==============================] - 0s 98us/step - loss: 0.2424 - accuracy: 0.8778 - val_loss: 1.5733 - val_accuracy: 0.7265
Epoch 725/1000
270/270 [==============================] - 0s 102us/step - loss: 0.2434 - accuracy: 0.8778 - val_loss: 1.5661 - val_accuracy: 0.7265
Epoch 726/1000
270/270 [==============================] - 0s 86us/step - loss: 0.2393 - accuracy: 0.8778 - val_loss: 1.5827 - val_accuracy: 0.7094
Epoch 727/1000
270/270 [==============================] - 0s 239us/step - loss: 0.2463 - accuracy: 0.8741 - val_loss: 1.5904 - val_accuracy: 0.7179
Epoch 728/1000
270/270 [==============================] - 0s 95us/step - loss: 0.2410 - accuracy: 0.8704 - val_loss:

Epoch 778/1000
270/270 [==============================] - 0s 208us/step - loss: 0.2455 - accuracy: 0.8778 - val_loss: 1.6465 - val_accuracy: 0.7179
Epoch 779/1000
270/270 [==============================] - 0s 274us/step - loss: 0.2466 - accuracy: 0.8667 - val_loss: 1.6155 - val_accuracy: 0.7265
Epoch 780/1000
270/270 [==============================] - 0s 169us/step - loss: 0.2449 - accuracy: 0.8778 - val_loss: 1.6246 - val_accuracy: 0.7265
Epoch 781/1000
270/270 [==============================] - 0s 287us/step - loss: 0.2422 - accuracy: 0.8778 - val_loss: 1.6307 - val_accuracy: 0.7179
Epoch 782/1000
270/270 [==============================] - 0s 139us/step - loss: 0.2411 - accuracy: 0.8815 - val_loss: 1.6209 - val_accuracy: 0.7179
Epoch 783/1000
270/270 [==============================] - 0s 86us/step - loss: 0.2396 - accuracy: 0.8815 - val_loss: 1.5852 - val_accuracy: 0.7179
Epoch 784/1000
270/270 [==============================] - 0s 76us/step - loss: 0.2411 - accuracy: 0.8815 - val_lo

270/270 [==============================] - 0s 75us/step - loss: 0.2422 - accuracy: 0.8778 - val_loss: 1.5866 - val_accuracy: 0.7265
Epoch 889/1000
270/270 [==============================] - 0s 77us/step - loss: 0.2403 - accuracy: 0.8741 - val_loss: 1.5967 - val_accuracy: 0.7265
Epoch 890/1000
270/270 [==============================] - 0s 92us/step - loss: 0.2402 - accuracy: 0.8741 - val_loss: 1.6212 - val_accuracy: 0.7265
Epoch 891/1000
270/270 [==============================] - 0s 96us/step - loss: 0.2417 - accuracy: 0.8815 - val_loss: 1.6592 - val_accuracy: 0.7009
Epoch 892/1000
270/270 [==============================] - 0s 77us/step - loss: 0.2422 - accuracy: 0.8778 - val_loss: 1.6548 - val_accuracy: 0.7094
Epoch 893/1000
270/270 [==============================] - 0s 69us/step - loss: 0.2436 - accuracy: 0.8778 - val_loss: 1.6465 - val_accuracy: 0.7094
Epoch 894/1000
270/270 [==============================] - 0s 96us/step - loss: 0.2414 - accuracy: 0.8815 - val_loss: 1.6504 - val_acc

Epoch 944/1000
270/270 [==============================] - 0s 145us/step - loss: 0.2447 - accuracy: 0.8778 - val_loss: 1.6856 - val_accuracy: 0.7094
Epoch 945/1000
270/270 [==============================] - 0s 105us/step - loss: 0.2459 - accuracy: 0.8815 - val_loss: 1.6492 - val_accuracy: 0.7179
Epoch 946/1000
270/270 [==============================] - 0s 92us/step - loss: 0.2437 - accuracy: 0.8481 - val_loss: 1.6234 - val_accuracy: 0.7179
Epoch 947/1000
270/270 [==============================] - 0s 110us/step - loss: 0.2428 - accuracy: 0.8741 - val_loss: 1.6181 - val_accuracy: 0.7179
Epoch 948/1000
270/270 [==============================] - 0s 166us/step - loss: 0.2440 - accuracy: 0.8852 - val_loss: 1.6590 - val_accuracy: 0.7179
Epoch 949/1000
270/270 [==============================] - 0s 251us/step - loss: 0.2414 - accuracy: 0.8815 - val_loss: 1.7017 - val_accuracy: 0.7179
Epoch 950/1000
270/270 [==============================] - 0s 83us/step - loss: 0.2416 - accuracy: 0.8815 - val_lo

In [192]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist2_over2.history['accuracy'])*100))

over-sampling train accuracy: 87.74%


In [44]:
df_proba6 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_lasso_2.xlsx",
                        sheet_name=1,
                        index_col=None)

In [45]:
df_proba6

,phage,strain,phenotype,prediction,0,1,2
0,p0006kpresabs_qual,NRS249,2,1,1.888869e-01,5.108038e-01,3.003094e-01
1,p0006kpresabs_qual,NRS188,1,1,1.888869e-01,5.108038e-01,3.003094e-01
2,p0006kpresabs_qual,NRS232,2,2,4.222906e-01,7.029924e-02,5.074101e-01
3,p0006kpresabs_qual,NY439,2,2,3.558408e-04,2.976018e-04,9.993465e-01
4,p0006kpresabs_qual,GA27,2,1,3.940971e-01,4.184215e-01,1.874814e-01
...,...,...,...,...,...,...,...
984,p0017Skpresabs_qual,NRS252,0,0,7.239556e-01,2.760444e-01,1.176030e-09
985,p0017Skpresabs_qual,SR2852,1,1,1.052276e-07,9.999999e-01,1.101559e-28
986,p0017Skpresabs_qual,NRS108,1,1,1.540350e-17,1.000000e+00,9.011977e-16
987,p0017Skpresabs_qual,NRS202,0,0,6.888959e-01,3.111042e-01,2.228958e-09


In [46]:
y_prob6 = df_proba6[df_proba6['phage']=='p0006presabs_qual'].iloc[:,-3:]
y_prob6 = y_prob6.to_numpy()
y_prob6

array([[1.48274810e-06, 3.35711570e-03, 9.96641400e-01],
       [3.50270250e-04, 8.62007500e-01, 1.37642200e-01],
       [6.21414500e-08, 1.82600270e-04, 9.99817300e-01],
       [2.24455470e-03, 3.04270340e-05, 9.97725070e-01],
       [2.45247290e-08, 5.29366300e-03, 9.94706330e-01],
       [1.48274810e-06, 3.35711570e-03, 9.96641400e-01],
       [5.33735550e-04, 1.25956920e-02, 9.86870600e-01],
       [9.47450340e-01, 1.88820180e-02, 3.36676720e-02],
       [1.33940570e-03, 9.98509800e-01, 1.50717420e-04],
       [2.24455470e-03, 3.04270340e-05, 9.97725070e-01],
       [9.10449450e-01, 7.02621340e-02, 1.92884100e-02],
       [4.41411400e-03, 9.92614750e-01, 2.97114670e-03],
       [5.93028730e-03, 1.49903550e-01, 8.44166200e-01],
       [2.66369790e-02, 3.78765050e-01, 5.94598000e-01],
       [5.57191450e-05, 9.99899150e-01, 4.51526540e-05],
       [1.45759020e-09, 9.21292100e-09, 1.00000000e+00],
       [2.14298140e-02, 9.78318630e-01, 2.51612040e-04],
       [1.98544900e-03, 9.98014

In [47]:
ovo6 = rocauc_ovo(y_sel_test_over, y_prob6, average="macro", multi_class="ovo")
ovo6

0.8210607056760902

In [48]:
ovr6 = rocauc_ovr(y_sel_test_over, y_prob6, average="macro", multi_class="ovr")
ovr6

0.8210607056760902

In [49]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_sel_train_over, X_sel_test_over, y_sel_train_over, y_sel_test_over = train_test_split(X_sel_over, y_sel_over,
                                                    test_size = 0.3,
                                                    random_state=789,
                                                    stratify=y_sel_over)

In [50]:
dat7 = pd.DataFrame(X_sel_test_over[:,-1])
dat7['test'] = y_sel_test_over

In [51]:
dat7

,0,test
0,NRS210,0
1,NRS205,2
2,312,2
3,GA15,2
4,SR4035,0
...,...,...
112,NRS265,2
113,CFBREBSa108,1
114,NY224,1
115,NRS386,2


In [52]:
X_sel_train_over = X_sel_train_over[:,:-1]
X_sel_test_over = X_sel_test_over[:,:-1]

In [53]:
model2_over3 = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train_over.shape[1],)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
])

In [110]:
model2_over3.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [111]:
model2_over3.fit(X_sel_train_over, y_sel_train_over,
          batch_size=64, epochs=1000,
          validation_data=(X_sel_test_over, y_sel_test_over))

Train on 270 samples, validate on 117 samples
Epoch 1/1000
270/270 [==============================] - 0s 867us/step - loss: 1.1220 - accuracy: 0.2852 - val_loss: 1.0656 - val_accuracy: 0.4274
Epoch 2/1000
270/270 [==============================] - 0s 180us/step - loss: 1.0702 - accuracy: 0.4000 - val_loss: 1.0274 - val_accuracy: 0.4957
Epoch 3/1000
270/270 [==============================] - 0s 195us/step - loss: 1.0359 - accuracy: 0.4815 - val_loss: 0.9970 - val_accuracy: 0.5983
Epoch 4/1000
270/270 [==============================] - 0s 158us/step - loss: 1.0042 - accuracy: 0.5889 - val_loss: 0.9707 - val_accuracy: 0.6325
Epoch 5/1000
270/270 [==============================] - 0s 120us/step - loss: 0.9791 - accuracy: 0.6000 - val_loss: 0.9457 - val_accuracy: 0.6581
Epoch 6/1000
270/270 [==============================] - 0s 135us/step - loss: 0.9540 - accuracy: 0.6111 - val_loss: 0.9228 - val_accuracy: 0.6667
Epoch 7/1000
270/270 [==============================] - 0s 118us/step - loss: 

Epoch 57/1000
270/270 [==============================] - 0s 57us/step - loss: 0.5332 - accuracy: 0.7556 - val_loss: 0.6927 - val_accuracy: 0.7265
Epoch 58/1000
270/270 [==============================] - 0s 58us/step - loss: 0.5279 - accuracy: 0.7667 - val_loss: 0.6874 - val_accuracy: 0.7521
Epoch 59/1000
270/270 [==============================] - 0s 62us/step - loss: 0.5286 - accuracy: 0.7704 - val_loss: 0.6870 - val_accuracy: 0.7436
Epoch 60/1000
270/270 [==============================] - 0s 81us/step - loss: 0.5251 - accuracy: 0.7667 - val_loss: 0.6941 - val_accuracy: 0.7265
Epoch 61/1000
270/270 [==============================] - 0s 62us/step - loss: 0.5194 - accuracy: 0.7667 - val_loss: 0.6880 - val_accuracy: 0.7350
Epoch 62/1000
270/270 [==============================] - 0s 56us/step - loss: 0.5132 - accuracy: 0.7630 - val_loss: 0.6873 - val_accuracy: 0.7265
Epoch 63/1000
270/270 [==============================] - 0s 87us/step - loss: 0.5118 - accuracy: 0.7704 - val_loss: 0.6950 -

Epoch 113/1000
270/270 [==============================] - 0s 60us/step - loss: 0.4007 - accuracy: 0.8148 - val_loss: 0.6832 - val_accuracy: 0.7692
Epoch 114/1000
270/270 [==============================] - 0s 61us/step - loss: 0.3940 - accuracy: 0.8481 - val_loss: 0.6747 - val_accuracy: 0.7521
Epoch 115/1000
270/270 [==============================] - 0s 78us/step - loss: 0.3909 - accuracy: 0.8444 - val_loss: 0.6841 - val_accuracy: 0.7692
Epoch 116/1000
270/270 [==============================] - 0s 58us/step - loss: 0.3887 - accuracy: 0.8481 - val_loss: 0.6995 - val_accuracy: 0.7521
Epoch 117/1000
270/270 [==============================] - 0s 57us/step - loss: 0.3928 - accuracy: 0.8370 - val_loss: 0.7089 - val_accuracy: 0.7607
Epoch 118/1000
270/270 [==============================] - 0s 55us/step - loss: 0.3995 - accuracy: 0.8185 - val_loss: 0.7283 - val_accuracy: 0.7692
Epoch 119/1000
270/270 [==============================] - 0s 100us/step - loss: 0.3884 - accuracy: 0.8519 - val_loss: 

Epoch 224/1000
270/270 [==============================] - 0s 57us/step - loss: 0.3003 - accuracy: 0.8556 - val_loss: 0.7946 - val_accuracy: 0.7692
Epoch 225/1000
270/270 [==============================] - 0s 65us/step - loss: 0.2984 - accuracy: 0.8667 - val_loss: 0.7987 - val_accuracy: 0.7521
Epoch 226/1000
270/270 [==============================] - 0s 83us/step - loss: 0.2968 - accuracy: 0.8593 - val_loss: 0.7728 - val_accuracy: 0.7607
Epoch 227/1000
270/270 [==============================] - 0s 80us/step - loss: 0.2990 - accuracy: 0.8667 - val_loss: 0.7732 - val_accuracy: 0.7607
Epoch 228/1000
270/270 [==============================] - 0s 58us/step - loss: 0.2993 - accuracy: 0.8519 - val_loss: 0.7957 - val_accuracy: 0.7521
Epoch 229/1000
270/270 [==============================] - 0s 55us/step - loss: 0.2956 - accuracy: 0.8667 - val_loss: 0.8154 - val_accuracy: 0.7521
Epoch 230/1000
270/270 [==============================] - 0s 56us/step - loss: 0.2964 - accuracy: 0.8630 - val_loss: 0

Epoch 335/1000
270/270 [==============================] - 0s 108us/step - loss: 0.2794 - accuracy: 0.8556 - val_loss: 0.8411 - val_accuracy: 0.7607
Epoch 336/1000
270/270 [==============================] - 0s 92us/step - loss: 0.2727 - accuracy: 0.8630 - val_loss: 0.8359 - val_accuracy: 0.7778
Epoch 337/1000
270/270 [==============================] - 0s 98us/step - loss: 0.2808 - accuracy: 0.8667 - val_loss: 0.8487 - val_accuracy: 0.7692
Epoch 338/1000
270/270 [==============================] - 0s 101us/step - loss: 0.2775 - accuracy: 0.8481 - val_loss: 0.8854 - val_accuracy: 0.7521
Epoch 339/1000
270/270 [==============================] - 0s 139us/step - loss: 0.2876 - accuracy: 0.8556 - val_loss: 0.9019 - val_accuracy: 0.7521
Epoch 340/1000
270/270 [==============================] - 0s 81us/step - loss: 0.2790 - accuracy: 0.8593 - val_loss: 0.8794 - val_accuracy: 0.7607
Epoch 341/1000
270/270 [==============================] - 0s 81us/step - loss: 0.2858 - accuracy: 0.8593 - val_loss

...

Epoch 726/1000
270/270 [==============================] - 0s 65us/step - loss: 0.2620 - accuracy: 0.8778 - val_loss: 0.9445 - val_accuracy: 0.7778
Epoch 727/1000
270/270 [==============================] - 0s 67us/step - loss: 0.2679 - accuracy: 0.8630 - val_loss: 0.9301 - val_accuracy: 0.7607
Epoch 728/1000
270/270 [==============================] - 0s 71us/step - loss: 0.2676 - accuracy: 0.8519 - val_loss: 0.9420 - val_accuracy: 0.7607
Epoch 729/1000
270/270 [==============================] - 0s 62us/step - loss: 0.2667 - accuracy: 0.8630 - val_loss: 0.9763 - val_accuracy: 0.7607
Epoch 730/1000
270/270 [==============================] - 0s 64us/step - loss: 0.2700 - accuracy: 0.8667 - val_loss: 0.9835 - val_accuracy: 0.7607
Epoch 731/1000
270/270 [==============================] - 0s 72us/step - loss: 0.2638 - accuracy: 0.8667 - val_loss: 0.9722 - val_accuracy: 0.7692
Epoch 732/1000
270/270 [==============================] - 0s 70us/step - loss: 0.2677 - accuracy: 0.8667 - val_loss: 0

Epoch 782/1000
270/270 [==============================] - 0s 55us/step - loss: 0.2703 - accuracy: 0.8630 - val_loss: 0.9773 - val_accuracy: 0.7607
Epoch 783/1000
270/270 [==============================] - 0s 50us/step - loss: 0.2679 - accuracy: 0.8630 - val_loss: 0.9829 - val_accuracy: 0.7692
Epoch 784/1000
270/270 [==============================] - 0s 64us/step - loss: 0.2643 - accuracy: 0.8593 - val_loss: 0.9942 - val_accuracy: 0.7607
Epoch 785/1000
270/270 [==============================] - 0s 69us/step - loss: 0.2676 - accuracy: 0.8481 - val_loss: 0.9644 - val_accuracy: 0.7778
Epoch 786/1000
270/270 [==============================] - 0s 71us/step - loss: 0.2636 - accuracy: 0.8593 - val_loss: 0.9587 - val_accuracy: 0.7607
Epoch 787/1000
270/270 [==============================] - 0s 71us/step - loss: 0.2636 - accuracy: 0.8630 - val_loss: 0.9685 - val_accuracy: 0.7607
Epoch 788/1000
270/270 [==============================] - 0s 65us/step - loss: 0.2642 - accuracy: 0.8667 - val_loss: 0

Epoch 838/1000
270/270 [==============================] - 0s 57us/step - loss: 0.2640 - accuracy: 0.8556 - val_loss: 0.9853 - val_accuracy: 0.7778
Epoch 839/1000
270/270 [==============================] - 0s 49us/step - loss: 0.2632 - accuracy: 0.8556 - val_loss: 1.0018 - val_accuracy: 0.7778
Epoch 840/1000
270/270 [==============================] - 0s 46us/step - loss: 0.2638 - accuracy: 0.8667 - val_loss: 1.0209 - val_accuracy: 0.7692
Epoch 841/1000
270/270 [==============================] - 0s 51us/step - loss: 0.2630 - accuracy: 0.8667 - val_loss: 1.0093 - val_accuracy: 0.7692
Epoch 842/1000
270/270 [==============================] - 0s 57us/step - loss: 0.2658 - accuracy: 0.8667 - val_loss: 0.9942 - val_accuracy: 0.7692
Epoch 843/1000
270/270 [==============================] - 0s 56us/step - loss: 0.2646 - accuracy: 0.8593 - val_loss: 1.0094 - val_accuracy: 0.7607
Epoch 844/1000
270/270 [==============================] - 0s 66us/step - loss: 0.2663 - accuracy: 0.8630 - val_loss: 1

Epoch 894/1000
270/270 [==============================] - 0s 65us/step - loss: 0.2621 - accuracy: 0.8667 - val_loss: 0.9948 - val_accuracy: 0.7692
Epoch 895/1000
270/270 [==============================] - 0s 67us/step - loss: 0.2630 - accuracy: 0.8556 - val_loss: 1.0284 - val_accuracy: 0.7436
Epoch 896/1000
270/270 [==============================] - 0s 67us/step - loss: 0.2657 - accuracy: 0.8704 - val_loss: 0.9726 - val_accuracy: 0.7778
Epoch 897/1000
270/270 [==============================] - 0s 76us/step - loss: 0.2661 - accuracy: 0.8556 - val_loss: 0.9619 - val_accuracy: 0.7607
Epoch 898/1000
270/270 [==============================] - 0s 69us/step - loss: 0.2668 - accuracy: 0.8593 - val_loss: 0.9906 - val_accuracy: 0.7692
Epoch 899/1000
270/270 [==============================] - 0s 65us/step - loss: 0.2669 - accuracy: 0.8593 - val_loss: 1.0206 - val_accuracy: 0.7607
Epoch 900/1000
270/270 [==============================] - 0s 61us/step - loss: 0.2648 - accuracy: 0.8667 - val_loss: 1

Epoch 950/1000
270/270 [==============================] - 0s 59us/step - loss: 0.2671 - accuracy: 0.8593 - val_loss: 0.9995 - val_accuracy: 0.7692
Epoch 951/1000
270/270 [==============================] - 0s 68us/step - loss: 0.2624 - accuracy: 0.8630 - val_loss: 1.0177 - val_accuracy: 0.7607
Epoch 952/1000
270/270 [==============================] - 0s 64us/step - loss: 0.2627 - accuracy: 0.8630 - val_loss: 1.0240 - val_accuracy: 0.7692
Epoch 953/1000
270/270 [==============================] - 0s 70us/step - loss: 0.2688 - accuracy: 0.8630 - val_loss: 1.0236 - val_accuracy: 0.7607
Epoch 954/1000
270/270 [==============================] - 0s 72us/step - loss: 0.2621 - accuracy: 0.8630 - val_loss: 0.9892 - val_accuracy: 0.7607
Epoch 955/1000
270/270 [==============================] - 0s 65us/step - loss: 0.2617 - accuracy: 0.8667 - val_loss: 0.9747 - val_accuracy: 0.7778
Epoch 956/1000
270/270 [==============================] - 0s 68us/step - loss: 0.2651 - accuracy: 0.8630 - val_loss: 0

In [208]:
acc_test2_over3 = model2_over3.evaluate(X_sel_test_over, y_sel_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test2_over3*100))

117/117 [==============================] - 0s 121us/step
over-sampling test accuracy: 78.63%


In [112]:
pred7 = model2_over3.predict_classes(X_sel_test_over)
pred7

array([0, 2, 2, 2, 0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 2, 0, 2, 2, 0, 0,
       1, 2, 1, 1, 1, 1, 1, 2, 0, 0, 0, 0, 1, 1, 1, 0, 0, 2, 0, 0, 2, 2,
       1, 0, 1, 2, 0, 0, 0, 0, 0, 0, 1, 0, 2, 2, 0, 2, 2, 1, 1, 0, 2, 0,
       2, 0, 1, 2, 1, 0, 2, 1, 2, 1, 2, 1, 1, 2, 2, 0, 0, 0, 2, 0, 1, 0,
       2, 1, 1, 1, 2, 2, 1, 2, 1, 1, 1, 2, 0, 1, 1, 0, 1, 2, 2, 2, 2, 2,
       2, 2, 2, 0, 1, 2, 2])

In [113]:
dat7['pred'] = pred7
dat7

,0,test,pred
0,NRS210,0,0
1,NRS205,2,2
2,312,2,2
3,GA15,2,2
4,SR4035,0,0
...,...,...,...
112,NRS265,2,2
113,CFBREBSa108,1,0
114,NY224,1,1
115,NRS386,2,2


In [114]:
proba7 = model2_over3.predict_proba(X_sel_test_over)
dat_proba7 = pd.DataFrame(proba7)

In [115]:
dat_proba7

,0,1,2
0,7.117269e-01,2.882596e-01,1.355703e-05
1,8.645844e-13,3.129307e-12,1.000000e+00
2,4.420781e-07,2.026271e-08,9.999995e-01
3,1.050341e-03,8.900376e-04,9.980596e-01
4,9.988620e-01,1.137946e-03,1.812745e-09
...,...,...,...
112,4.759788e-06,3.114278e-06,9.999921e-01
113,5.931377e-01,3.554063e-01,5.145596e-02
114,3.256392e-03,7.567788e-01,2.399649e-01
115,3.174596e-09,1.028540e-02,9.897146e-01


In [116]:
dat_proba7.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba7.csv", index = False,
         header=None)

In [117]:
dat7.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/7p006p.csv", index = False,
         header=None)

In [212]:
hist2_over3 = model2_over3.fit(X_sel_train_over, y_sel_train_over,
          batch_size=64, epochs=1000,
          validation_data=(X_sel_test_over, y_sel_test_over))

Train on 270 samples, validate on 117 samples
Epoch 1/1000
270/270 [==============================] - 0s 76us/step - loss: 0.2690 - accuracy: 0.8259 - val_loss: 0.7458 - val_accuracy: 0.8034
Epoch 2/1000
270/270 [==============================] - 0s 77us/step - loss: 0.2625 - accuracy: 0.8593 - val_loss: 0.7340 - val_accuracy: 0.8034
Epoch 3/1000
270/270 [==============================] - 0s 76us/step - loss: 0.2650 - accuracy: 0.8630 - val_loss: 0.7400 - val_accuracy: 0.8034
Epoch 4/1000
270/270 [==============================] - 0s 89us/step - loss: 0.2633 - accuracy: 0.8630 - val_loss: 0.7479 - val_accuracy: 0.7949
Epoch 5/1000
270/270 [==============================] - 0s 97us/step - loss: 0.2656 - accuracy: 0.8667 - val_loss: 0.7557 - val_accuracy: 0.7949
Epoch 6/1000
270/270 [==============================] - 0s 88us/step - loss: 0.2644 - accuracy: 0.8667 - val_loss: 0.7467 - val_accuracy: 0.8120
Epoch 7/1000
270/270 [==============================] - 0s 92us/step - loss: 0.2660 

Epoch 57/1000
270/270 [==============================] - 0s 98us/step - loss: 0.2638 - accuracy: 0.8667 - val_loss: 0.7530 - val_accuracy: 0.7949
Epoch 58/1000
270/270 [==============================] - 0s 83us/step - loss: 0.2648 - accuracy: 0.8593 - val_loss: 0.7629 - val_accuracy: 0.7949
Epoch 59/1000
270/270 [==============================] - 0s 77us/step - loss: 0.2606 - accuracy: 0.8741 - val_loss: 0.7575 - val_accuracy: 0.7949
Epoch 60/1000
270/270 [==============================] - 0s 58us/step - loss: 0.2655 - accuracy: 0.8630 - val_loss: 0.7495 - val_accuracy: 0.7949
Epoch 61/1000
270/270 [==============================] - 0s 72us/step - loss: 0.2608 - accuracy: 0.8667 - val_loss: 0.7566 - val_accuracy: 0.7949
Epoch 62/1000
270/270 [==============================] - 0s 67us/step - loss: 0.2638 - accuracy: 0.8556 - val_loss: 0.7593 - val_accuracy: 0.8034
Epoch 63/1000
270/270 [==============================] - 0s 70us/step - loss: 0.2619 - accuracy: 0.8630 - val_loss: 0.7432 -

Epoch 113/1000
270/270 [==============================] - 0s 76us/step - loss: 0.2650 - accuracy: 0.8704 - val_loss: 0.7471 - val_accuracy: 0.8034
Epoch 114/1000
270/270 [==============================] - 0s 66us/step - loss: 0.2651 - accuracy: 0.8630 - val_loss: 0.7542 - val_accuracy: 0.7949
Epoch 115/1000
270/270 [==============================] - 0s 64us/step - loss: 0.2638 - accuracy: 0.8667 - val_loss: 0.7423 - val_accuracy: 0.7949
Epoch 116/1000
270/270 [==============================] - 0s 66us/step - loss: 0.2635 - accuracy: 0.8630 - val_loss: 0.7463 - val_accuracy: 0.7949
Epoch 117/1000
270/270 [==============================] - 0s 81us/step - loss: 0.2621 - accuracy: 0.8630 - val_loss: 0.7429 - val_accuracy: 0.8034
Epoch 118/1000
270/270 [==============================] - 0s 91us/step - loss: 0.2618 - accuracy: 0.8630 - val_loss: 0.7463 - val_accuracy: 0.7949
Epoch 119/1000
270/270 [==============================] - 0s 75us/step - loss: 0.2653 - accuracy: 0.8593 - val_loss: 0

Epoch 224/1000
270/270 [==============================] - 0s 216us/step - loss: 0.2626 - accuracy: 0.8741 - val_loss: 0.7328 - val_accuracy: 0.8120
Epoch 225/1000
270/270 [==============================] - 0s 218us/step - loss: 0.2659 - accuracy: 0.8630 - val_loss: 0.7390 - val_accuracy: 0.8120
Epoch 226/1000
270/270 [==============================] - 0s 487us/step - loss: 0.2637 - accuracy: 0.8593 - val_loss: 0.7461 - val_accuracy: 0.8034
Epoch 227/1000
270/270 [==============================] - 0s 86us/step - loss: 0.2656 - accuracy: 0.8556 - val_loss: 0.7515 - val_accuracy: 0.8120
Epoch 228/1000
270/270 [==============================] - 0s 78us/step - loss: 0.2625 - accuracy: 0.8667 - val_loss: 0.7491 - val_accuracy: 0.8034
Epoch 229/1000
270/270 [==============================] - 0s 71us/step - loss: 0.2659 - accuracy: 0.8667 - val_loss: 0.7598 - val_accuracy: 0.8034
Epoch 230/1000
270/270 [==============================] - 0s 287us/step - loss: 0.2629 - accuracy: 0.8630 - val_los

270/270 [==============================] - 0s 190us/step - loss: 0.2644 - accuracy: 0.8593 - val_loss: 0.7522 - val_accuracy: 0.8120
Epoch 335/1000
270/270 [==============================] - 0s 133us/step - loss: 0.2621 - accuracy: 0.8630 - val_loss: 0.7624 - val_accuracy: 0.7949
Epoch 336/1000
270/270 [==============================] - 0s 77us/step - loss: 0.2619 - accuracy: 0.8667 - val_loss: 0.7891 - val_accuracy: 0.8034
Epoch 337/1000
270/270 [==============================] - 0s 86us/step - loss: 0.2655 - accuracy: 0.8630 - val_loss: 0.7730 - val_accuracy: 0.8034
Epoch 338/1000
270/270 [==============================] - 0s 76us/step - loss: 0.2624 - accuracy: 0.8704 - val_loss: 0.7596 - val_accuracy: 0.8034
Epoch 339/1000
270/270 [==============================] - 0s 77us/step - loss: 0.2654 - accuracy: 0.8593 - val_loss: 0.7494 - val_accuracy: 0.8120
Epoch 340/1000
270/270 [==============================] - 0s 302us/step - loss: 0.2626 - accuracy: 0.8519 - val_loss: 0.7543 - val_

...

270/270 [==============================] - 0s 77us/step - loss: 0.2658 - accuracy: 0.8667 - val_loss: 0.7849 - val_accuracy: 0.8120
Epoch 501/1000
270/270 [==============================] - 0s 86us/step - loss: 0.2635 - accuracy: 0.8556 - val_loss: 0.7676 - val_accuracy: 0.8034
Epoch 502/1000
270/270 [==============================] - 0s 61us/step - loss: 0.2629 - accuracy: 0.8630 - val_loss: 0.7706 - val_accuracy: 0.8205
Epoch 503/1000
270/270 [==============================] - 0s 89us/step - loss: 0.2635 - accuracy: 0.8593 - val_loss: 0.7692 - val_accuracy: 0.8120
Epoch 504/1000
270/270 [==============================] - 0s 147us/step - loss: 0.2638 - accuracy: 0.8667 - val_loss: 0.7754 - val_accuracy: 0.8120
Epoch 505/1000
270/270 [==============================] - 0s 138us/step - loss: 0.2633 - accuracy: 0.8630 - val_loss: 0.7723 - val_accuracy: 0.8120
Epoch 506/1000
270/270 [==============================] - 0s 87us/step - loss: 0.2663 - accuracy: 0.8667 - val_loss: 0.7703 - val_a

270/270 [==============================] - 0s 109us/step - loss: 0.2615 - accuracy: 0.8630 - val_loss: 0.7817 - val_accuracy: 0.8120
Epoch 611/1000
270/270 [==============================] - 0s 97us/step - loss: 0.2626 - accuracy: 0.8704 - val_loss: 0.7874 - val_accuracy: 0.8034
Epoch 612/1000
270/270 [==============================] - 0s 83us/step - loss: 0.2656 - accuracy: 0.8630 - val_loss: 0.7787 - val_accuracy: 0.8120
Epoch 613/1000
270/270 [==============================] - 0s 75us/step - loss: 0.2648 - accuracy: 0.8556 - val_loss: 0.7985 - val_accuracy: 0.8034
Epoch 614/1000
270/270 [==============================] - 0s 77us/step - loss: 0.2596 - accuracy: 0.8630 - val_loss: 0.7866 - val_accuracy: 0.8034
Epoch 615/1000
270/270 [==============================] - 0s 91us/step - loss: 0.2587 - accuracy: 0.8667 - val_loss: 0.7774 - val_accuracy: 0.8120
Epoch 616/1000
270/270 [==============================] - 0s 65us/step - loss: 0.2655 - accuracy: 0.8556 - val_loss: 0.7676 - val_ac

Epoch 721/1000
270/270 [==============================] - 0s 104us/step - loss: 0.2684 - accuracy: 0.8444 - val_loss: 0.8144 - val_accuracy: 0.7692
Epoch 722/1000
270/270 [==============================] - 0s 77us/step - loss: 0.2630 - accuracy: 0.8667 - val_loss: 0.7831 - val_accuracy: 0.7949
Epoch 723/1000
270/270 [==============================] - 0s 102us/step - loss: 0.2653 - accuracy: 0.8593 - val_loss: 0.7782 - val_accuracy: 0.7949
Epoch 724/1000
270/270 [==============================] - 0s 127us/step - loss: 0.2652 - accuracy: 0.8593 - val_loss: 0.7868 - val_accuracy: 0.8120
Epoch 725/1000
270/270 [==============================] - 0s 95us/step - loss: 0.2603 - accuracy: 0.8667 - val_loss: 0.8116 - val_accuracy: 0.7949
Epoch 726/1000
270/270 [==============================] - 0s 97us/step - loss: 0.2623 - accuracy: 0.8667 - val_loss: 0.8037 - val_accuracy: 0.7863
Epoch 727/1000
270/270 [==============================] - 0s 77us/step - loss: 0.2607 - accuracy: 0.8667 - val_loss

Epoch 832/1000
270/270 [==============================] - 0s 89us/step - loss: 0.2596 - accuracy: 0.8667 - val_loss: 0.8174 - val_accuracy: 0.7863
Epoch 833/1000
270/270 [==============================] - 0s 90us/step - loss: 0.2601 - accuracy: 0.8593 - val_loss: 0.7942 - val_accuracy: 0.7949
Epoch 834/1000
270/270 [==============================] - 0s 66us/step - loss: 0.2606 - accuracy: 0.8481 - val_loss: 0.7824 - val_accuracy: 0.8034
Epoch 835/1000
270/270 [==============================] - 0s 100us/step - loss: 0.2608 - accuracy: 0.8630 - val_loss: 0.7883 - val_accuracy: 0.8034
Epoch 836/1000
270/270 [==============================] - 0s 118us/step - loss: 0.2630 - accuracy: 0.8630 - val_loss: 0.7838 - val_accuracy: 0.8034
Epoch 837/1000
270/270 [==============================] - 0s 372us/step - loss: 0.2627 - accuracy: 0.8593 - val_loss: 0.7932 - val_accuracy: 0.8034
Epoch 838/1000
270/270 [==============================] - 0s 281us/step - loss: 0.2623 - accuracy: 0.8630 - val_los

270/270 [==============================] - 0s 86us/step - loss: 0.2697 - accuracy: 0.8481 - val_loss: 0.8538 - val_accuracy: 0.7863
Epoch 943/1000
270/270 [==============================] - 0s 92us/step - loss: 0.2642 - accuracy: 0.8667 - val_loss: 0.8102 - val_accuracy: 0.7949
Epoch 944/1000
270/270 [==============================] - 0s 199us/step - loss: 0.2609 - accuracy: 0.8630 - val_loss: 0.7889 - val_accuracy: 0.8120
Epoch 945/1000
270/270 [==============================] - 0s 97us/step - loss: 0.2625 - accuracy: 0.8481 - val_loss: 0.7938 - val_accuracy: 0.8120
Epoch 946/1000
270/270 [==============================] - 0s 60us/step - loss: 0.2595 - accuracy: 0.8593 - val_loss: 0.8154 - val_accuracy: 0.7949
Epoch 947/1000
270/270 [==============================] - 0s 172us/step - loss: 0.2600 - accuracy: 0.8667 - val_loss: 0.8206 - val_accuracy: 0.7949
Epoch 948/1000
270/270 [==============================] - 0s 249us/step - loss: 0.2621 - accuracy: 0.8630 - val_loss: 0.8040 - val_

In [213]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist2_over3.history['accuracy'])*100))

over-sampling train accuracy: 86.29%


In [54]:
df_proba7 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_lasso_2.xlsx",
                        sheet_name=2,
                        index_col=None)

In [55]:
df_proba7

,phage,strain,phenotype,prediction,0,1,2
0,p0006kpresabs_qual,NRS210,0,0,6.132076e-01,2.812180e-01,1.055744e-01
1,p0006kpresabs_qual,NRS205,2,2,1.993202e-04,6.834937e-07,9.998000e-01
2,p0006kpresabs_qual,312,2,1,3.589463e-01,3.982787e-01,2.427750e-01
3,p0006kpresabs_qual,GA15,2,1,3.589463e-01,3.982787e-01,2.427750e-01
4,p0006kpresabs_qual,SR4035,0,1,3.589463e-01,3.982787e-01,2.427750e-01
...,...,...,...,...,...,...,...
984,p0017Skpresabs_qual,NRS383,1,0,5.477194e-01,4.522807e-01,1.761374e-08
985,p0017Skpresabs_qual,NRS218,1,1,6.953657e-05,9.999305e-01,3.132419e-10
986,p0017Skpresabs_qual,NRS209,2,2,2.713214e-09,6.656316e-09,1.000000e+00
987,p0017Skpresabs_qual,SR2852,1,1,9.956684e-12,1.000000e+00,7.441288e-26


In [56]:
y_prob7 = df_proba7[df_proba7['phage']=='p0006presabs_qual'].iloc[:,-3:]
y_prob7 = y_prob7.to_numpy()
y_prob7

array([[7.11726900e-01, 2.88259600e-01, 1.35570260e-05],
       [8.64584450e-13, 3.12930700e-12, 1.00000000e+00],
       [4.42078150e-07, 2.02627070e-08, 9.99999500e-01],
       [1.05034110e-03, 8.90037600e-04, 9.98059600e-01],
       [9.98862000e-01, 1.13794590e-03, 1.81274480e-09],
       [1.82078290e-03, 1.85080750e-04, 9.97994200e-01],
       [9.99999760e-01, 6.61693800e-08, 8.31988100e-08],
       [9.55438100e-01, 4.45601600e-02, 1.73015540e-06],
       [9.99999900e-01, 8.78319100e-08, 9.45232800e-16],
       [9.99964830e-01, 4.30483030e-06, 3.08697930e-05],
       [9.20596800e-04, 9.98794900e-01, 2.84447680e-04],
       [9.99609530e-01, 3.89117660e-04, 1.47760730e-06],
       [9.99926700e-01, 7.32737000e-05, 1.31415840e-11],
       [9.99974700e-01, 2.20703840e-05, 3.25563630e-06],
       [9.20596800e-04, 9.98794900e-01, 2.84447680e-04],
       [3.35391100e-01, 6.57018500e-01, 7.59035670e-03],
       [2.65663800e-09, 2.10090070e-05, 9.99979000e-01],
       [9.99999170e-01, 8.43095

In [57]:
ovo7 = rocauc_ovo(y_sel_test_over, y_prob7, average="macro", multi_class="ovo")
ovo7

0.9037913653298268

In [58]:
ovr7 = rocauc_ovr(y_sel_test_over, y_prob7, average="macro", multi_class="ovr")
ovr7

0.9037913653298268

In [59]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_sel_train_over, X_sel_test_over, y_sel_train_over, y_sel_test_over = train_test_split(X_sel_over, y_sel_over,
                                                    test_size = 0.3,
                                                    random_state=890,
                                                    stratify=y_sel_over)

In [60]:
dat8 = pd.DataFrame(X_sel_test_over[:,-1])
dat8['test'] = y_sel_test_over

In [61]:
dat8

,0,test
0,NRS236,1
1,NRS113,2
2,CFBRSa23,0
3,NRS249,2
4,107,1
...,...,...
112,NY439,2
113,NRS106,0
114,221,0
115,NRS386,2


In [62]:
X_sel_train_over = X_sel_train_over[:,:-1]
X_sel_test_over = X_sel_test_over[:,:-1]

In [63]:
model2_over4 = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [123]:
model2_over4.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [124]:
model2_over4.fit(X_sel_train_over, y_sel_train_over,
          batch_size=64, epochs=1000,
          validation_data=(X_sel_test_over, y_sel_test_over))

Train on 270 samples, validate on 117 samples
Epoch 1/1000
270/270 [==============================] - 0s 467us/step - loss: 1.0203 - accuracy: 0.4630 - val_loss: 1.0255 - val_accuracy: 0.4274
Epoch 2/1000
270/270 [==============================] - 0s 56us/step - loss: 0.9843 - accuracy: 0.4444 - val_loss: 0.9999 - val_accuracy: 0.4444
Epoch 3/1000
270/270 [==============================] - 0s 79us/step - loss: 0.9620 - accuracy: 0.5185 - val_loss: 0.9818 - val_accuracy: 0.5043
Epoch 4/1000
270/270 [==============================] - 0s 85us/step - loss: 0.9404 - accuracy: 0.5407 - val_loss: 0.9678 - val_accuracy: 0.5726
Epoch 5/1000
270/270 [==============================] - 0s 108us/step - loss: 0.9204 - accuracy: 0.5889 - val_loss: 0.9587 - val_accuracy: 0.5556
Epoch 6/1000
270/270 [==============================] - 0s 90us/step - loss: 0.9033 - accuracy: 0.5889 - val_loss: 0.9492 - val_accuracy: 0.5556
Epoch 7/1000
270/270 [==============================] - 0s 95us/step - loss: 0.886

Epoch 57/1000
270/270 [==============================] - 0s 58us/step - loss: 0.6259 - accuracy: 0.7519 - val_loss: 0.8094 - val_accuracy: 0.6325
Epoch 58/1000
270/270 [==============================] - 0s 59us/step - loss: 0.6230 - accuracy: 0.7481 - val_loss: 0.8087 - val_accuracy: 0.5983
Epoch 59/1000
270/270 [==============================] - 0s 83us/step - loss: 0.6197 - accuracy: 0.7370 - val_loss: 0.8077 - val_accuracy: 0.6068
Epoch 60/1000
270/270 [==============================] - 0s 71us/step - loss: 0.6169 - accuracy: 0.7370 - val_loss: 0.8048 - val_accuracy: 0.6410
Epoch 61/1000
270/270 [==============================] - 0s 94us/step - loss: 0.6140 - accuracy: 0.7444 - val_loss: 0.8034 - val_accuracy: 0.6410
Epoch 62/1000
270/270 [==============================] - 0s 96us/step - loss: 0.6112 - accuracy: 0.7519 - val_loss: 0.8020 - val_accuracy: 0.6410
Epoch 63/1000
270/270 [==============================] - 0s 128us/step - loss: 0.6101 - accuracy: 0.7630 - val_loss: 0.7986 

Epoch 113/1000
270/270 [==============================] - 0s 81us/step - loss: 0.5118 - accuracy: 0.7926 - val_loss: 0.7767 - val_accuracy: 0.6752
Epoch 114/1000
270/270 [==============================] - 0s 94us/step - loss: 0.5100 - accuracy: 0.7963 - val_loss: 0.7762 - val_accuracy: 0.6667
Epoch 115/1000
270/270 [==============================] - 0s 78us/step - loss: 0.5091 - accuracy: 0.8037 - val_loss: 0.7771 - val_accuracy: 0.6667
Epoch 116/1000
270/270 [==============================] - 0s 94us/step - loss: 0.5076 - accuracy: 0.8000 - val_loss: 0.7772 - val_accuracy: 0.6667
Epoch 117/1000
270/270 [==============================] - 0s 111us/step - loss: 0.5062 - accuracy: 0.8000 - val_loss: 0.7755 - val_accuracy: 0.6667
Epoch 118/1000
270/270 [==============================] - 0s 105us/step - loss: 0.5043 - accuracy: 0.7963 - val_loss: 0.7765 - val_accuracy: 0.6667
Epoch 119/1000
270/270 [==============================] - 0s 73us/step - loss: 0.5046 - accuracy: 0.8000 - val_loss:

Epoch 169/1000
270/270 [==============================] - 0s 49us/step - loss: 0.4496 - accuracy: 0.8296 - val_loss: 0.7821 - val_accuracy: 0.6838
Epoch 170/1000
270/270 [==============================] - 0s 54us/step - loss: 0.4509 - accuracy: 0.8296 - val_loss: 0.7831 - val_accuracy: 0.6923
Epoch 171/1000
270/270 [==============================] - 0s 71us/step - loss: 0.4488 - accuracy: 0.8185 - val_loss: 0.7826 - val_accuracy: 0.7009
Epoch 172/1000
270/270 [==============================] - 0s 68us/step - loss: 0.4470 - accuracy: 0.8148 - val_loss: 0.7824 - val_accuracy: 0.7009
Epoch 173/1000
270/270 [==============================] - 0s 62us/step - loss: 0.4469 - accuracy: 0.8185 - val_loss: 0.7854 - val_accuracy: 0.7009
Epoch 174/1000
270/270 [==============================] - 0s 58us/step - loss: 0.4453 - accuracy: 0.8259 - val_loss: 0.7802 - val_accuracy: 0.7009
Epoch 175/1000
270/270 [==============================] - 0s 74us/step - loss: 0.4447 - accuracy: 0.8222 - val_loss: 0

Epoch 225/1000
270/270 [==============================] - 0s 59us/step - loss: 0.4096 - accuracy: 0.8296 - val_loss: 0.7859 - val_accuracy: 0.6923
Epoch 226/1000
270/270 [==============================] - 0s 52us/step - loss: 0.4072 - accuracy: 0.8370 - val_loss: 0.7887 - val_accuracy: 0.7009
Epoch 227/1000
270/270 [==============================] - 0s 53us/step - loss: 0.4064 - accuracy: 0.8407 - val_loss: 0.7922 - val_accuracy: 0.7009
Epoch 228/1000
270/270 [==============================] - 0s 53us/step - loss: 0.4074 - accuracy: 0.8407 - val_loss: 0.7882 - val_accuracy: 0.7009
Epoch 229/1000
270/270 [==============================] - 0s 60us/step - loss: 0.4056 - accuracy: 0.8519 - val_loss: 0.7871 - val_accuracy: 0.7009
Epoch 230/1000
270/270 [==============================] - 0s 57us/step - loss: 0.4060 - accuracy: 0.8556 - val_loss: 0.7859 - val_accuracy: 0.7009
Epoch 231/1000
270/270 [==============================] - 0s 73us/step - loss: 0.4047 - accuracy: 0.8519 - val_loss: 0

...

Epoch 672/1000
270/270 [==============================] - 0s 65us/step - loss: 0.2951 - accuracy: 0.8852 - val_loss: 0.9867 - val_accuracy: 0.7179
Epoch 673/1000
270/270 [==============================] - 0s 59us/step - loss: 0.3000 - accuracy: 0.8852 - val_loss: 0.9935 - val_accuracy: 0.7179
Epoch 674/1000
270/270 [==============================] - 0s 61us/step - loss: 0.2990 - accuracy: 0.8852 - val_loss: 0.9912 - val_accuracy: 0.7265
Epoch 675/1000
270/270 [==============================] - 0s 57us/step - loss: 0.2949 - accuracy: 0.8852 - val_loss: 0.9966 - val_accuracy: 0.7265
Epoch 676/1000
270/270 [==============================] - 0s 58us/step - loss: 0.2956 - accuracy: 0.8815 - val_loss: 0.9989 - val_accuracy: 0.7265
Epoch 677/1000
270/270 [==============================] - 0s 67us/step - loss: 0.3002 - accuracy: 0.8815 - val_loss: 1.0037 - val_accuracy: 0.7265
Epoch 678/1000
270/270 [==============================] - 0s 56us/step - loss: 0.2957 - accuracy: 0.8778 - val_loss: 1

Epoch 728/1000
270/270 [==============================] - 0s 62us/step - loss: 0.2907 - accuracy: 0.8852 - val_loss: 1.0093 - val_accuracy: 0.7265
Epoch 729/1000
270/270 [==============================] - 0s 79us/step - loss: 0.2890 - accuracy: 0.8852 - val_loss: 1.0109 - val_accuracy: 0.7265
Epoch 730/1000
270/270 [==============================] - 0s 64us/step - loss: 0.2908 - accuracy: 0.8852 - val_loss: 1.0117 - val_accuracy: 0.7179
Epoch 731/1000
270/270 [==============================] - 0s 60us/step - loss: 0.2906 - accuracy: 0.8852 - val_loss: 1.0183 - val_accuracy: 0.7179
Epoch 732/1000
270/270 [==============================] - 0s 61us/step - loss: 0.2901 - accuracy: 0.8889 - val_loss: 1.0287 - val_accuracy: 0.6752
Epoch 733/1000
270/270 [==============================] - 0s 64us/step - loss: 0.2933 - accuracy: 0.8815 - val_loss: 1.0272 - val_accuracy: 0.6752
Epoch 734/1000
270/270 [==============================] - 0s 63us/step - loss: 0.2925 - accuracy: 0.8778 - val_loss: 1

Epoch 784/1000
270/270 [==============================] - 0s 64us/step - loss: 0.2890 - accuracy: 0.8815 - val_loss: 1.0482 - val_accuracy: 0.7179
Epoch 785/1000
270/270 [==============================] - 0s 71us/step - loss: 0.2870 - accuracy: 0.8852 - val_loss: 1.0309 - val_accuracy: 0.7179
Epoch 786/1000
270/270 [==============================] - 0s 64us/step - loss: 0.2865 - accuracy: 0.8852 - val_loss: 1.0260 - val_accuracy: 0.7179
Epoch 787/1000
270/270 [==============================] - 0s 66us/step - loss: 0.2874 - accuracy: 0.8852 - val_loss: 1.0248 - val_accuracy: 0.7179
Epoch 788/1000
270/270 [==============================] - 0s 58us/step - loss: 0.2875 - accuracy: 0.8889 - val_loss: 1.0311 - val_accuracy: 0.7350
Epoch 789/1000
270/270 [==============================] - 0s 54us/step - loss: 0.2862 - accuracy: 0.8889 - val_loss: 1.0289 - val_accuracy: 0.7265
Epoch 790/1000
270/270 [==============================] - 0s 77us/step - loss: 0.2862 - accuracy: 0.8889 - val_loss: 1

Epoch 840/1000
270/270 [==============================] - 0s 62us/step - loss: 0.2871 - accuracy: 0.8815 - val_loss: 1.0569 - val_accuracy: 0.6752
Epoch 841/1000
270/270 [==============================] - 0s 54us/step - loss: 0.2873 - accuracy: 0.8889 - val_loss: 1.0361 - val_accuracy: 0.7265
Epoch 842/1000
270/270 [==============================] - 0s 54us/step - loss: 0.2872 - accuracy: 0.8889 - val_loss: 1.0351 - val_accuracy: 0.7436
Epoch 843/1000
270/270 [==============================] - 0s 67us/step - loss: 0.2870 - accuracy: 0.8815 - val_loss: 1.0387 - val_accuracy: 0.7436
Epoch 844/1000
270/270 [==============================] - 0s 70us/step - loss: 0.2873 - accuracy: 0.8815 - val_loss: 1.0439 - val_accuracy: 0.7436
Epoch 845/1000
270/270 [==============================] - 0s 69us/step - loss: 0.2851 - accuracy: 0.8815 - val_loss: 1.0481 - val_accuracy: 0.7350
Epoch 846/1000
270/270 [==============================] - 0s 70us/step - loss: 0.2817 - accuracy: 0.8889 - val_loss: 1

Epoch 951/1000
270/270 [==============================] - 0s 61us/step - loss: 0.2779 - accuracy: 0.8889 - val_loss: 1.0679 - val_accuracy: 0.7350
Epoch 952/1000
270/270 [==============================] - 0s 65us/step - loss: 0.2777 - accuracy: 0.8889 - val_loss: 1.0605 - val_accuracy: 0.7265
Epoch 953/1000
270/270 [==============================] - 0s 71us/step - loss: 0.2785 - accuracy: 0.8889 - val_loss: 1.0624 - val_accuracy: 0.7179
Epoch 954/1000
270/270 [==============================] - 0s 59us/step - loss: 0.2774 - accuracy: 0.8889 - val_loss: 1.0778 - val_accuracy: 0.7265
Epoch 955/1000
270/270 [==============================] - 0s 75us/step - loss: 0.2736 - accuracy: 0.8889 - val_loss: 1.0962 - val_accuracy: 0.7350
Epoch 956/1000
270/270 [==============================] - 0s 64us/step - loss: 0.2846 - accuracy: 0.8778 - val_loss: 1.1205 - val_accuracy: 0.6752
Epoch 957/1000
270/270 [==============================] - 0s 61us/step - loss: 0.2804 - accuracy: 0.8889 - val_loss: 1

In [243]:
acc_test2_over4 = model2_over4.evaluate(X_sel_test_over, y_sel_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test2_over4*100))

117/117 [==============================] - 0s 59us/step
over-sampling test accuracy: 71.79%


In [125]:
pred8 = model2_over4.predict_classes(X_sel_test_over)
pred8

array([2, 2, 0, 2, 0, 2, 0, 0, 0, 0, 0, 1, 1, 1, 0, 2, 2, 1, 2, 2, 1, 2,
       1, 2, 1, 1, 2, 0, 2, 1, 0, 2, 2, 1, 0, 0, 1, 2, 0, 2, 0, 1, 2, 2,
       2, 1, 2, 1, 1, 1, 2, 2, 2, 2, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 2,
       2, 1, 2, 2, 0, 1, 2, 0, 0, 0, 2, 2, 2, 0, 0, 0, 1, 0, 1, 0, 2, 0,
       0, 0, 0, 1, 0, 1, 2, 1, 0, 2, 0, 2, 1, 2, 1, 2, 0, 0, 2, 1, 0, 1,
       1, 1, 2, 0, 1, 1, 1])

In [126]:
dat8['pred'] = pred8
dat8

,0,test,pred
0,NRS236,1,2
1,NRS113,2,2
2,CFBRSa23,0,0
3,NRS249,2,2
4,107,1,0
...,...,...,...
112,NY439,2,2
113,NRS106,0,0
114,221,0,1
115,NRS386,2,1


In [127]:
proba8 = model2_over4.predict_proba(X_sel_test_over)
dat_proba8 = pd.DataFrame(proba8)

In [128]:
dat_proba8

,0,1,2
0,1.155347e-04,0.117173,0.882712
1,2.696634e-03,0.004637,0.992666
2,9.967662e-01,0.000017,0.003217
3,4.470928e-04,0.002181,0.997372
4,6.406378e-01,0.314578,0.044784
...,...,...,...
112,4.558951e-04,0.000234,0.999310
113,5.037265e-01,0.495453,0.000820
114,1.713146e-01,0.790131,0.038555
115,5.473031e-10,0.587913,0.412087


In [129]:
dat_proba8.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba8.csv", index = False,
         header=None)

In [130]:
dat8.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/8p006p.csv", index = False,
         header=None)

In [247]:
hist2_over4 = model2_over4.fit(X_sel_train_over, y_sel_train_over,
          batch_size=64, epochs=1000,
          validation_data=(X_sel_test_over, y_sel_test_over))

Train on 270 samples, validate on 117 samples
Epoch 1/1000
270/270 [==============================] - 0s 78us/step - loss: 0.2557 - accuracy: 0.8889 - val_loss: 0.8965 - val_accuracy: 0.7009
Epoch 2/1000
270/270 [==============================] - 0s 76us/step - loss: 0.2570 - accuracy: 0.8889 - val_loss: 0.9051 - val_accuracy: 0.7094
Epoch 3/1000
270/270 [==============================] - 0s 84us/step - loss: 0.2561 - accuracy: 0.8889 - val_loss: 0.9053 - val_accuracy: 0.7009
Epoch 4/1000
270/270 [==============================] - 0s 109us/step - loss: 0.2556 - accuracy: 0.8889 - val_loss: 0.9030 - val_accuracy: 0.7009
Epoch 5/1000
270/270 [==============================] - 0s 88us/step - loss: 0.2559 - accuracy: 0.8889 - val_loss: 0.9047 - val_accuracy: 0.7009
Epoch 6/1000
270/270 [==============================] - 0s 89us/step - loss: 0.2556 - accuracy: 0.8889 - val_loss: 0.9122 - val_accuracy: 0.7009
Epoch 7/1000
270/270 [==============================] - 0s 75us/step - loss: 0.2558

Epoch 57/1000
270/270 [==============================] - 0s 63us/step - loss: 0.2540 - accuracy: 0.8889 - val_loss: 0.9165 - val_accuracy: 0.7179
Epoch 58/1000
270/270 [==============================] - 0s 70us/step - loss: 0.2539 - accuracy: 0.8889 - val_loss: 0.9296 - val_accuracy: 0.6667
Epoch 59/1000
270/270 [==============================] - 0s 62us/step - loss: 0.2551 - accuracy: 0.8741 - val_loss: 0.9288 - val_accuracy: 0.7094
Epoch 60/1000
270/270 [==============================] - 0s 70us/step - loss: 0.2547 - accuracy: 0.8889 - val_loss: 0.9195 - val_accuracy: 0.7009
Epoch 61/1000
270/270 [==============================] - 0s 58us/step - loss: 0.2526 - accuracy: 0.8889 - val_loss: 0.9100 - val_accuracy: 0.7009
Epoch 62/1000
270/270 [==============================] - 0s 61us/step - loss: 0.2562 - accuracy: 0.8889 - val_loss: 0.9152 - val_accuracy: 0.7009
Epoch 63/1000
270/270 [==============================] - 0s 61us/step - loss: 0.2539 - accuracy: 0.8889 - val_loss: 0.9264 -

270/270 [==============================] - 0s 76us/step - loss: 0.2575 - accuracy: 0.8778 - val_loss: 0.9277 - val_accuracy: 0.6496
Epoch 114/1000
270/270 [==============================] - 0s 66us/step - loss: 0.2542 - accuracy: 0.8852 - val_loss: 0.9274 - val_accuracy: 0.6496
Epoch 115/1000
270/270 [==============================] - 0s 56us/step - loss: 0.2530 - accuracy: 0.8852 - val_loss: 0.9340 - val_accuracy: 0.6581
Epoch 116/1000
270/270 [==============================] - 0s 153us/step - loss: 0.2536 - accuracy: 0.8852 - val_loss: 0.9326 - val_accuracy: 0.7094
Epoch 117/1000
270/270 [==============================] - 0s 76us/step - loss: 0.2567 - accuracy: 0.8852 - val_loss: 0.9460 - val_accuracy: 0.6667
Epoch 118/1000
270/270 [==============================] - 0s 90us/step - loss: 0.2551 - accuracy: 0.8815 - val_loss: 0.9372 - val_accuracy: 0.7009
Epoch 119/1000
270/270 [==============================] - 0s 71us/step - loss: 0.2539 - accuracy: 0.8889 - val_loss: 0.9305 - val_ac

Epoch 169/1000
270/270 [==============================] - 0s 108us/step - loss: 0.2555 - accuracy: 0.8852 - val_loss: 0.9756 - val_accuracy: 0.6581
Epoch 170/1000
270/270 [==============================] - 0s 71us/step - loss: 0.2581 - accuracy: 0.8741 - val_loss: 0.9543 - val_accuracy: 0.7094
Epoch 171/1000
270/270 [==============================] - 0s 72us/step - loss: 0.2535 - accuracy: 0.8889 - val_loss: 0.9404 - val_accuracy: 0.7009
Epoch 172/1000
270/270 [==============================] - 0s 67us/step - loss: 0.2522 - accuracy: 0.8889 - val_loss: 0.9375 - val_accuracy: 0.7009
Epoch 173/1000
270/270 [==============================] - 0s 74us/step - loss: 0.2512 - accuracy: 0.8889 - val_loss: 0.9473 - val_accuracy: 0.7009
Epoch 174/1000
270/270 [==============================] - 0s 68us/step - loss: 0.2546 - accuracy: 0.8852 - val_loss: 0.9714 - val_accuracy: 0.6581
Epoch 175/1000
270/270 [==============================] - 0s 73us/step - loss: 0.2558 - accuracy: 0.8852 - val_loss: 

Epoch 225/1000
270/270 [==============================] - 0s 68us/step - loss: 0.2538 - accuracy: 0.8889 - val_loss: 0.9227 - val_accuracy: 0.7094
Epoch 226/1000
270/270 [==============================] - 0s 61us/step - loss: 0.2543 - accuracy: 0.8889 - val_loss: 0.9339 - val_accuracy: 0.7094
Epoch 227/1000
270/270 [==============================] - 0s 67us/step - loss: 0.2535 - accuracy: 0.8889 - val_loss: 0.9386 - val_accuracy: 0.7094
Epoch 228/1000
270/270 [==============================] - 0s 119us/step - loss: 0.2533 - accuracy: 0.8889 - val_loss: 0.9435 - val_accuracy: 0.7094
Epoch 229/1000
270/270 [==============================] - 0s 169us/step - loss: 0.2516 - accuracy: 0.8852 - val_loss: 0.9528 - val_accuracy: 0.7094
Epoch 230/1000
270/270 [==============================] - 0s 220us/step - loss: 0.2504 - accuracy: 0.8852 - val_loss: 0.9506 - val_accuracy: 0.6581
Epoch 231/1000
270/270 [==============================] - 0s 223us/step - loss: 0.2509 - accuracy: 0.8778 - val_los

...

Epoch 558/1000
270/270 [==============================] - 0s 74us/step - loss: 0.2480 - accuracy: 0.8852 - val_loss: 1.0032 - val_accuracy: 0.7179
Epoch 559/1000
270/270 [==============================] - 0s 67us/step - loss: 0.2470 - accuracy: 0.8889 - val_loss: 0.9990 - val_accuracy: 0.7094
Epoch 560/1000
270/270 [==============================] - 0s 57us/step - loss: 0.2469 - accuracy: 0.8889 - val_loss: 0.9928 - val_accuracy: 0.7094
Epoch 561/1000
270/270 [==============================] - 0s 69us/step - loss: 0.2476 - accuracy: 0.8889 - val_loss: 0.9895 - val_accuracy: 0.7179
Epoch 562/1000
270/270 [==============================] - 0s 66us/step - loss: 0.2479 - accuracy: 0.8852 - val_loss: 0.9982 - val_accuracy: 0.7265
Epoch 563/1000
270/270 [==============================] - 0s 91us/step - loss: 0.2478 - accuracy: 0.8889 - val_loss: 1.0079 - val_accuracy: 0.7265
Epoch 564/1000
270/270 [==============================] - 0s 144us/step - loss: 0.2479 - accuracy: 0.8889 - val_loss: 

270/270 [==============================] - 0s 68us/step - loss: 0.2490 - accuracy: 0.8889 - val_loss: 1.0229 - val_accuracy: 0.7094
Epoch 669/1000
270/270 [==============================] - 0s 59us/step - loss: 0.2523 - accuracy: 0.8778 - val_loss: 1.0334 - val_accuracy: 0.6667
Epoch 670/1000
270/270 [==============================] - 0s 66us/step - loss: 0.2488 - accuracy: 0.8852 - val_loss: 1.0336 - val_accuracy: 0.6581
Epoch 671/1000
270/270 [==============================] - 0s 64us/step - loss: 0.2467 - accuracy: 0.8963 - val_loss: 1.0305 - val_accuracy: 0.7094
Epoch 672/1000
270/270 [==============================] - 0s 61us/step - loss: 0.2478 - accuracy: 0.8889 - val_loss: 1.0181 - val_accuracy: 0.7179
Epoch 673/1000
270/270 [==============================] - 0s 81us/step - loss: 0.2473 - accuracy: 0.8889 - val_loss: 1.0159 - val_accuracy: 0.7179
Epoch 674/1000
270/270 [==============================] - 0s 223us/step - loss: 0.2482 - accuracy: 0.8889 - val_loss: 1.0159 - val_ac

Epoch 724/1000
270/270 [==============================] - 0s 97us/step - loss: 0.2468 - accuracy: 0.8889 - val_loss: 1.0224 - val_accuracy: 0.7179
Epoch 725/1000
270/270 [==============================] - 0s 72us/step - loss: 0.2490 - accuracy: 0.8889 - val_loss: 1.0227 - val_accuracy: 0.7179
Epoch 726/1000
270/270 [==============================] - 0s 75us/step - loss: 0.2476 - accuracy: 0.8889 - val_loss: 1.0146 - val_accuracy: 0.7179
Epoch 727/1000
270/270 [==============================] - 0s 77us/step - loss: 0.2462 - accuracy: 0.8889 - val_loss: 1.0179 - val_accuracy: 0.7094
Epoch 728/1000
270/270 [==============================] - 0s 136us/step - loss: 0.2464 - accuracy: 0.8889 - val_loss: 1.0279 - val_accuracy: 0.7094
Epoch 729/1000
270/270 [==============================] - 0s 88us/step - loss: 0.2464 - accuracy: 0.8889 - val_loss: 1.0261 - val_accuracy: 0.7094
Epoch 730/1000
270/270 [==============================] - 0s 132us/step - loss: 0.2458 - accuracy: 0.8889 - val_loss:

Epoch 835/1000
270/270 [==============================] - 0s 226us/step - loss: 0.2485 - accuracy: 0.8852 - val_loss: 1.0286 - val_accuracy: 0.7094
Epoch 836/1000
270/270 [==============================] - 0s 185us/step - loss: 0.2462 - accuracy: 0.8889 - val_loss: 1.0379 - val_accuracy: 0.7094
Epoch 837/1000
270/270 [==============================] - 0s 182us/step - loss: 0.2483 - accuracy: 0.8852 - val_loss: 1.0398 - val_accuracy: 0.6581
Epoch 838/1000
270/270 [==============================] - 0s 111us/step - loss: 0.2486 - accuracy: 0.8852 - val_loss: 1.0312 - val_accuracy: 0.7094
Epoch 839/1000
270/270 [==============================] - 0s 111us/step - loss: 0.2473 - accuracy: 0.8889 - val_loss: 1.0192 - val_accuracy: 0.7350
Epoch 840/1000
270/270 [==============================] - 0s 111us/step - loss: 0.2482 - accuracy: 0.8778 - val_loss: 1.0221 - val_accuracy: 0.7179
Epoch 841/1000
270/270 [==============================] - 0s 75us/step - loss: 0.2474 - accuracy: 0.8889 - val_l

Epoch 946/1000
270/270 [==============================] - 0s 63us/step - loss: 0.2472 - accuracy: 0.8889 - val_loss: 1.0573 - val_accuracy: 0.7179
Epoch 947/1000
270/270 [==============================] - 0s 49us/step - loss: 0.2485 - accuracy: 0.8889 - val_loss: 1.0373 - val_accuracy: 0.7179
Epoch 948/1000
270/270 [==============================] - 0s 57us/step - loss: 0.2484 - accuracy: 0.8889 - val_loss: 1.0388 - val_accuracy: 0.7179
Epoch 949/1000
270/270 [==============================] - 0s 78us/step - loss: 0.2463 - accuracy: 0.8889 - val_loss: 1.0442 - val_accuracy: 0.7265
Epoch 950/1000
270/270 [==============================] - 0s 197us/step - loss: 0.2460 - accuracy: 0.8889 - val_loss: 1.0534 - val_accuracy: 0.7265
Epoch 951/1000
270/270 [==============================] - 0s 101us/step - loss: 0.2454 - accuracy: 0.8852 - val_loss: 1.0498 - val_accuracy: 0.7179
Epoch 952/1000
270/270 [==============================] - 0s 72us/step - loss: 0.2462 - accuracy: 0.8889 - val_loss:

In [248]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist2_over4.history['accuracy'])*100))

over-sampling train accuracy: 88.70%


In [64]:
df_proba8 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_lasso_2.xlsx",
                        sheet_name=3,
                        index_col=None)

In [65]:
df_proba8

,phage,strain,phenotype,prediction,0,1,2
0,p0006kpresabs_qual,NRS236,1,2,1.321970e-02,2.446264e-01,7.421539e-01
1,p0006kpresabs_qual,NRS113,2,2,3.478230e-02,2.806685e-01,6.845492e-01
2,p0006kpresabs_qual,CFBRSa23,0,0,4.090251e-01,3.405008e-01,2.504741e-01
3,p0006kpresabs_qual,NRS249,2,1,1.987907e-01,5.331044e-01,2.681049e-01
4,p0006kpresabs_qual,107,1,0,4.090251e-01,3.405008e-01,2.504741e-01
...,...,...,...,...,...,...,...
984,p0017Skpresabs_qual,CFBRSa30,0,0,7.207667e-01,2.792331e-01,2.571588e-07
985,p0017Skpresabs_qual,NRS383,1,0,6.129044e-01,3.870795e-01,1.601290e-05
986,p0017Skpresabs_qual,NRS110,2,2,3.260306e-07,7.910664e-07,9.999989e-01
987,p0017Skpresabs_qual,NRS209,2,2,3.604249e-12,2.698129e-07,9.999998e-01


In [66]:
y_prob8 = df_proba8[df_proba8['phage']=='p0006presabs_qual'].iloc[:,-3:]
y_prob8 = y_prob8.to_numpy()
y_prob8

array([[1.15534710e-04, 1.17172600e-01, 8.82711900e-01],
       [2.69663380e-03, 4.63731350e-03, 9.92666070e-01],
       [9.96766200e-01, 1.70009880e-05, 3.21681200e-03],
       [4.47092750e-04, 2.18084270e-03, 9.97372030e-01],
       [6.40637760e-01, 3.14578120e-01, 4.47840900e-02],
       [8.74220600e-05, 7.78390700e-04, 9.99134240e-01],
       [6.40637760e-01, 3.14578120e-01, 4.47840900e-02],
       [8.07282270e-01, 1.87858800e-01, 4.85888900e-03],
       [9.99445260e-01, 5.21134000e-04, 3.36222840e-05],
       [9.88914430e-01, 1.10523130e-02, 3.32677000e-05],
       [6.40637760e-01, 3.14578120e-01, 4.47840900e-02],
       [1.11698620e-05, 9.83830900e-01, 1.61578600e-02],
       [1.23514020e-01, 4.96119680e-01, 3.80366300e-01],
       [3.46699200e-01, 5.35712360e-01, 1.17588505e-01],
       [9.95891000e-01, 3.70014830e-03, 4.08814730e-04],
       [2.70234360e-08, 1.19652910e-10, 1.00000000e+00],
       [1.93955090e-04, 1.48521110e-02, 9.84953900e-01],
       [1.23514020e-01, 4.96119

In [67]:
ovo8 = rocauc_ovo(y_sel_test_over, y_prob8, average="macro", multi_class="ovo")
ovo8

0.8668639053254438

In [68]:
ovr8 = rocauc_ovr(y_sel_test_over, y_prob8, average="macro", multi_class="ovr")
ovr8

0.8668639053254438

In [69]:
ovos2 = [ovo5, ovo6, ovo7, ovo8]
np.mean(ovos2)

0.8620973044049968

In [70]:
np.std(ovos2)

0.02947260633480655

In [71]:
ovrs2 = [ovr5, ovr6, ovr7, ovr8]
np.mean(ovrs2)

0.8620973044049968

In [72]:
np.std(ovrs2)

0.02947260633480655

In [249]:
accs_l_over = [acc_test2_over, acc_test2_over2, acc_test2_over3, acc_test2_over4]

In [250]:
mean_l_over = np.mean(accs_l_over)
print('over-sampling test accuracy mean after lasso: %.2f%%' % (mean_l_over*100))

over-sampling test accuracy mean after lasso: 74.57%


In [251]:
std_l_over = np.std(accs_l_over)
print('over-sampling test accuracy standard deviation after lasso:', std_l_over)

over-sampling test accuracy standard deviation after lasso: 0.026602347982728397


In [252]:
accs_train_l_over = [np.mean(hist2_over.history['accuracy']), np.mean(hist2_over2.history['accuracy']), np.mean(hist2_over3.history['accuracy']),
             np.mean(hist2_over4.history['accuracy'])]

In [253]:
mean_train_l_over = np.mean(accs_train_l_over)
print('over-sampling train accuracy mean after lasso: %.2f%%' % (mean_train_l_over*100))

over-sampling train accuracy mean after lasso: 87.38%


In [254]:
std_train_l_over = np.std(accs_train_l_over)
print('over-sampling train accuracy standard deviation after lasso:', std_train_l_over)

over-sampling train accuracy standard deviation after lasso: 0.009234734
